# Gamma Trench2 Data Preparation

# <div align = "center" style = "color:rgb(50, 255, 0);"> Applied Loans </div>

# Define Libraries

In [1]:

# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os

import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd



path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)


# Constant

In [2]:
CURRENT_DATE = datetime.now().strftime("%Y%m%d")
print(f"The current date is: {CURRENT_DATE}")

The current date is: 20250916


# <div align="left" style="color:rgb(51, 250, 250);"> Functions </div>

## <div align="left" style="color:rgb(51, 250, 250);"> Save the data to google clound storage </div>

In [3]:
def save_df_to_gcs(df, bucket_name, destination_blob_name, file_format='csv'):
    """Saves a pandas DataFrame to Google Cloud Storage.

    Args:
        df: The pandas DataFrame to save.
        bucket_name: The name of the GCS bucket.
        destination_blob_name: The name of the blob to be created.
        file_format: The file format to save the DataFrame in ('csv' or 'parquet').
    """

    # Create a temporary file
    if file_format == 'csv':
        temp_file = 'temp.csv'
        df.to_csv(temp_file, index=False)
    elif file_format == 'parquet':
        temp_file = 'temp.parquet'
        df.to_parquet(temp_file, index=False)
    else:
        raise ValueError("Invalid file format. Please choose 'csv' or 'parquet'.")

    # Upload the file to GCS
    storage_client = storage.Client(project="prj-prod-dataplatform")

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(temp_file)

    # Remove the temporary file
    import os
    os.remove(temp_file)
    


## <div align="left" style="color:rgb(51, 250, 250);"> Read the Data from Google Cloud Storage </div>

In [4]:
def read_df_from_gcs(bucket_name, source_blob_name, file_format='csv'):
    """Reads a DataFrame from Google Cloud Storage.

    Args:
        bucket_name: The name of the GCS bucket.
        source_blob_name: The name of the blob to read.
        file_format: The file format to read ('csv' or 'parquet').

    Returns:
        pandas.DataFrame: The data loaded from the GCS file.
    """
    # Create a temporary file name
    temp_file = f'temp.{file_format}'
    
    try:
        # Initialize GCS client
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(source_blob_name)

        # Download the file to a temporary location
        blob.download_to_filename(temp_file)

        # Read the file into a DataFrame
        if file_format == 'csv':
            df = pd.read_csv(temp_file, low_memory=False)
        elif file_format == 'parquet':
            df = pd.read_parquet(temp_file)
        else:
            raise ValueError("Invalid file format. Please choose 'csv' or 'parquet'.")

        return df

    finally:
        # Clean up the temporary file
        if os.path.exists(temp_file):
            os.remove(temp_file)

## <div align = "left" style="color:rgb(51, 250, 250);"> Data Quality Report </div>

In [5]:
def data_quality_report(df, target_col='ln_fspd30_flag'):
    # Initialize an empty list to store each row of data
    report_data = []
    # Iterate over each column in the DataFrame to compute metrics
    for col in df.columns:
        # Determine the data type of the column
        data_type = df[col].dtype
       
        # Calculate the number of missing values in the column
        missing_values = df[col].isnull().sum()
       
        # Calculate the percentage of missing values relative to the total number of rows
        missing_percentage = (missing_values / len(df)) * 100
       
        # Calculate the number of unique values in the column
        unique_values = df[col].nunique()
       
        # Calculate the percentage of non-missing values
        non_missing_percentage = ((len(df) - missing_values) / len(df)) * 100
       
        # Check if the column is numeric to compute additional metrics
        if pd.api.types.is_numeric_dtype(df[col]):
            # Compute minimum, maximum, mean, median, mode, mode percentage, standard deviation, and quantiles
            min_value = df[col].min()
            max_value = df[col].max()
            mean_value = df[col].mean()
            median_value = df[col].median()
            mode_value = df[col].mode().iloc[0] if not df[col].mode().empty else None
            mode_percentage = (df[col] == mode_value).sum() / len(df) * 100 if mode_value is not None else None
            std_dev = df[col].std()
            quantile_25 = df[col].quantile(0.25)
            quantile_50 = df[col].quantile(0.50)  # Same as median
            quantile_75 = df[col].quantile(0.75)
            
            # Calculate the Interquartile Range (IQR)
            iqr = quantile_75 - quantile_25
            
            # Calculate Skewness and Kurtosis
            skewness = df[col].skew()
            kurtosis = df[col].kurt()
            
            # Calculate Coefficient of Variation (CV) - standardized measure of dispersion
            cv = (std_dev / mean_value) * 100 if mean_value != 0 else None
            
            # Calculate correlation with target variable if target exists in dataframe
            if target_col in df.columns and col != target_col and pd.api.types.is_numeric_dtype(df[target_col]):
                # Calculate correlation only using rows where both columns have non-null values
                correlation = df[[col, target_col]].dropna().corr().iloc[0, 1]
            else:
                correlation = None
        else:
            # Assign None for non-numeric columns where appropriate
            min_value = None
            max_value = None
            mean_value = None
            median_value = None
            mode_value = df[col].mode().iloc[0] if not df[col].mode().empty else None
            mode_percentage = (df[col] == mode_value).sum() / len(df) * 100 if mode_value is not None else None
            std_dev = None
            quantile_25 = None
            quantile_50 = None
            quantile_75 = None
            iqr = None
            skewness = None
            kurtosis = None
            cv = None
            correlation = None
       
        # Append the computed metrics for the current column to the list
        report_data.append({
            'Column': col,
            'Data Type': data_type,
            'Missing Values': missing_values,
            'Missing Percentage': missing_percentage,
            'Unique Values': unique_values,
            'Min': min_value,
            'Max': max_value,
            'Mean': mean_value,
            'Median': median_value,
            'Mode': mode_value,
            'Mode Percentage': mode_percentage,
            'Std Dev': std_dev,
            'Non-missing Percentage': non_missing_percentage,
            '25% Quantile': quantile_25,
            '50% Quantile': quantile_50,
            '75% Quantile': quantile_75,
            'IQR': iqr,
            'Skewness': skewness,
            'Kurtosis': kurtosis,
            'CV (%)': cv,
            f'Correlation with {target_col}': correlation
        })
    # Create the DataFrame from the list of dictionaries
    report = pd.DataFrame(report_data)
   
    # Return the complete data quality report DataFrame
    return report

# <div align = "left" style="color:rgb(51,250,250);"> Upload pickle file to Google Cloud Storage Bucke </div>

In [6]:
def upload_to_gcs(bucket_name, source_file_path, destination_blob_name):
    """Uploads a file to Google Cloud Storage"""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    blob.upload_from_filename(source_file_path)
    print(f"File {source_file_path} uploaded to {bucket_name}/{destination_blob_name}")

In [7]:
import pickle
import io
from google.cloud import storage
def save_pickle_to_gcs(data, bucket_name, destination_blob_name):
    """
    Save any Python object as a pickle file to Google Cloud Storage
    
    Args:
        data: The Python object to pickle (DataFrame, dict, list, etc.)
        bucket_name: Name of the GCS bucket
        destination_blob_name: Path/filename in the bucket
    """
    # Initialize the GCS client
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    # Serialize the data to pickle format in memory
    pickle_buffer = io.BytesIO()
    pickle.dump(data, pickle_buffer)
    pickle_buffer.seek(0)
    
    # Upload the pickle data to GCS
    blob.upload_from_file(pickle_buffer, content_type='application/octet-stream')
    print(f"Pickle file uploaded to gs://{bucket_name}/{destination_blob_name}")

# Applied Table

In [8]:
schema1 = 'risk_mart'

OBDATE = '2025-08-18'
SDATE = '2023-01-01'
EDATE = '2025-08-18'
# change it to obdate from current date if the requirement is for a specific date

formatted_date = datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')

print(formatted_date)  # Output: 20230101



al = f'applied_loans_{datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')}_{datetime.strptime(EDATE, '%Y-%m-%d').strftime('%Y%m%d')}'
print(al)



20230101
applied_loans_20230101_20250818


# Applied Queries

In [9]:
sq = f"""  
create or replace table {schema1}.{al} as
select distinct customerId,
digitalLoanAccountId,
loanAccountNumber,
onboarding_datetime onb_tsa_onboarding_datetime,
first_name onb_first_name,
middle_name onb_middle_name,
last_name onb_last_name,
onb_age,
gender onb_gender,
onb_mobile_no,
email onb_email,
email_verified_flag onb_email_verified_flag,
place_of_birth onb_place_of_birth,
doctype onb_doc_type,          
onb_country,
onb_province,
onb_city,
onb_barangay,
onb_postalcode,
onb_latitude,
onb_longitude,
onb_osversion,        
kyc_status onb_kyc_status,
datetime(timestamp(kyc_status_change_dt), 'Asia/Manila') onb_kyc_status_upgrade_datetime,
--kyc_status_change_dt onb_kyc_status_upgrade_datetime,
user_type ln_user_type,
ln_loan_level_user_type ln_loan_level_user_type_old,
--ln_loan_level_user_type_v1 ln_loan_level_user_type,
if(loan_type in ('Flex-up','Reloan') ,'1_Repeat Applicant',ln_loan_level_user_type_v1) ln_loan_level_user_type,
loan_type ln_loan_type,
loan_product_type ln_prod_type,
flg_applied_loan ln_loan_applied_flag,
fatca ln_facta_flag,
flg_reject_rule_check ln_dl_rule_reject_flag,
ln_taran_rule_reject_flag,
flg_taran_reject_flag ln_taran_scorecard_reject_flag,
flg_cdd_reject_flag ln_cdd_reject_flag,
ln_marked_underwriter_check_flag,
flg_hard_reject_flag ln_underwriting_reject_flag, 
if(flg_disbursed_loan=1,1,flg_loan_approved_not_disbursed) ln_final_approved_flag,
flg_disbursed_loan ln_disb_flag,
flg_loan_approved_not_disbursed ln_approved_not_disb_flag,
loan_submission_date ln_appln_submit_datetime,
disbursementDateTime ln_disb_dtime,
loan_chosen_principal ln_chosen_principal,
loan_chosen_tenor ln_chosen_tenor,
loan_approved_principal ln_approved_principal,
loan_approved_tenor ln_approved_tenor,
main.onb_cnt_ongoing_loans ln_cnt_ongoing_tdb_loans,
onb_tot_ongoing_loans_emi ln_tot_ongoing_tdb_loans_emi,
loan_purpose ln_purpose,
vas_opted_flag ln_vas_opted_flag,
vas_used_flag ln_vas_used_flag,
loan_age ln_age,
loan_mobile_no ln_mobile_no,
loan_alternate_mobile_no ln_alt_mobile_no,
osversion_v2 ln_osversion,
brand ln_brand,
ln_os_type,
ln_model,
coalesce(ln_email,email) ln_email,
coalesce(loan_address,addressline) ln_address,
coalesce(loan_province,province) ln_province,
coalesce(loan_city,city) ln_city,
coalesce(loan_barangay,barangay) ln_barangay,
coalesce(cast(loan_postal_code as string),postalcode) ln_postal_code,
loan_latitude ln_latitude,
loan_longitude ln_longitude,
loan_docType ln_doc_type,
loan_docNumber ln_doc_number,
if(loan_marital_status='Live-in Partner','With a Live-in Partner',loan_marital_status) ln_marital_status,
--case when loan_dependents_count in ('No dependents') then null else loan_dependents_count end ln_cnt_dependents,
loan_dependents_count ln_cnt_dependents,
loan_education_level ln_education_level,
coalesce(purpose_source_of_fund_name,main.source_funds) ln_source_funds,
--coalesce(source_funds_new.source_funds_new,main.source_funds) ln_source_funds_new,
case when coalesce(purpose_source_of_fund_name,main.source_funds) in ('Others') then null
when coalesce(purpose_source_of_fund_name,main.source_funds) in ('Commissions') then null else coalesce(purpose_source_of_fund_name,main.source_funds) end as ln_source_funds_new,
coalesce(purpose_employment_status_name,employmentstatus) ln_employment_type,
--coalesce(employment_type_new.employment_type_new,main.employment_type) ln_employment_type_new,
case when coalesce(purpose_employment_status_name,employmentstatus)='Employed' and Mod(cast(customerId as int64),10)<3 then 'Employed - Govt. Employee/Govt. Official' 
when coalesce(purpose_employment_status_name,employmentstatus)='Employed' and Mod(cast(customerId as int64),10)>2 then'Employed - Private Employee' 
else coalesce(purpose_employment_status_name,employmentstatus) end as ln_employment_type_new,
-- case when upper(coalesce(purpose_employment_status_name,main.employment_type)) like'%EMPLOYED%' and Mod(cast(customerId as int64),10)<3 then 'Employed - Govt. Employee/Govt. Official'
-- when upper(coalesce(purpose_employment_status_name,main.employment_type)) like'%EMPLOYED%' and Mod(cast(customerId as int64),10)>2 then'Employed - Private Employee'
-- else coalesce(purpose_employment_status_name,main.employment_type) end as ln_employment_type_new,
--main.employment_type_new ln_employment_type_new,
coalesce(purpose_nature_of_work_name,natureofwork) ln_nature_of_work,
--coalesce(nature_of_work_new.nature_of_work_new,main.nature_of_work) ln_nature_of_work_new,
case when coalesce(purpose_nature_of_work_name,natureofwork) in ('Admin/Clerical Staff/Officer') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Architect/Interior Decorator') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Bank Employee') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Broker/Dealer') then  'Broker/Dealer'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Call Center Agent/Tele Marketer') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Cashier/Food Server/Waiter') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Consultant') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Doctor/Dentist/Medical Professional') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Engineer') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Entertainer/Artist') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Foreign Ambassador/Diplomat') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Gov Employee') then  'Govt Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Govt Employee') then  'Govt Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Insurance Agent/Financial Advisor') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('IT Professional') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Lawyer') then  'Licensed Professional - Lawyer'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Military/Armed Forces/Police/Navy/Airforce') then  'Government - Uniformed Service'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Newscaster/Media Personality') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Pilot/Airline Crew') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Seafarer') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Teacher/Instructor/Coach') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Utility Personnel/Household Help') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Accountant/Auditor') then  'Licensed Professional - Accountant'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Utilities (Electricity Gas and Water)') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Wholesale and Retail Trade') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Sales/Marketing Personnel') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Other Professional services') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Other Non Professional Services') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Owner') then  'Owner'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Religious Leader') then  'Religious Leader'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Fire Fighter') then  'Government - Uniformed Service'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Farmer/Fisherman') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Debt Collection/Recovery Agents/ Collection Agency Employee') then  'Debt Collection/Recovery Agents/ Collection Agency Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Court Personnel/Bailiff') then  'Court Personnel/Bailiff'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Store/Service Manager') then  'Manager/Executive'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Sales Marketing Professional') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Barber / Beauty Salon') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Business Owner (MSME)') then  'Owner'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Business Owner') then  'Owner'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Delivery Rider/Driver') then  'Delivery Rider/Driver'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Delivery Rider / Driver') then  'Delivery Rider / Driver'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Manpower Agency') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Online Freelancing / Virtual Assistant / Online teacher') then  'Virtual/Online/Remote Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Online Seller') then  'Virtual/Online/Remote Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Real Estate agent') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Skilled Worker') then  'Skilled Worker'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Technician (Cellphone / Laptop)') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Vendor (Sari-Sari store Wet market etc)') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Staff/Rank and File') then  'Staff/Rank and File'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Officer/Supervisor') then  'Officer/Supervisor'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Manager/Executive') then  'Manager/Executive'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Licensed Professional - Lawyer') then  'Licensed Professional - Lawyer'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Licensed Professional - Accountant') then  'Licensed Professional - Accountant'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Licensed Professional - Others') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Religious Minister') then  'Religious Minister'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Government - Elected Official') then  'Government - Elected Official'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Government - Uniformed Service') then  'Government - Uniformed Service'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Director/Trustee/Stockholder') then  'Director/Trustee/Stockholder'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Virtual/Online/Remote Employee') then  'Virtual/Online/Remote Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Government - Appointed Official') then  'Government - Appointed Official'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('AIR FORCE') then  'Military/Armed Forces/Police/Navy/Airforce'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('NAVY') then  'Military/Armed Forces/Police/Navy/Airforce'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Admin/Secretarial') then  'Admin/Clerical Staff/Officer'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Sales/Marketing Professional') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('COURT-PERSONNEL/BAILIFFS') then  'Court Personnel/Bailiff'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('DEBT COLLECTION/RECOVERY AGENTS/COLLECTION AGENCY EMPLOYEE') then  'Debt Collection/Recovery Agents/ Collection Agency Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Accountant') then  'Licensed Professional - Accountant' else coalesce(purpose_nature_of_work_name,natureofwork) end as ln_nature_of_work_new,
if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) ln_industry,
case when main.source_funds in ('Remittance','Pension','Inheritance/Retirement Funds','Interest on Deposits and Investments','Allowance') then null 
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Wholesale and Retail Trade') then 'Wholesale and Retail Trade'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Utilities (Electricity, Gas and Water)') then 'Utilities (Electricity, Gas and Water)'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Transportation') then 'Transportation'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Tourism, Hotel and Resorts') then 'Tourism, Hotel and Resorts'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('TeleCo') then 'Telecommunication'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Telecommunication') then 'Telecommunication'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Religious Organization') then 'Religious Organization'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Entertainment and Media') then 'Recreation (Arts, Entertainment/Media, Sports)'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Real Estate') then 'Real Estate'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Pawnshop/Remittance Agent/Virtual Currency') then 'Pawnshop/Remittance Agent/Virtual Currency'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Others') then 'Others'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Non Government Organization/Foundation') then 'NGOs/NonProfit Organization/Foundation'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Military and Weaponry Business') then 'Military and Weaponry Business'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Maritime/Shipping') then 'Maritime/Shipping'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Manufacturing - Food') then 'Manufacturing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Manufacturing - Non Food') then 'Manufacturing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Manpower/Manning Agencies') then 'Manpower/Manning Agencies'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Legal Service/Law Firm') then 'Legal Service/Law Firm'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Insurance Activities') then 'Insurance Activities'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Hospital and Medical Services') then 'Hospital and Medical Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Govt. Institution') then 'Govt. Institution'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Gold, Jewelry and Precious Metals') then 'Gold/Jewelry/Precious Metals/Art/Antique Dealership'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Online Gaming/Lottery/Casino') then 'Gambling and Betting (Online Gaming, Lottery, Casino)'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Freight and Cargo') then 'Freight and Cargo'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Food Industry/Restaurants/Fastfoods') then 'Food Industry/Restaurants/Fastfoods'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Fin Tech/Financial Services') then 'Fin Tech/Financial Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('FX Dealer/Money Changer') then 'FX Dealer/Money Changer'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Education/Schools') then 'Education/Private Schools'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ("Convenience Store/Internet Cafe's") then "Convenience Store/Internet Cafe's"
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Construction') then 'Construction'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Computer/IT Related Services') then 'Computer/IT Related Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Business Process Outsourcing/Tele Marketing') then 'Business Process Outsourcing/Tele Marketing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Beauty and Health Services') then 'Beauty and Health Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Health and Beauty Services') then 'Beauty and Health Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Banking') then 'Banking'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Architecture/Engineering') then 'Architecture/Engineering'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Airline/Aviation') then 'Airline/Aviation'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Agribusiness/Animal Farming/Fishing') then 'Agribusiness/Animal Farming/Fishing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Advertising/Sales/Marketing') then 'Advertising/Sales/Marketing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Accounting/Bookkeeping') then 'Accounting/Bookkeeping/Auditing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) is null then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Leasing and Rental Services') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Repair, Sanitation and Housekeeping') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Repair, Sanitation and Housekeeping') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Embassy') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Recycling') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Publishing and Printing') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('IT services','ONLINE SELLER','business owner','Human Resources','Head of sales','Human Resource Management',
'Digital Marketing','Food services','Manufacture of Leather and Related Products','Manufacture of Other Non-Metallic Mineral Products',
'Businesswoman','Business Owner','Agency Manpower','Advertising and marketing','Manufacture of Coke and Refined Petroleum Products',
'Sales Coffee Trading','Pharmaceutical','Manufacture of Furniture','Software as a Service','Nurse','Manufacture of Rubber Products','Shared Services',
'Waste and recycling solutions','Event Supplier','Fishing and Aquaculture','Manufacture of Motor Vehicles, Trailers and Semi-Trailers',
'Crop and Animal Production, Hunting and Related Service Activities; Forestry and Logging','Publishing Activities',
'Manufacture of Basic Metals','Manufacture of Textiles','Manufacture of Electrical Equipment',
'Water Collection, Treatment and Supply','Water Transport','Manufacture of Basic Pharmaceutical Products and Pharmaceutical Preparation',
'Manufacture of Chemicals and Chemical Products','Insurance, Reinsurance and Pension Funding except Compulsory Social Security',
'Electricity, Gas, Steam and Air Conditioning Supply','Manufacture of Machinery and Equipment, n.e.c.',
'Travel Agency, Tour Operator, Reservation Service and Related Activities','Medical Transcription  Activities',
'Manufacture of Fabricated Metal Products except Machinery and Equipment','Manufacture of Plastic Products',
'Postal and Courier Activities','Real Estate Activities','Printing and Reproduction of Recorded Media',
'Manufacture of Computer, Electronic and Optical Products','Transport via Buses',
'Religious Organization ','Designated NonFinancial Businesses ','Manufacture of Wearing Apparel',
'Warehousing and Support Activities for Transportation','Architectural and Engineering Activities and Related Technical Consultancy',
'Wholesale  and Retail Trade and Repair of Motor Vehicles and Motorcycles','Designated NonFinancial Businesses',
'Human Health Activities except Public Health Activities','Manufacture of Food Products',
'Military and Weaponry Business','Retail Trade except of Motor Vehicles and Motorcycles',
'Land Transport and Transport via Pipelines except  Transport via Buses','Education except Public Education',
'Financial Service Activities except Insurance, Pension Funding and Central Banking','Wholesale Trade except of Motor Vehicles and Motorcycles',
'Accounting, Bookkeeping and Auditing Activities; Tax Consultancy','Accommodation and Food Service Activities',
'NGOs/NonProfit Organization/Foundation ','Mining and Quarrying ','Computer Programming, Consultancy and Related Activities; Information Service Activities',
'Call Center Activities (Voice)','Back-Office Operations Activities (Non-Voice) Except Medical Transcription Activities',
'Nightclubs/Discos/Karaoke Center','Import Export Companies ','Mining and Quarrying','Import Export Companies',
'Gambling and Betting (Online Gaming, Lottery, Casino) ','Other Personal Services Activities','FX Dealer/Money Changer',
'Religious Organization','Securities/Holding Companies ','Car/Boat/Plane/Used Truck/AutoParts Manufacturing/Dealership or Auction Houses') then null
else if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) end as ln_industry_new, 
--purpose_source_of_fund_name,purpose_nature_of_work_name,purpose_employment_status_name,
--coalesce(industry_description_new.industry_description_new,if(pupose_industryid is not null , coalesce(purpose_industryName,if(purpose_created_dt>='2025-05-20',purpose_industryName,industry)),if(purpose_created_dt>='2025-05-20',purpose_industryName,industry))) ln_industry_new,
loan_companyName ln_company_name,
loan_self_dec_income ln_self_dec_income,
loan_salary_scaled_income ln_salary_scaled_income,
loan_ref_type1 ln_ref1_type,
loan_ref_type2 ln_ref2_type,
flg_cic_called cic_called_flag,
flg_cic_hit cic_hit_flag,
ScoreRange cic_score_range,
demo_score old_demo_score,
cic_score old_cic_score,
credo_score old_gen_credo_score,
trusting_social_score ts_trusting_social_score,
bureau_score bu_bureau_score,
c_credo_score,s_credo_score,fu_credo_score,r_credo_score,gen_credo_score,
flg_mature_fpd10 ln_mature_fpd10_flag,
flg_mature_fpd30 ln_mature_fpd30_flag,
flg_mature_fspd_30 ln_mature_fspd30_flag,
flg_mature_fstpd_30 ln_mature_fstpd30_flag,
deffpd10 ln_fpd10_flag,
deffpd30 ln_fpd30_flag,
deffspd30 ln_fspd30_flag,
deffstpd30 ln_fstpd30_flag,
--loan_defFPD30 ln_defFPD30,
loan_defSPD30 ln_spd30_flag,loan_defTPD30 ln_tpd30_flag,
defpd10_outstanding_principal ln_fpd10_os_principal,defpd30_outstanding_principal ln_fpd30_os_principal,
defspd30_outstanding_principal ln_fspd30_os_principal,deftpd30_outstanding_principal ln_fstpd30_os_principal,
credolabRefNumber,
datetime(credo_inquiry_date,'Asia/Manila') credo_inquiry_date,                              
status cust_status_flag,
onb_self_dec_income,
onb_company_name,
cust_status_close_date

from 
(with educationtype AS (
        SELECT
        loan.digitalLoanAccountId,loanAccountNumber, loan.loanPurposeId,   education_id,
        description AS education_level
        FROM  dl_loans_db_raw.tdbk_digital_loan_application loan
        LEFT JOIN  dl_loans_db_raw.tdbk_loan_purpose purpose
        ON   loan.loanPurposeId = purpose.loanPurposeId
        LEFT JOIN dl_loans_db_raw.tdbk_loan_lov_mtb
        ON education_id = id AND module = 'Education'
         ),

ref_type as (with Ref_type1 as (select  digitalLoanAccountId,relationship_id , description AS loan_ref_type1 from dl_loans_db_raw.tdbk_loan_refernce_details A
        LEFT JOIN dl_loans_db_raw.tdbk_loan_lov_mtb B  ON A.relationship_id = B.id
 where refPrefrenceOrder='1' qualify row_number() over (partition by digitalLoanAccountId order by refCreatedDateAndTime desc) =1),
    Ref_type2 as (select  digitalLoanAccountId,relationship_id , description AS loan_ref_type2 from dl_loans_db_raw.tdbk_loan_refernce_details A
        LEFT JOIN dl_loans_db_raw.tdbk_loan_lov_mtb B  ON A.relationship_id = B.id
 where refPrefrenceOrder='2' qualify row_number() over (partition by digitalLoanAccountId order by refCreatedDateAndTime desc) =1)
select Ref_type1.digitalLoanAccountId,loan_ref_type1,loan_ref_type2 from Ref_type1 join Ref_type2 on Ref_type1.digitalLoanAccountId=Ref_type2.digitalLoanAccountId) ,

user_type_min as (select customerId,min(case 
when digitalLoanAccountId is null then '4_Never_start_loan_appln'
when application_submit_date is null and rn1_application_status <=1 then '4_Never_start_loan_appln'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status is null then '3_Applied_Not_Disbursed'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status =1 then '2_New Applicant'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status >1 then '1_Repeat Applicant'

else '5_others' end ) user_type from
(select A.*,B.rn_disburement_status from
(select digitalLoanAccountId,customerId,applicationStatus, coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime)) application_submit_date,disbursementDateTime,
row_number() over (partition by customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))) rn1_application_status,
from risk_credit_mis.loan_master_table) A

left join 

(select digitalLoanAccountId,customerId,applicationStatus, disbursementDateTime,
case when disbursementDateTime is not null then 
row_number() over (partition by customerId order by disbursementDateTime) else 0 end rn_disburement_status,
from risk_credit_mis.loan_master_table where  disbursementDateTime is not null) B on A.digitalLoanAccountId=B.digitalLoanAccountId) 
--order by A.customerId 
group by 1 ),

user_type_loan_level as (select customerId,digitalLoanAccountId,min(case 
when digitalLoanAccountId is null then '4_Never_start_loan_appln'
when application_submit_date is null and rn1_application_status <=1 then '4_Never_start_loan_appln'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status is null then '3_Applied_Not_Disbursed'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status =1 then '2_New Applicant'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status >1 then '1_Repeat Applicant'

else '5_others' end ) user_type from
(select A.*,B.rn_disburement_status from
(select digitalLoanAccountId,customerId,applicationStatus, coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime)) application_submit_date,disbursementDateTime,
row_number() over (partition by customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))) rn1_application_status,
from risk_credit_mis.loan_master_table
where if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '{SDATE}' and '2025-08-31'
) A

left join 

(select digitalLoanAccountId,customerId,applicationStatus, disbursementDateTime,
case when disbursementDateTime is not null then 
row_number() over (partition by customerId order by disbursementDateTime) else 0 end rn_disburement_status,
from risk_credit_mis.loan_master_table where  disbursementDateTime is not null and  if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '{SDATE}' and '2025-08-31') B on A.digitalLoanAccountId=B.digitalLoanAccountId) 

group by 1 ,2 order by customerId),

user_type_loan_level_v1 as (select customerId,digitalLoanAccountId,disbursementDateTime,application_submit_date,rn_disburement_status,first_disbursment_loan,  min(case 
when digitalLoanAccountId is null then '4_Never_start_loan_appln'
when application_submit_date is null and rn1_application_status <=1 then '4_Never_start_loan_appln'
--when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status is null then '3_Applied_Not_Disbursed'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status =1 and first_disbursment_loan > application_submit_date then '2_New Applicant'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status >=1  then '1_Repeat Applicant'
when application_submit_date is not null and rn1_application_status >=1  and  first_disbursment_loan < application_submit_date  then '1_Repeat Applicant'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status is null then '2_New Applicant'

else '5_others' end ) user_type from
(select A.*,
B.rn_disburement_status ,C.first_disbursment_loan,
from
(select digitalLoanAccountId,customerId,applicationStatus, coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime)) application_submit_date,disbursementDateTime,
row_number() over (partition by customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))) rn1_application_status,
from risk_credit_mis.loan_master_table
where if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '{SDATE}' and '2025-08-31'
) A

left join 

(select digitalLoanAccountId,customerId,applicationStatus, disbursementDateTime,
case when disbursementDateTime is not null then 
row_number() over (partition by customerId order by disbursementDateTime) else 0 end rn_disburement_status,
from risk_credit_mis.loan_master_table where  disbursementDateTime is not null and  credolabRefNumber is not null and if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '{SDATE}' and '2025-08-31') B on A.digitalLoanAccountId=B.digitalLoanAccountId 

left join 

(select min(disbursementDateTime) first_disbursment_loan,customerId,
from risk_credit_mis.loan_master_table where  disbursementDateTime is not null and  if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '{SDATE}' and '2025-08-31'
group by customerId) C on A.customerId=C.customerId
) 
--where customerId in (1668511,2480643,2027516,1802996,2469891,2418795)

group by 1 ,2,3,4,5,6 order by customerId ,application_submit_date),


deliquency as
(select loanAccountNumber,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30  in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),

cust_info as (
with cust_base as(
select * from `dl_customers_db_raw.tdbk_customer_mtb`
where cust_id is not null
--and status in ('C','A')
--and date(created_dt,'Asia/Manila') >= '{SDATE}'
),
first_loan_application as (
select custId,digitalLoanAccountId,loanaccountnumber,isTermsAndConditionstimestamp,poi1RejectType,loantype from
--(select * from 
`dl_loans_db_raw.tdbk_digital_loan_application`
--where custId in (select cast(cust_id as integer) from cust_base) qualify row_number() over (partition by custId order by created_dt) = 1)
--where --loanType in ('TUL') and coalesce(flowType,'1') <> '2'
),
loan_calculator as (
select custid,digitalLoanAccountId from
(SELECT fla.custid,fla.digitalLoanAccountId,
sum(case when a.applicationStatus='INITIATE' then 1 else 0 end) loan_calculator
 FROM `dl_loans_db_raw.tdbk_status_trace`  a
 join first_loan_application fla
 on fla.digitalLoanAccountId=a.digitalLoanAccountId
 group by 1,2)
where loan_calculator>0
),
disbursed_undisbursed_loan as (
SELECT fla.custid,fla.digitalLoanAccountId,
 FROM `dl_loans_db_raw.tdbk_status_trace`  a
 join first_loan_application fla
 on fla.digitalLoanAccountId=a.digitalLoanAccountId
 where a.applicationStatus ='APPROVED'
 group by 1,2
)


select 
 cb.*, fla.digitalLoanAccountId,--onboarding_datetime, 
 fla.loantype,
case when lc.custid is not null then 1 else 0 end as flg_loan_calculator,
case when fla.poi1RejectType is not null or fla.loantype is null then 1 else 0 end as flg_reject_rule_check,
case when (fla.isTermsAndConditionstimestamp is not null or loan.new_loan_type ='Flex-up')  then 1 else 0 end as flg_applied_loan,
case when ul.custid is not null then 1 else 0 end as flg_approved_scorecard_loan,
case when fla.loanaccountnumber is not null then 1 else 0 end as flg_disbursed_loan,
fla.loanaccountnumber as loan_id
from cust_base cb
left join first_loan_application fla
on cast(cb.cust_id as integer)=fla.custId
left join loan_calculator lc
--on cast(cb.cust_id as integer)=lc.custid and 
on lc.digitalLoanAccountId=fla.digitalLoanAccountId
left join disbursed_undisbursed_loan ul
--on cast(cb.cust_id as integer)=ul.custid)
on ul.digitalLoanAccountId=fla.digitalLoanAccountId
left join 
risk_credit_mis.loan_master_table loan on loan.digitalLoanAccountId=fla.digitalLoanAccountId),                                                                                                  


default_outstanding_principal as (with all_default_account as (select digitalLoanAccountId,loanAccountNumber,firstDueDate,secondDueDate,thirdDueDate,defFPD30,defSPD30,defTPD30 from 
`risk_credit_mis.loan_master_table`
where defFPD10=1 or defFPD30=1 or defSPD30=1 or defTPD30=1),

defpd10_pricipal_outstanding as (
select loanAccountNumber,firstDueDate,PRINCIPALARREARS defpd10_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(firstDueDate ,INTERVAL 10 DAY)=core._Partitiondate or Date_add(firstDueDate ,INTERVAL 11 DAY)=core._Partitiondate)
where defFPD10=1 qualify row_number() over (partition by loanAccountNumber order by defpd10_outstanding_principal desc )=1),

defpd30_pricipal_outstanding as (
select loanAccountNumber,firstDueDate,PRINCIPALARREARS defpd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(firstDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(firstDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defFPD30=1 qualify row_number() over (partition by loanAccountNumber order by defpd30_outstanding_principal desc )=1),

defspd30_pricipal_outstanding as (
select loanAccountNumber,secondDueDate,PRINCIPALARREARS defspd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(secondDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(secondDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defSPD30=1 qualify row_number() over (partition by loanAccountNumber order by defspd30_outstanding_principal desc )=1),

deftpd30_pricipal_outstanding as (
select loanAccountNumber,thirdDueDate,PRINCIPALARREARS deftpd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(thirdDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(thirdDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defTPD30=1 qualify row_number() over (partition by loanAccountNumber order by deftpd30_outstanding_principal desc )=1)

select digitalLoanAccountId,allaccount.loanAccountNumber,defpd10_outstanding_principal,defpd30_outstanding_principal,defspd30_outstanding_principal,deftpd30_outstanding_principal,
allaccount.firstDueDate,allaccount.secondDueDate,allaccount.thirdDueDate,defFPD30 loan_defFPD30,defSPD30 loan_defSPD30,defTPD30 loan_defTPD30  from all_default_account allaccount
left join defpd10_pricipal_outstanding day10dpd on allaccount.loanAccountNumber=day10dpd.loanAccountNumber
left join defpd30_pricipal_outstanding firstdpd on allaccount.loanAccountNumber=firstdpd.loanAccountNumber
left join defspd30_pricipal_outstanding seconddpd on allaccount.loanAccountNumber=seconddpd.loanAccountNumber
left join deftpd30_pricipal_outstanding thirddpd on allaccount.loanAccountNumber=thirddpd.loanAccountNumber),

osversion_credolab as (select * from (
(select distinct deviceId,'android'||generalInfo.release as credolab_osversion ,GeneralInfo.model onb_model from credolab_raw.android_credolab_datasets_struct_columns
union all 
select distinct deviceId,'ios'||generalInfo.release as credolab_osversion ,GeneralInfo.model onb_model from  credolab_raw.ios_credolab_datasets_struct_columns))),

ln_model_credolab as (select * from (
(select distinct deviceId, GeneralInfo.model ln_model  from credolab_raw.android_credolab_datasets_struct_columns
union all 
select distinct deviceId,GeneralInfo.model ln_model from  credolab_raw.ios_credolab_datasets_struct_columns))),

onb_self_dec_income as 
(select custmtb.cust_id,monthly_income,company_name from 
`prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_mtb`  custmtb
join `prj-prod-dataplatform.dl_customers_db_raw.tdbk_cust_profile_mtb` profile on custmtb.user_id=profile.user_id
where monthly_income is not null),

cust_closed_date as (select OFCUSTOMERID,OFDATEOPENED, IF(OFDATECLOSED='1970-01-01' , NULL, OFDATECLOSED) OFDATECLOSED from `prj-prod-dataplatform.core_raw.customer_accounts`
join `dl_customers_db_raw.tdbk_customer_mtb` mtb on mtb.cust_id=OFCUSTOMERID
 where  CRINTERCODE = 'POC20000'
qualify row_number() over (partition by OFCUSTOMERID order by OFDATECLOSED desc) =1 and mtb.status='C'),

                                 
credolab_creation as (select --min(A.created_dt),min(B.createdOn)
digitalLoanAccountId,credolabRefNumber,A.userId,loanDigitalAppliedDateAndTime,A.created_dt,B.createdOn,A.clCreditScore,B.creditScore 
from `dl_loans_db_raw.tdbk_digital_loan_application` A
join dl_loans_db_raw.tdbk_credolab_track B on refno=credolabRefNumber qualify row_number() over(partition by credolabRefNumber,digitalLoanAccountId order by createdOn desc )=1),

credo_score_static_insght as (select * from (select 'static' as source, loan.digitalLoanAccountId,reference_number referenceNumber,datetime(credo_inquiry_date) requestedDate,null score_all_score,generic_score score_all_probability,
null score_sil_score,sil_score score_sil_probability, null score_quick_score,quick_score score_quick_probability,null score_flex_score,flexup_score score_flex_probability,null score_reloan_score, reloan_score score_reloan_probability from worktable_data_analysis.dema_score_final_may12 dima
left join risk_credit_mis.loan_master_table loan on dima.reference_number=loan.credolabRefNumber

union all

select 'credotrace' as source, loan.digitalLoanAccountId,referenceNumber referenceNumber,datetime(requestedDate) requestedDate,score_all_score,cast (score_all_probability as float64) score_all_probability,
score_sil_score,cast (score_sil_probability as float64) score_sil_probability,score_quick_score,cast(score_quick_probability as float64) score_quick_probability,score_flex_score,cast(score_flex_probability as float64) score_flex_probability,score_reloan_score,cast(score_reloan_probability as float64) score_reloan_probability from risk_credit_mis.credolab_trace_insight_all_loans  dima
left join risk_credit_mis.loan_master_table loan on dima.referenceNumber=loan.credolabRefNumber

union all
select 'insight' as source , digitalLoanAccountId,referenceNumber,datetime(requestedDate) requestedDate,score_all_score,cast (score_all_probability as float64) score_all_probability,
score_sil_score,cast (score_sil_probability as float64) score_sil_probability,score_quick_score,cast(score_quick_probability as float64) score_quick_probability,score_flex_score,cast(score_flex_probability as float64) score_flex_probability,score_reloan_score,cast(score_reloan_probability as float64) score_reloan_probability  from 

(
(select digitalLoanAccountId,referenceNumber,datetime(requestedDate) requestedDate,score_all_score,score_all_probability,
score_sil_score,score_sil_probability,score_quick_score,score_quick_probability,score_flex_score,score_flex_probability,score_reloan_score,score_reloan_probability from credolab_raw.credolab_insights_android
union all 
select digitalLoanAccountId,referenceNumber,requestedDate,score_all_score,score_all_probability,
score_sil_score,score_sil_probability,score_quick_score,score_quick_probability,score_flex_score,score_flex_probability,score_reloan_score,score_reloan_probability from credolab_raw.credolab_insights_ios))
qualify row_number() over(partition by digitalLoanAccountId order by requestedDate desc)  =1)
qualify row_number() over(partition by digitalLoanAccountId order by requestedDate)  =1)

select loanmaster.customerId,
cust_mtb.email as email,
cust_mtb.email_status as email_verified_flag,
cust_mtb.birthplace as place_of_birth,
cust_profile.first_nm first_name,
cust_profile.middle_nm middle_name,
cust_profile.last_nm last_name,
hm_city as onb_city,
hm_province as onb_province,
hm_postalcode as onb_postalcode,
hm_barangay as onb_barangay,
hm_country as onb_country,
latitude onb_latitude,
longitude onb_longitude,
DATE_DIFF(DATE(datetime(cust_mtb.created_dt,'Asia/Manila')), parse_date('%d %b %Y', cust_mtb.dob), YEAR) - IF(EXTRACT(MONTH FROM parse_date('%d %b %Y', cust_mtb.dob))*100 + EXTRACT(DAY FROM parse_date('%d %b %Y', cust_mtb.dob)) > EXTRACT(MONTH FROM DATE(datetime(cust_mtb.created_dt,'Asia/Manila')))*100 + EXTRACT(DAY FROM DATE(datetime(cust_mtb.created_dt,'Asia/Manila'))),1,0) as onb_age,
DATE_DIFF(DATE(if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)), parse_date('%d %b %Y', cust_mtb.dob), YEAR) - IF(EXTRACT(MONTH FROM parse_date('%d %b %Y', cust_mtb.dob))*100 + EXTRACT(DAY FROM parse_date('%d %b %Y', cust_mtb.dob)) > EXTRACT(MONTH FROM DATE(if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)))*100 + EXTRACT(DAY FROM DATE(if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime))),1,0) as loan_age,
cust_mtb.gender,
if (user_type_min.user_type='1_Repeat Applicant' or (new_loan_type in ('Flex-up') and decision_date is not null),1,0) onb_cnt_ongoing_loans,
loanmaster.digitalLoanAccountId,loanmaster.loanAccountNumber,datetime(cust_mtb.created_dt,'Asia/Manila') onboarding_datetime,
cust_mtb.mobile_no onb_mobile_no, mobileNo loan_mobile_no,case when length(digitalloan.alternateNumber)=10 then concat('63',digitalloan.alternateNumber) else digitalloan.alternateNumber end loan_alternate_mobile_no,loanmaster.purposeDescription loan_purpose,
source_fund.description AS source_funds,
emp_status.description AS employment_type,
nat.description AS nature_of_work,
industry.value industry_description,
loanmaster.companyName loan_companyName,
loanmaster.maritalStatus loan_marital_status,
loanmaster.dependentsCount loan_dependents_count,
educationtype.education_level loan_education_level,
if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime) loan_submission_date,
cust_mtb.kyc_status,
cust_mtb.kyc_status_change_dt,
loan_ref_type1,loan_ref_type2,
loanmaster.disbursementDateTime,
loan_level.addressline,
loan_level.province,
loan_level.city,
loan_level.barangay,
loan_level.postalcode,
loanaddress.full_address loan_address,
loanaddress.province loan_province,
loanaddress.city loan_city,
loanaddress.barangay loan_barangay,
loanaddress.postal_code loan_postal_code,
SPLIT(loan_level.geolocation, ' | ')[SAFE_OFFSET(0)] AS loan_latitude,
SPLIT(loan_level.geolocation, ' | ')[SAFE_OFFSET(1)] AS loan_longitude,
loan_level.docType loan_docType,
loan_level.docNumber loan_docNumber,
user_type_min.user_type,
user_type_loan_level.user_type ln_loan_level_user_type,
user_type_loan_level_v1.user_type ln_loan_level_user_type_v1,
loanmaster.new_loan_type,
case when loanmaster.new_loan_type ='Flex-up' and reloan_flag=1 then 'Reloan' else new_loan_type end loan_type,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
loanmaster.osversion_v2 osversion_v2,
if(lower(loanmaster.osversion_v2) like 'ios%','Apple', loan_level.brand) brand,
if(lower(loanmaster.osversion_v2) like 'ios%','iOS','Android') ln_os_type,
loanmaster.monthlyIncome loan_self_dec_income,
incomeperSalaryScale loan_salary_scaled_income,
case when vasflag.digitalLoanAccountId is not null then 1 else 0 end vas_used_flag,
if(poi3.crifApplicationId is not null,1,0) flg_cic_called,
flagmatched.matched_flag flg_cic_hit,
summary.ScoreRange,
--summary_matchedflag flg_cic_hit,
loanmaster.demographicScore demo_score,poi3.cicScorecard cic_score,loanmaster.credolabScore credo_score,trustingsocial.score trusting_social_score,poi3.bureauScore bureau_score,
--quick_score c_credo_score,sil_score s_credo_score,flexup_score fu_credo_score,reloan_score r_credo_score,generic_score gen_credo_score,
score_quick_probability c_credo_score,score_sil_probability s_credo_score,score_flex_probability fu_credo_score,score_reloan_probability r_credo_score,score_all_probability gen_credo_score,
if (user_type_min.user_type='1_Repeat Applicant' or (new_loan_type in ('Flex-up') and decision_date is not null),loanaccts.monthlytermamount,0) as onb_tot_ongoing_loans_emi,
empstatus.employment_type_new,
deliquency.flg_mature_fpd10,
deliquency.flg_mature_fpd30,
deliquency.flg_mature_fspd_30,
deliquency.flg_mature_fstpd_30,
deliquency.deffpd10,
deliquency.deffpd30,
deliquency.deffspd30,
deliquency.deffstpd30,
if(digitalloan.vas_flag='true',1,0) vas_opted_flag,
cust_profile.fatca,
flg_reject_rule_check,
flg_applied_loan,
--flg_approved_scorecard_loan,
flg_disbursed_loan,
CASE WHEN cddRejectReason IS NOT NULL and cddStatus <> 'ACTIVATED' THEN 1 ELSE 0 END flg_cdd_reject_flag,
CASE WHEN rejectedStage = 'Stage0' THEN 1 ELSE 0 END flg_hard_reject_flag, 
CASE WHEN rejectedStage = 'Stage1' THEN 1 ELSE 0 END flg_taran_reject_flag, 
case when digitalloan.poi3RejectType in ('SOFT','HARD') and digitalloan.poi3StatusCode like 'RJ%' then 1 else 0 end ln_taran_rule_reject_flag,
case when poi3.uw_flag in ('Streamline UW','Full UW') then 1 else 0 end ln_marked_underwriter_check_flag,

-- CASE WHEN cddRejectReason IS NOT NULL and cddStatus = 'ACTIVATED' THEN 1 ELSE 0 END flg_cdd_approved_flag,
-- CASE WHEN rejectedStage <> 'Stage0' THEN 1 ELSE 0 END flg_not_hard_uw_approved, 
-- CASE WHEN rejectedStage <> 'Stage1' THEN 1 ELSE 0 END flg_taran_approved_flag, 

CASE WHEN loanmaster.applicationstatus='APPROVED' THEN 1 ELSE 0 END flg_loan_approved_not_disbursed,
digitalloan.loanRequestAmount loan_chosen_principal,digitalloan.loanRequestTenure loan_chosen_tenor,
coalesce(disbursedLoanAmount,if(loanmaster.applicationstatus='APPROVED',loanmaster.approvedLoanAmount,NULL)) loan_approved_principal,
coalesce(disbursedLoanTenur,if(loanmaster.applicationstatus='APPROVED',loanmaster.approvedLoanTenure,NULL)) loan_approved_tenor,
loan_defFPD30,loan_defSPD30,loan_defTPD30,defpd10_outstanding_principal,defpd30_outstanding_principal,defspd30_outstanding_principal,deftpd30_outstanding_principal,loanmaster.credolabRefNumber,
loanmaster.docType,

coalesce(osversion_credolab.credolab_osversion,osversion) onb_osversion,  
coalesce(ln_model_credolab.ln_model,osversion_credolab.onb_model) ln_model,        
cust_mtb.status,
onb_self_dec_income.monthly_income onb_self_dec_income,
onb_self_dec_income.company_name onb_company_name,
cust_closed_date.OFDATECLOSED cust_status_close_date,
credolab_creation.createdOn credo_inquiry_date,
purpose.source_of_fund_name purpose_source_of_fund_name,purpose.nature_of_work_name purpose_nature_of_work_name,purpose.employment_status_name purpose_employment_status_name,purpose.industryName purpose_industryName,purpose.industryid pupose_industryid,purpose.created_dt purpose_created_dt,
loan_level.industry,loan_level.natureofwork,
loan_level.employmentstatus,
loan_level.source_of_funds,
ln_mail.email ln_email
                                    
--CASE WHEN loanPaidStatus in ('Normal','In Arrears') THEN 1 ELSE 0 END count_ongoing_loans 

from risk_credit_mis.loan_master_table loanmaster
inner join `dl_loans_db_raw.tdbk_digital_loan_application` digitalloan on loanmaster.digitalLoanAccountId=digitalloan.digitalLoanAccountId
left join `dl_customers_db_raw.tdbk_customer_mtb` cust_mtb on cast(loanmaster.customerId as string)=cust_mtb.cust_id
left join dl_loans_db_raw.tdbk_digital_loan_application digital on digital.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join dl_loans_db_raw.tdbk_loan_purpose purpose on purpose.loanPurposeId=digital.loanPurposeId
left join cust_info on cust_info.digitalLoanAccountId=loanmaster.digitalLoanAccountId
LEFT JOIN dl_dynamo_db_raw.tdbk_nature_of_work_mtb nat
ON cast(nat.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(3)]
LEFT JOIN dl_dynamo_db_raw.tdbk_employment_status_mtb emp_status
ON cast(emp_status.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(0)]
LEFT JOIN dl_dynamo_db_raw.tdbk_source_of_funds_mtb source_fund
ON cast(source_fund.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(4)]
LEFT JOIN dl_dynamo_db_raw.tdbk_industry_list_mtb industry
ON cast(industry.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(6)]
left join educationtype on educationtype.digitalLoanAccountId=digitalloan.digitalLoanAccountId
left join ref_type on ref_type.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join `risk_credit_mis.loan_level_master_table` loan_level on loan_level.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join (SELECT * FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_vas_holiday_htb`  where  req_type=1
qualify row_number() over (partition by digitalLoanAccountId order by repayment_number desc)=1) vasflag on loanmaster.digitalLoanAccountId=vasflag.digitalLoanAccountId
LEFT JOIN (SELECT customer_id,    latitude,    longitude,  FROM    `risk_mart.customer_gps_location`
  WHERE event_description = 'Apigee Logs' AND latitude <> 'undefined' AND longitude <> 'undefined' 
  QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id order by gps_collection_date desc ) =1 ) loc on cast(loc.customer_id as int64)=loanmaster.customerId
  LEFT JOIN (select digitalLoanAccountId,max(installmentAmount) monthlytermamount from `risk_credit_mis.loan_installments_table` instal
--join core_raw_staging.loan_accounts loanacct on instal.loanAccountNumber=loanacct.ACCOUNTNUMBER  where LOANSTATUS in ('Normal','In Arrears')  
group by 1 ) loanaccts on loanaccts.digitalLoanAccountId = loanmaster.digitalLoanAccountId 
left join user_type_min on cast(user_type_min.customerId as string)=cust_mtb.cust_id 
left join user_type_loan_level on user_type_loan_level.digitalLoanAccountId=loanmaster.digitalLoanAccountId 
left join user_type_loan_level_v1 on user_type_loan_level_v1.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join worktable_data_analysis.lov_sf_industry_now_emp_status_apl15 empstatus on empstatus.digitalLoanAccountId=loanmaster.digitalLoanAccountId
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  LEFT JOIN dl_customers_db_raw.tdbk_cust_profile_mtb cust_profile on cust_profile.user_id = cust_mtb.user_id
  left join dl_customers_db_raw.tdbk_customer_add_mtb  per_addr on per_addr.user_id = cust_mtb.user_id
   left join deliquency on loanmaster.loanAccountNumber=deliquency.loanAccountNumber
   left join default_outstanding_principal on default_outstanding_principal.digitalLoanAccountId=loanmaster.digitalLoanAccountId
   left join `dl_loans_db_raw.tdbk_loan_poi3_response` poi3 on loanmaster.crifApplicationId=poi3.crifApplicationId
left join (SELECT digitalLoanAccountId,score FROM `dl_loans_db_derived.tdbk_loan_poi3_response_trustingsocial_data` 
qualify row_number() over (partition by digitalLoanAccountId order by timestamp_received desc)=1) trustingsocial on 
loanmaster.digitalLoanAccountId=trustingsocial.digitalLoanAccountId
 left join (select digitalLoanAccountId, MAX(coalesce(cast(FlagMatched as int64),0)) as matched_flag from dl_loans_db_derived.tdbk_loan_poi3_response_bureau_string_xml_parsed_matchedsubject
group by 1) flagmatched
on loanmaster.digitalLoanAccountId = flagmatched.digitalLoanAccountId
left join 
(SELECT digitalLoanAccountId,if(substr(ScoreRange,1,3)='NH_',0,1) summary_matchedflag,ScoreRange FROM `dl_loans_db_derived.tdbk_loan_poi3_response_bureau_string_xml_parsed_summary` 
qualify row_number() over (partition by digitalLoanAccountId order by run_date desc)=1) summary 
on summary.digitalLoanAccountId = loanmaster.digitalLoanAccountId

--only applied loans 2 condition's has to check 
left join credo_score_static_insght credo_score_dima on credo_score_dima.digitalLoanAccountId=loanmaster.digitalLoanAccountId
--worktable_data_analysis.dema_score_final_may12 credo_score_dima  on 
--credo_score_dima.reference_number=loanmaster.credolabRefNumber 
--and loanmaster.credolabRefNumber is not null
left join (select userId,refno,createdOn from  `dl_loans_db_raw.tdbk_credolab_track`  qualify  row_number() over (partition by userId order by createdOn)=1) onb_osversion 
on onb_osversion.userId=digitalloan.userid
left join osversion_credolab on onb_osversion.refno=osversion_credolab.deviceid
left join ln_model_credolab on loanmaster.credolabRefNumber=ln_model_credolab.deviceid
left join onb_self_dec_income on onb_self_dec_income.cust_id=cast(loanmaster.customerId as string)
left join cust_closed_date on cust_closed_date.OFCUSTOMERID=cust_mtb.cust_id 
left join credolab_creation on credolab_creation.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join datalake_worktables.customer_mobile_mail_dtls ln_mail on ln_mail.cust_id=cast(loanmaster.customerId as string)  
and (coalesce(loanmaster.termsAndConditionsSubmitDateTime,if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)) between DATETIME(ln_mail.valid_from_dt)   and  DATETIME(ln_mail.valid_to_dt))
left join (select * from `prj-prod-dataplatform.dl_loans_db_raw.tdbk_customer_loan_address` 

where status=1 qualify row_number() over (partition by digital_loan_account_id order by created_dt desc)=1) loanaddress on loanaddress.digital_loan_account_id=loanmaster.digitalLoanAccountId
                                 
where coalesce(loanmaster.termsAndConditionsSubmitDateTime,if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)) 
between '{SDATE}' 
-- and date_sub(date(current_date()), interval 1 day) 
and '{EDATE}'

and  loanmaster.credolabRefNumber is not null) main
--where digitalLoanAccountId in (select digitalLoanAccountId from risk_mart.applied_loans_20230101_20250615)

qualify row_number() over(partition by main.digitalLoanAccountId)=1
;
"""

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema1}.{al} created successfully')

Table risk_mart.applied_loans_20230101_20250818 created successfully


# Applied Trans

# Applied Trans Table

In [10]:
schema1 = 'risk_mart'

OBDATE = '2025-08-18'
# change it to obdate from current date if the requirement is for a specific date
SDATE = '2021-07-01'
EDATE = '2025-08-18'
# change it to obdate from current date if the requirement is for a specific date

formatted_date = datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')

print(formatted_date)  # Output: 20230101

altrans = f'applied_loans_{datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')}_{datetime.strptime(EDATE, '%Y-%m-%d').strftime('%Y%m%d')}_trans'

print(altrans)


20210701
applied_loans_20210701_20250818_trans


# Applied Trans Query

In [11]:
sq = f"""
create or replace table {schema1}.{altrans} as
select distinct customerId,
digitalLoanAccountId,
loanAccountNumber,
onboarding_datetime onb_tsa_onboarding_datetime,
first_name onb_first_name,
middle_name onb_middle_name,
last_name onb_last_name,
onb_age,
gender onb_gender,
onb_mobile_no,
email onb_email,
email_verified_flag onb_email_verified_flag,
place_of_birth onb_place_of_birth,
doctype onb_doc_type,					 
onb_country,
onb_province,
onb_city,
onb_barangay,
onb_postalcode,
onb_latitude,
onb_longitude,
onb_osversion,			  
kyc_status onb_kyc_status,
datetime(timestamp(kyc_status_change_dt), 'Asia/Manila') onb_kyc_status_upgrade_datetime,
--kyc_status_change_dt onb_kyc_status_upgrade_datetime,
user_type ln_user_type,
ln_loan_level_user_type ln_loan_level_user_type_old,
--ln_loan_level_user_type_v1 ln_loan_level_user_type,
if(loan_type in ('Flex-up','Reloan') ,'1_Repeat Applicant',ln_loan_level_user_type_v1) ln_loan_level_user_type,
loan_type ln_loan_type,
loan_product_type ln_prod_type,
flg_applied_loan ln_loan_applied_flag,
fatca ln_facta_flag,
flg_reject_rule_check ln_dl_rule_reject_flag,
ln_taran_rule_reject_flag,
flg_taran_reject_flag ln_taran_scorecard_reject_flag,
flg_cdd_reject_flag ln_cdd_reject_flag,
ln_marked_underwriter_check_flag,
flg_hard_reject_flag ln_underwriting_reject_flag, 
if(flg_disbursed_loan=1,1,flg_loan_approved_not_disbursed) ln_final_approved_flag,
flg_disbursed_loan ln_disb_flag,
flg_loan_approved_not_disbursed ln_approved_not_disb_flag,
loan_submission_date ln_appln_submit_datetime,
disbursementDateTime ln_disb_dtime,
loan_chosen_principal ln_chosen_principal,
loan_chosen_tenor ln_chosen_tenor,
loan_approved_principal ln_approved_principal,
loan_approved_tenor ln_approved_tenor,
main.onb_cnt_ongoing_loans ln_cnt_ongoing_tdb_loans,
onb_tot_ongoing_loans_emi ln_tot_ongoing_tdb_loans_emi,
loan_purpose ln_purpose,
vas_opted_flag ln_vas_opted_flag,
vas_used_flag ln_vas_used_flag,
loan_age ln_age,
loan_mobile_no ln_mobile_no,
loan_alternate_mobile_no ln_alt_mobile_no,
osversion_v2 ln_osversion,
brand ln_brand,
ln_os_type,
ln_model,
coalesce(ln_email,email) ln_email,							 
coalesce(loan_address,addressline) ln_address,
coalesce(loan_province,province) ln_province,
coalesce(loan_city,city) ln_city,
coalesce(loan_barangay,barangay) ln_barangay,
coalesce(cast(loan_postal_code as string),postalcode) ln_postal_code,
loan_latitude ln_latitude,
loan_longitude ln_longitude,
loan_docType ln_doc_type,
loan_docNumber ln_doc_number,
if(loan_marital_status='Live-in Partner','With a Live-in Partner',loan_marital_status) ln_marital_status,
--case when loan_dependents_count in ('No dependents') then null else loan_dependents_count end ln_cnt_dependents,
loan_dependents_count ln_cnt_dependents,
loan_education_level ln_education_level,
coalesce(purpose_source_of_fund_name,main.source_funds) ln_source_funds,
--coalesce(source_funds_new.source_funds_new,main.source_funds) ln_source_funds_new,
case when coalesce(purpose_source_of_fund_name,main.source_funds) in ('Others') then null
when coalesce(purpose_source_of_fund_name,main.source_funds) in ('Commissions') then null else coalesce(purpose_source_of_fund_name,main.source_funds) end as ln_source_funds_new,
coalesce(purpose_employment_status_name,employmentstatus) ln_employment_type,
--coalesce(employment_type_new.employment_type_new,main.employment_type) ln_employment_type_new,
case when coalesce(purpose_employment_status_name,employmentstatus)='Employed' and Mod(cast(customerId as int64),10)<3 then 'Employed - Govt. Employee/Govt. Official' 
when coalesce(purpose_employment_status_name,employmentstatus)='Employed' and Mod(cast(customerId as int64),10)>2 then'Employed - Private Employee' 
else coalesce(purpose_employment_status_name,employmentstatus) end as ln_employment_type_new,
-- case when upper(coalesce(purpose_employment_status_name,main.employment_type)) like'%EMPLOYED%' and Mod(cast(customerId as int64),10)<3 then 'Employed - Govt. Employee/Govt. Official'
-- when upper(coalesce(purpose_employment_status_name,main.employment_type)) like'%EMPLOYED%' and Mod(cast(customerId as int64),10)>2 then'Employed - Private Employee'
-- else coalesce(purpose_employment_status_name,main.employment_type) end as ln_employment_type_new,
--main.employment_type_new ln_employment_type_new,
coalesce(purpose_nature_of_work_name,natureofwork) ln_nature_of_work,
--coalesce(nature_of_work_new.nature_of_work_new,main.nature_of_work) ln_nature_of_work_new,
case when coalesce(purpose_nature_of_work_name,natureofwork) in ('Admin/Clerical Staff/Officer') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Architect/Interior Decorator') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Bank Employee') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Broker/Dealer') then  'Broker/Dealer'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Call Center Agent/Tele Marketer') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Cashier/Food Server/Waiter') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Consultant') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Doctor/Dentist/Medical Professional') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Engineer') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Entertainer/Artist') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Foreign Ambassador/Diplomat') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Gov Employee') then  'Govt Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Govt Employee') then  'Govt Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Insurance Agent/Financial Advisor') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('IT Professional') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Lawyer') then  'Licensed Professional - Lawyer'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Military/Armed Forces/Police/Navy/Airforce') then  'Government - Uniformed Service'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Newscaster/Media Personality') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Pilot/Airline Crew') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Seafarer') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Teacher/Instructor/Coach') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Utility Personnel/Household Help') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Accountant/Auditor') then  'Licensed Professional - Accountant'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Utilities (Electricity Gas and Water)') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Wholesale and Retail Trade') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Sales/Marketing Personnel') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Other Professional services') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Other Non Professional Services') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Owner') then  'Owner'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Religious Leader') then  'Religious Leader'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Fire Fighter') then  'Government - Uniformed Service'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Farmer/Fisherman') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Debt Collection/Recovery Agents/ Collection Agency Employee') then  'Debt Collection/Recovery Agents/ Collection Agency Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Court Personnel/Bailiff') then  'Court Personnel/Bailiff'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Store/Service Manager') then  'Manager/Executive'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Sales Marketing Professional') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Barber / Beauty Salon') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Business Owner (MSME)') then  'Owner'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Business Owner') then  'Owner'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Delivery Rider/Driver') then  'Delivery Rider/Driver'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Delivery Rider / Driver') then  'Delivery Rider / Driver'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Manpower Agency') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Online Freelancing / Virtual Assistant / Online teacher') then  'Virtual/Online/Remote Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Online Seller') then  'Virtual/Online/Remote Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Real Estate agent') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Skilled Worker') then  'Skilled Worker'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Technician (Cellphone / Laptop)') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Vendor (Sari-Sari store Wet market etc)') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Staff/Rank and File') then  'Staff/Rank and File'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Officer/Supervisor') then  'Officer/Supervisor'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Manager/Executive') then  'Manager/Executive'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Licensed Professional - Lawyer') then  'Licensed Professional - Lawyer'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Licensed Professional - Accountant') then  'Licensed Professional - Accountant'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Licensed Professional - Others') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Religious Minister') then  'Religious Minister'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Government - Elected Official') then  'Government - Elected Official'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Government - Uniformed Service') then  'Government - Uniformed Service'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Director/Trustee/Stockholder') then  'Director/Trustee/Stockholder'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Virtual/Online/Remote Employee') then  'Virtual/Online/Remote Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Government - Appointed Official') then  'Government - Appointed Official'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('AIR FORCE') then  'Military/Armed Forces/Police/Navy/Airforce'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('NAVY') then  'Military/Armed Forces/Police/Navy/Airforce'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Admin/Secretarial') then  'Admin/Clerical Staff/Officer'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Sales/Marketing Professional') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('COURT-PERSONNEL/BAILIFFS') then  'Court Personnel/Bailiff'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('DEBT COLLECTION/RECOVERY AGENTS/COLLECTION AGENCY EMPLOYEE') then  'Debt Collection/Recovery Agents/ Collection Agency Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Accountant') then  'Licensed Professional - Accountant' else coalesce(purpose_nature_of_work_name,natureofwork) end as ln_nature_of_work_new,
if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) ln_industry,
case when main.source_funds in ('Remittance','Pension','Inheritance/Retirement Funds','Interest on Deposits and Investments','Allowance') then null 
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Wholesale and Retail Trade') then 'Wholesale and Retail Trade'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Utilities (Electricity, Gas and Water)') then 'Utilities (Electricity, Gas and Water)'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Transportation') then 'Transportation'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Tourism, Hotel and Resorts') then 'Tourism, Hotel and Resorts'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('TeleCo') then 'Telecommunication'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Telecommunication') then 'Telecommunication'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Religious Organization') then 'Religious Organization'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Entertainment and Media') then 'Recreation (Arts, Entertainment/Media, Sports)'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Real Estate') then 'Real Estate'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Pawnshop/Remittance Agent/Virtual Currency') then 'Pawnshop/Remittance Agent/Virtual Currency'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Others') then 'Others'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Non Government Organization/Foundation') then 'NGOs/NonProfit Organization/Foundation'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Military and Weaponry Business') then 'Military and Weaponry Business'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Maritime/Shipping') then 'Maritime/Shipping'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Manufacturing - Food') then 'Manufacturing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Manufacturing - Non Food') then 'Manufacturing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Manpower/Manning Agencies') then 'Manpower/Manning Agencies'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Legal Service/Law Firm') then 'Legal Service/Law Firm'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Insurance Activities') then 'Insurance Activities'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Hospital and Medical Services') then 'Hospital and Medical Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Govt. Institution') then 'Govt. Institution'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Gold, Jewelry and Precious Metals') then 'Gold/Jewelry/Precious Metals/Art/Antique Dealership'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Online Gaming/Lottery/Casino') then 'Gambling and Betting (Online Gaming, Lottery, Casino)'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Freight and Cargo') then 'Freight and Cargo'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Food Industry/Restaurants/Fastfoods') then 'Food Industry/Restaurants/Fastfoods'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Fin Tech/Financial Services') then 'Fin Tech/Financial Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('FX Dealer/Money Changer') then 'FX Dealer/Money Changer'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Education/Schools') then 'Education/Private Schools'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ("Convenience Store/Internet Cafe's") then "Convenience Store/Internet Cafe's"
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Construction') then 'Construction'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Computer/IT Related Services') then 'Computer/IT Related Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Business Process Outsourcing/Tele Marketing') then 'Business Process Outsourcing/Tele Marketing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Beauty and Health Services') then 'Beauty and Health Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Health and Beauty Services') then 'Beauty and Health Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Banking') then 'Banking'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Architecture/Engineering') then 'Architecture/Engineering'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Airline/Aviation') then 'Airline/Aviation'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Agribusiness/Animal Farming/Fishing') then 'Agribusiness/Animal Farming/Fishing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Advertising/Sales/Marketing') then 'Advertising/Sales/Marketing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Accounting/Bookkeeping') then 'Accounting/Bookkeeping/Auditing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) is null then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Leasing and Rental Services') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Repair, Sanitation and Housekeeping') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Repair, Sanitation and Housekeeping') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Embassy') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Recycling') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Publishing and Printing') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('IT services','ONLINE SELLER','business owner','Human Resources','Head of sales','Human Resource Management',
'Digital Marketing','Food services','Manufacture of Leather and Related Products','Manufacture of Other Non-Metallic Mineral Products',
'Businesswoman','Business Owner','Agency Manpower','Advertising and marketing','Manufacture of Coke and Refined Petroleum Products',
'Sales Coffee Trading','Pharmaceutical','Manufacture of Furniture','Software as a Service','Nurse','Manufacture of Rubber Products','Shared Services',
'Waste and recycling solutions','Event Supplier','Fishing and Aquaculture','Manufacture of Motor Vehicles, Trailers and Semi-Trailers',
'Crop and Animal Production, Hunting and Related Service Activities; Forestry and Logging','Publishing Activities',
'Manufacture of Basic Metals','Manufacture of Textiles','Manufacture of Electrical Equipment',
'Water Collection, Treatment and Supply','Water Transport','Manufacture of Basic Pharmaceutical Products and Pharmaceutical Preparation',
'Manufacture of Chemicals and Chemical Products','Insurance, Reinsurance and Pension Funding except Compulsory Social Security',
'Electricity, Gas, Steam and Air Conditioning Supply','Manufacture of Machinery and Equipment, n.e.c.',
'Travel Agency, Tour Operator, Reservation Service and Related Activities','Medical Transcription  Activities',
'Manufacture of Fabricated Metal Products except Machinery and Equipment','Manufacture of Plastic Products',
'Postal and Courier Activities','Real Estate Activities','Printing and Reproduction of Recorded Media',
'Manufacture of Computer, Electronic and Optical Products','Transport via Buses',
'Religious Organization ','Designated NonFinancial Businesses ','Manufacture of Wearing Apparel',
'Warehousing and Support Activities for Transportation','Architectural and Engineering Activities and Related Technical Consultancy',
'Wholesale  and Retail Trade and Repair of Motor Vehicles and Motorcycles','Designated NonFinancial Businesses',
'Human Health Activities except Public Health Activities','Manufacture of Food Products',
'Military and Weaponry Business','Retail Trade except of Motor Vehicles and Motorcycles',
'Land Transport and Transport via Pipelines except  Transport via Buses','Education except Public Education',
'Financial Service Activities except Insurance, Pension Funding and Central Banking','Wholesale Trade except of Motor Vehicles and Motorcycles',
'Accounting, Bookkeeping and Auditing Activities; Tax Consultancy','Accommodation and Food Service Activities',
'NGOs/NonProfit Organization/Foundation ','Mining and Quarrying ','Computer Programming, Consultancy and Related Activities; Information Service Activities',
'Call Center Activities (Voice)','Back-Office Operations Activities (Non-Voice) Except Medical Transcription Activities',
'Nightclubs/Discos/Karaoke Center','Import Export Companies ','Mining and Quarrying','Import Export Companies',
'Gambling and Betting (Online Gaming, Lottery, Casino) ','Other Personal Services Activities','FX Dealer/Money Changer',
'Religious Organization','Securities/Holding Companies ','Car/Boat/Plane/Used Truck/AutoParts Manufacturing/Dealership or Auction Houses') then null
else if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) end as ln_industry_new, 
--purpose_source_of_fund_name,purpose_nature_of_work_name,purpose_employment_status_name,
--coalesce(industry_description_new.industry_description_new,if(pupose_industryid is not null , coalesce(purpose_industryName,if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description)),if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description))) ln_industry_new,
loan_companyName ln_company_name,
loan_self_dec_income ln_self_dec_income,
loan_salary_scaled_income ln_salary_scaled_income,
loan_ref_type1 ln_ref1_type,
loan_ref_type2 ln_ref2_type,
flg_cic_called cic_called_flag,
flg_cic_hit cic_hit_flag,
ScoreRange cic_score_range,
demo_score old_demo_score,
cic_score old_cic_score,
credo_score old_gen_credo_score,
trusting_social_score ts_trusting_social_score,
bureau_score bu_bureau_score,
c_credo_score,s_credo_score,fu_credo_score,r_credo_score,gen_credo_score,
flg_mature_fpd10 ln_mature_fpd10_flag,
flg_mature_fpd30 ln_mature_fpd30_flag,
flg_mature_fspd_30 ln_mature_fspd30_flag,
flg_mature_fstpd_30 ln_mature_fstpd30_flag,
deffpd10 ln_fpd10_flag,
deffpd30 ln_fpd30_flag,
deffspd30 ln_fspd30_flag,
deffstpd30 ln_fstpd30_flag,
--loan_defFPD30 ln_defFPD30,
loan_defSPD30 ln_spd30_flag,loan_defTPD30 ln_tpd30_flag,
defpd10_outstanding_principal ln_fpd10_os_principal,defpd30_outstanding_principal ln_fpd30_os_principal,
defspd30_outstanding_principal ln_fspd30_os_principal,deftpd30_outstanding_principal ln_fstpd30_os_principal,
credolabRefNumber,
datetime(credo_inquiry_date,'Asia/Manila') credo_inquiry_date,														  
status cust_status_flag,
onb_self_dec_income,
onb_company_name,
cust_status_close_date

from 
(with educationtype AS (
        SELECT
        loan.digitalLoanAccountId,loanAccountNumber, loan.loanPurposeId,   education_id,
        description AS education_level
        FROM  dl_loans_db_raw.tdbk_digital_loan_application loan
        LEFT JOIN  dl_loans_db_raw.tdbk_loan_purpose purpose
        ON   loan.loanPurposeId = purpose.loanPurposeId
        LEFT JOIN dl_loans_db_raw.tdbk_loan_lov_mtb
        ON education_id = id AND module = 'Education'
         ),

ref_type as (with Ref_type1 as (select  digitalLoanAccountId,relationship_id , description AS loan_ref_type1 from dl_loans_db_raw.tdbk_loan_refernce_details A
        LEFT JOIN dl_loans_db_raw.tdbk_loan_lov_mtb B  ON A.relationship_id = B.id
 where refPrefrenceOrder='1' qualify row_number() over (partition by digitalLoanAccountId order by refCreatedDateAndTime desc) =1),
    Ref_type2 as (select  digitalLoanAccountId,relationship_id , description AS loan_ref_type2 from dl_loans_db_raw.tdbk_loan_refernce_details A
        LEFT JOIN dl_loans_db_raw.tdbk_loan_lov_mtb B  ON A.relationship_id = B.id
 where refPrefrenceOrder='2' qualify row_number() over (partition by digitalLoanAccountId order by refCreatedDateAndTime desc) =1)
select Ref_type1.digitalLoanAccountId,loan_ref_type1,loan_ref_type2 from Ref_type1 join Ref_type2 on Ref_type1.digitalLoanAccountId=Ref_type2.digitalLoanAccountId) ,

user_type_min as (select customerId,min(case 
when digitalLoanAccountId is null then '4_Never_start_loan_appln'
when application_submit_date is null and rn1_application_status <=1 then '4_Never_start_loan_appln'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status is null then '3_Applied_Not_Disbursed'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status =1 then '2_New Applicant'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status >1 then '1_Repeat Applicant'

else '5_others' end ) user_type from
(select A.*,B.rn_disburement_status from
(select digitalLoanAccountId,customerId,applicationStatus, coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime)) application_submit_date,disbursementDateTime,
row_number() over (partition by customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))) rn1_application_status,
from risk_credit_mis.loan_master_table) A

left join 

(select digitalLoanAccountId,customerId,applicationStatus, disbursementDateTime,
case when disbursementDateTime is not null then 
row_number() over (partition by customerId order by disbursementDateTime) else 0 end rn_disburement_status,
from risk_credit_mis.loan_master_table where  disbursementDateTime is not null) B on A.digitalLoanAccountId=B.digitalLoanAccountId) 
--order by A.customerId 
group by 1 ),

user_type_loan_level as (select customerId,digitalLoanAccountId,min(case 
when digitalLoanAccountId is null then '4_Never_start_loan_appln'
when application_submit_date is null and rn1_application_status <=1 then '4_Never_start_loan_appln'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status is null then '3_Applied_Not_Disbursed'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status =1 then '2_New Applicant'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status >1 then '1_Repeat Applicant'

else '5_others' end ) user_type from
(select A.*,B.rn_disburement_status from
(select digitalLoanAccountId,customerId,applicationStatus, coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime)) application_submit_date,disbursementDateTime,
row_number() over (partition by customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))) rn1_application_status,
from risk_credit_mis.loan_master_table
where if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '2021-07-01' and '2025-08-31'
) A

left join 

(select digitalLoanAccountId,customerId,applicationStatus, disbursementDateTime,
case when disbursementDateTime is not null then 
row_number() over (partition by customerId order by disbursementDateTime) else 0 end rn_disburement_status,
from risk_credit_mis.loan_master_table where  disbursementDateTime is not null and  if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '2021-07-01' and '2025-08-31') B on A.digitalLoanAccountId=B.digitalLoanAccountId) 

group by 1 ,2 order by customerId),

user_type_loan_level_v1 as (select customerId,digitalLoanAccountId,disbursementDateTime,application_submit_date,rn_disburement_status,first_disbursment_loan,  min(case 
when digitalLoanAccountId is null then '4_Never_start_loan_appln'
when application_submit_date is null and rn1_application_status <=1 then '4_Never_start_loan_appln'
--when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status is null then '3_Applied_Not_Disbursed'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status =1 and first_disbursment_loan > application_submit_date then '2_New Applicant'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status >=1  then '1_Repeat Applicant'
when application_submit_date is not null and rn1_application_status >=1  and  first_disbursment_loan < application_submit_date  then '1_Repeat Applicant'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status is null then '2_New Applicant'

else '5_others' end ) user_type from
(select A.*,
B.rn_disburement_status ,C.first_disbursment_loan,
from
(select digitalLoanAccountId,customerId,applicationStatus, coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime)) application_submit_date,disbursementDateTime,
row_number() over (partition by customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))) rn1_application_status,
from risk_credit_mis.loan_master_table
where if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '2021-07-01' and '2025-08-31'
) A

left join 

(select digitalLoanAccountId,customerId,applicationStatus, disbursementDateTime,
case when disbursementDateTime is not null then 
row_number() over (partition by customerId order by disbursementDateTime) else 0 end rn_disburement_status,
from risk_credit_mis.loan_master_table where  disbursementDateTime is not null and if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '2021-07-01' and '2025-08-31') B on A.digitalLoanAccountId=B.digitalLoanAccountId 

left join 

(select min(disbursementDateTime) first_disbursment_loan,customerId,
from risk_credit_mis.loan_master_table where  disbursementDateTime is not null and  if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '2021-07-01' and '2025-08-31'
group by customerId) C on A.customerId=C.customerId
) 
--where customerId in (1668511,2480643,2027516,1802996,2469891,2418795)

group by 1 ,2,3,4,5,6 order by customerId ,application_submit_date),


deliquency as
(select loanAccountNumber,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30  in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),

cust_info as (
with cust_base as(
select * from `dl_customers_db_raw.tdbk_customer_mtb`
where cust_id is not null
--and status in ('C','A')
--and date(created_dt,'Asia/Manila') >= '2021-07-01'
),
first_loan_application as (
select custId,digitalLoanAccountId,loanaccountnumber,isTermsAndConditionstimestamp,poi1RejectType,loantype from
--(select * from 
`dl_loans_db_raw.tdbk_digital_loan_application`
--where custId in (select cast(cust_id as integer) from cust_base) qualify row_number() over (partition by custId order by created_dt) = 1)
--where --loanType in ('TUL') and coalesce(flowType,'1') <> '2'
),
loan_calculator as (
select custid,digitalLoanAccountId from
(SELECT fla.custid,fla.digitalLoanAccountId,
sum(case when a.applicationStatus='INITIATE' then 1 else 0 end) loan_calculator
 FROM `dl_loans_db_raw.tdbk_status_trace`  a
 join first_loan_application fla
 on fla.digitalLoanAccountId=a.digitalLoanAccountId
 group by 1,2)
where loan_calculator>0
),
disbursed_undisbursed_loan as (
SELECT fla.custid,fla.digitalLoanAccountId,
 FROM `dl_loans_db_raw.tdbk_status_trace`  a
 join first_loan_application fla
 on fla.digitalLoanAccountId=a.digitalLoanAccountId
 where a.applicationStatus ='APPROVED'
 group by 1,2
)


select 
 cb.*, fla.digitalLoanAccountId,--onboarding_datetime, 
 fla.loantype,
case when lc.custid is not null then 1 else 0 end as flg_loan_calculator,
case when fla.poi1RejectType is not null or fla.loantype is null then 1 else 0 end as flg_reject_rule_check,
case when (fla.isTermsAndConditionstimestamp is not null or loan.new_loan_type ='Flex-up')  then 1 else 0 end as flg_applied_loan,
case when ul.custid is not null then 1 else 0 end as flg_approved_scorecard_loan,
case when fla.loanaccountnumber is not null then 1 else 0 end as flg_disbursed_loan,
fla.loanaccountnumber as loan_id
from cust_base cb
left join first_loan_application fla
on cast(cb.cust_id as integer)=fla.custId
left join loan_calculator lc
--on cast(cb.cust_id as integer)=lc.custid and 
on lc.digitalLoanAccountId=fla.digitalLoanAccountId
left join disbursed_undisbursed_loan ul
--on cast(cb.cust_id as integer)=ul.custid)
on ul.digitalLoanAccountId=fla.digitalLoanAccountId
left join 
risk_credit_mis.loan_master_table loan on loan.digitalLoanAccountId=fla.digitalLoanAccountId),																																																	


default_outstanding_principal as (with all_default_account as (select digitalLoanAccountId,loanAccountNumber,firstDueDate,secondDueDate,thirdDueDate,defFPD30,defSPD30,defTPD30 from 
`risk_credit_mis.loan_master_table`
where defFPD10=1 or defFPD30=1 or defSPD30=1 or defTPD30=1),

defpd10_pricipal_outstanding as (
select loanAccountNumber,firstDueDate,PRINCIPALARREARS defpd10_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(firstDueDate ,INTERVAL 10 DAY)=core._Partitiondate or Date_add(firstDueDate ,INTERVAL 11 DAY)=core._Partitiondate)
where defFPD10=1 qualify row_number() over (partition by loanAccountNumber order by defpd10_outstanding_principal desc )=1),

defpd30_pricipal_outstanding as (
select loanAccountNumber,firstDueDate,PRINCIPALARREARS defpd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(firstDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(firstDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defFPD30=1 qualify row_number() over (partition by loanAccountNumber order by defpd30_outstanding_principal desc )=1),

defspd30_pricipal_outstanding as (
select loanAccountNumber,secondDueDate,PRINCIPALARREARS defspd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(secondDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(secondDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defSPD30=1 qualify row_number() over (partition by loanAccountNumber order by defspd30_outstanding_principal desc )=1),

deftpd30_pricipal_outstanding as (
select loanAccountNumber,thirdDueDate,PRINCIPALARREARS deftpd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(thirdDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(thirdDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defTPD30=1 qualify row_number() over (partition by loanAccountNumber order by deftpd30_outstanding_principal desc )=1)

select digitalLoanAccountId,allaccount.loanAccountNumber,defpd10_outstanding_principal,defpd30_outstanding_principal,defspd30_outstanding_principal,deftpd30_outstanding_principal,
allaccount.firstDueDate,allaccount.secondDueDate,allaccount.thirdDueDate,defFPD30 loan_defFPD30,defSPD30 loan_defSPD30,defTPD30 loan_defTPD30  from all_default_account allaccount
left join defpd10_pricipal_outstanding day10dpd on allaccount.loanAccountNumber=day10dpd.loanAccountNumber
left join defpd30_pricipal_outstanding firstdpd on allaccount.loanAccountNumber=firstdpd.loanAccountNumber
left join defspd30_pricipal_outstanding seconddpd on allaccount.loanAccountNumber=seconddpd.loanAccountNumber
left join deftpd30_pricipal_outstanding thirddpd on allaccount.loanAccountNumber=thirddpd.loanAccountNumber),

osversion_credolab as (select * from (
(select distinct deviceId,'android'||generalInfo.release as credolab_osversion ,GeneralInfo.model onb_model from credolab_raw.android_credolab_datasets_struct_columns
union all 
select distinct deviceId,'ios'||generalInfo.release as credolab_osversion ,GeneralInfo.model onb_model from  credolab_raw.ios_credolab_datasets_struct_columns))),

ln_model_credolab as (select * from (
(select distinct deviceId, GeneralInfo.model ln_model  from credolab_raw.android_credolab_datasets_struct_columns
union all 
select distinct deviceId,GeneralInfo.model ln_model from  credolab_raw.ios_credolab_datasets_struct_columns))),

onb_self_dec_income as 
(select custmtb.cust_id,monthly_income,company_name from 
`prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_mtb`  custmtb
join `prj-prod-dataplatform.dl_customers_db_raw.tdbk_cust_profile_mtb` profile on custmtb.user_id=profile.user_id
where monthly_income is not null),

cust_closed_date as (select OFCUSTOMERID,OFDATEOPENED, IF(OFDATECLOSED='1970-01-01' , NULL, OFDATECLOSED) OFDATECLOSED from `prj-prod-dataplatform.core_raw.customer_accounts`
join `dl_customers_db_raw.tdbk_customer_mtb` mtb on mtb.cust_id=OFCUSTOMERID
 where  CRINTERCODE = 'POC20000'
qualify row_number() over (partition by OFCUSTOMERID order by OFDATECLOSED desc) =1 and mtb.status='C'),

																 
credolab_creation as (select --min(A.created_dt),min(B.createdOn)
digitalLoanAccountId,credolabRefNumber,A.userId,loanDigitalAppliedDateAndTime,A.created_dt,B.createdOn,A.clCreditScore,B.creditScore 
from `dl_loans_db_raw.tdbk_digital_loan_application` A
join dl_loans_db_raw.tdbk_credolab_track B on refno=credolabRefNumber qualify row_number() over(partition by credolabRefNumber,digitalLoanAccountId order by createdOn desc )=1),

credo_score_static_insght as (select * from (select 'static' as source, loan.digitalLoanAccountId,reference_number referenceNumber,datetime(credo_inquiry_date) requestedDate,null score_all_score,generic_score score_all_probability,
null score_sil_score,sil_score score_sil_probability, null score_quick_score,quick_score score_quick_probability,null score_flex_score,flexup_score score_flex_probability,null score_reloan_score, reloan_score score_reloan_probability from worktable_data_analysis.dema_score_final_may12 dima
left join risk_credit_mis.loan_master_table loan on dima.reference_number=loan.credolabRefNumber

union all

select 'credotrace' as source, loan.digitalLoanAccountId,referenceNumber referenceNumber,datetime(requestedDate) requestedDate,score_all_score,cast (score_all_probability as float64) score_all_probability,
score_sil_score,cast (score_sil_probability as float64) score_sil_probability,score_quick_score,cast(score_quick_probability as float64) score_quick_probability,score_flex_score,cast(score_flex_probability as float64) score_flex_probability,score_reloan_score,cast(score_reloan_probability as float64) score_reloan_probability from risk_credit_mis.credolab_trace_insight_all_loans  dima
left join risk_credit_mis.loan_master_table loan on dima.referenceNumber=loan.credolabRefNumber

union all
select 'insight' as source , digitalLoanAccountId,referenceNumber,datetime(requestedDate) requestedDate,score_all_score,cast (score_all_probability as float64) score_all_probability,
score_sil_score,cast (score_sil_probability as float64) score_sil_probability,score_quick_score,cast(score_quick_probability as float64) score_quick_probability,score_flex_score,cast(score_flex_probability as float64) score_flex_probability,score_reloan_score,cast(score_reloan_probability as float64) score_reloan_probability  from 

(
(select digitalLoanAccountId,referenceNumber,datetime(requestedDate) requestedDate,score_all_score,score_all_probability,
score_sil_score,score_sil_probability,score_quick_score,score_quick_probability,score_flex_score,score_flex_probability,score_reloan_score,score_reloan_probability from credolab_raw.credolab_insights_android
union all 
select digitalLoanAccountId,referenceNumber,requestedDate,score_all_score,score_all_probability,
score_sil_score,score_sil_probability,score_quick_score,score_quick_probability,score_flex_score,score_flex_probability,score_reloan_score,score_reloan_probability from credolab_raw.credolab_insights_ios))
qualify row_number() over(partition by digitalLoanAccountId order by requestedDate desc)  =1)
qualify row_number() over(partition by digitalLoanAccountId order by requestedDate)  =1)


select loanmaster.customerId,
cust_mtb.email as email,
cust_mtb.email_status as email_verified_flag,
cust_mtb.birthplace as place_of_birth,
cust_profile.first_nm first_name,
cust_profile.middle_nm middle_name,
cust_profile.last_nm last_name,
hm_city as onb_city,
hm_province as onb_province,
hm_postalcode as onb_postalcode,
hm_barangay as onb_barangay,
hm_country as onb_country,
latitude onb_latitude,
longitude onb_longitude,
DATE_DIFF(DATE(datetime(cust_mtb.created_dt,'Asia/Manila')), parse_date('%d %b %Y', cust_mtb.dob), YEAR) - IF(EXTRACT(MONTH FROM parse_date('%d %b %Y', cust_mtb.dob))*100 + EXTRACT(DAY FROM parse_date('%d %b %Y', cust_mtb.dob)) > EXTRACT(MONTH FROM DATE(datetime(cust_mtb.created_dt,'Asia/Manila')))*100 + EXTRACT(DAY FROM DATE(datetime(cust_mtb.created_dt,'Asia/Manila'))),1,0) as onb_age,
DATE_DIFF(DATE(if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)), parse_date('%d %b %Y', cust_mtb.dob), YEAR) - IF(EXTRACT(MONTH FROM parse_date('%d %b %Y', cust_mtb.dob))*100 + EXTRACT(DAY FROM parse_date('%d %b %Y', cust_mtb.dob)) > EXTRACT(MONTH FROM DATE(if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)))*100 + EXTRACT(DAY FROM DATE(if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime))),1,0) as loan_age,
cust_mtb.gender,
if (user_type_min.user_type='1_Repeat Applicant' or (new_loan_type in ('Flex-up') and decision_date is not null),1,0) onb_cnt_ongoing_loans,
loanmaster.digitalLoanAccountId,loanmaster.loanAccountNumber,datetime(cust_mtb.created_dt,'Asia/Manila') onboarding_datetime,
cust_mtb.mobile_no onb_mobile_no, mobileNo loan_mobile_no,case when length(digitalloan.alternateNumber)=10 then concat('63',digitalloan.alternateNumber) else digitalloan.alternateNumber end loan_alternate_mobile_no,loanmaster.purposeDescription loan_purpose,
source_fund.description AS source_funds,
emp_status.description AS employment_type,
nat.description AS nature_of_work,
industry.value industry_description,
loanmaster.companyName loan_companyName,
loanmaster.maritalStatus loan_marital_status,
loanmaster.dependentsCount loan_dependents_count,
educationtype.education_level loan_education_level,
if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime) loan_submission_date,
cust_mtb.kyc_status,
cust_mtb.kyc_status_change_dt,
loan_ref_type1,loan_ref_type2,
loanmaster.disbursementDateTime,
loan_level.addressline,
loan_level.province,
loan_level.city,
loan_level.barangay,
loan_level.postalcode,
loanaddress.full_address loan_address,
loanaddress.province loan_province,
loanaddress.city loan_city,
loanaddress.barangay loan_barangay,
loanaddress.postal_code loan_postal_code,
SPLIT(loan_level.geolocation, ' | ')[SAFE_OFFSET(0)] AS loan_latitude,
SPLIT(loan_level.geolocation, ' | ')[SAFE_OFFSET(1)] AS loan_longitude,
loan_level.docType loan_docType,
loan_level.docNumber loan_docNumber,
user_type_min.user_type,
user_type_loan_level.user_type ln_loan_level_user_type,
user_type_loan_level_v1.user_type ln_loan_level_user_type_v1,
case when loanmaster.new_loan_type ='Flex-up' and reloan_flag=1 then 'Reloan' else new_loan_type end loan_type,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
loanmaster.osversion_v2 osversion_v2,
if(lower(loanmaster.osversion_v2) like 'ios%','Apple', loan_level.brand) brand,
if(lower(loanmaster.osversion_v2) like 'ios%','iOS','Android') ln_os_type,
loanmaster.monthlyIncome loan_self_dec_income,
incomeperSalaryScale loan_salary_scaled_income,
case when vasflag.digitalLoanAccountId is not null then 1 else 0 end vas_used_flag,
if(poi3.crifApplicationId is not null,1,0) flg_cic_called,
flagmatched.matched_flag flg_cic_hit,
summary.ScoreRange,
--summary_matchedflag flg_cic_hit,
loanmaster.demographicScore demo_score,poi3.cicScorecard cic_score,loanmaster.credolabScore credo_score,trustingsocial.score trusting_social_score,poi3.bureauScore bureau_score,
--quick_score c_credo_score,sil_score s_credo_score,flexup_score fu_credo_score,reloan_score r_credo_score,generic_score gen_credo_score,
score_quick_probability c_credo_score,score_sil_probability s_credo_score,score_flex_probability fu_credo_score,score_reloan_probability r_credo_score,score_all_probability gen_credo_score,
if (user_type_min.user_type='1_Repeat Applicant' or (new_loan_type in ('Flex-up') and decision_date is not null),loanaccts.monthlytermamount,0) as onb_tot_ongoing_loans_emi,
empstatus.employment_type_new,
deliquency.flg_mature_fpd10,
deliquency.flg_mature_fpd30,
deliquency.flg_mature_fspd_30,
deliquency.flg_mature_fstpd_30,
deliquency.deffpd10,
deliquency.deffpd30,
deliquency.deffspd30,
deliquency.deffstpd30,
if(digitalloan.vas_flag='true',1,0) vas_opted_flag,
cust_profile.fatca,
flg_reject_rule_check,
flg_applied_loan,
--flg_approved_scorecard_loan,
flg_disbursed_loan,
CASE WHEN cddRejectReason IS NOT NULL and cddStatus <> 'ACTIVATED' THEN 1 ELSE 0 END flg_cdd_reject_flag,
CASE WHEN rejectedStage = 'Stage0' THEN 1 ELSE 0 END flg_hard_reject_flag, 
CASE WHEN rejectedStage = 'Stage1' THEN 1 ELSE 0 END flg_taran_reject_flag, 
case when digitalloan.poi3RejectType in ('SOFT','HARD') and digitalloan.poi3StatusCode like 'RJ%' then 1 else 0 end ln_taran_rule_reject_flag,
case when poi3.uw_flag in ('Streamline UW','Full UW') then 1 else 0 end ln_marked_underwriter_check_flag,

-- CASE WHEN cddRejectReason IS NOT NULL and cddStatus = 'ACTIVATED' THEN 1 ELSE 0 END flg_cdd_approved_flag,
-- CASE WHEN rejectedStage <> 'Stage0' THEN 1 ELSE 0 END flg_not_hard_uw_approved, 
-- CASE WHEN rejectedStage <> 'Stage1' THEN 1 ELSE 0 END flg_taran_approved_flag, 

CASE WHEN loanmaster.applicationstatus='APPROVED' THEN 1 ELSE 0 END flg_loan_approved_not_disbursed,
digitalloan.loanRequestAmount loan_chosen_principal,digitalloan.loanRequestTenure loan_chosen_tenor,
coalesce(disbursedLoanAmount,if(loanmaster.applicationstatus='APPROVED',loanmaster.approvedLoanAmount,NULL)) loan_approved_principal,
coalesce(disbursedLoanTenur,if(loanmaster.applicationstatus='APPROVED',loanmaster.approvedLoanTenure,NULL)) loan_approved_tenor,
loan_defFPD30,loan_defSPD30,loan_defTPD30,defpd10_outstanding_principal,defpd30_outstanding_principal,defspd30_outstanding_principal,deftpd30_outstanding_principal,loanmaster.credolabRefNumber,
loanmaster.docType,

coalesce(osversion_credolab.credolab_osversion,osversion) onb_osversion,	
coalesce(ln_model_credolab.ln_model,osversion_credolab.onb_model) ln_model,			   
cust_mtb.status,
onb_self_dec_income.monthly_income onb_self_dec_income,
onb_self_dec_income.company_name onb_company_name,
cust_closed_date.OFDATECLOSED cust_status_close_date,
credolab_creation.createdOn credo_inquiry_date,
purpose.source_of_fund_name purpose_source_of_fund_name,purpose.nature_of_work_name purpose_nature_of_work_name,purpose.employment_status_name purpose_employment_status_name,purpose.industryName purpose_industryName,purpose.industryid pupose_industryid,purpose.created_dt purpose_created_dt,
loan_level.industry,loan_level.natureofwork,
loan_level.employmentstatus,
loan_level.source_of_funds,
ln_mail.email ln_email
																		
--CASE WHEN loanPaidStatus in ('Normal','In Arrears') THEN 1 ELSE 0 END count_ongoing_loans 

from risk_credit_mis.loan_master_table loanmaster
inner join `dl_loans_db_raw.tdbk_digital_loan_application` digitalloan on loanmaster.digitalLoanAccountId=digitalloan.digitalLoanAccountId
left join `dl_customers_db_raw.tdbk_customer_mtb` cust_mtb on cast(loanmaster.customerId as string)=cust_mtb.cust_id
left join dl_loans_db_raw.tdbk_digital_loan_application digital on digital.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join dl_loans_db_raw.tdbk_loan_purpose purpose on purpose.loanPurposeId=digital.loanPurposeId
left join cust_info on cust_info.digitalLoanAccountId=loanmaster.digitalLoanAccountId
LEFT JOIN dl_dynamo_db_raw.tdbk_nature_of_work_mtb nat
ON cast(nat.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(3)]
LEFT JOIN dl_dynamo_db_raw.tdbk_employment_status_mtb emp_status
ON cast(emp_status.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(0)]
LEFT JOIN dl_dynamo_db_raw.tdbk_source_of_funds_mtb source_fund
ON cast(source_fund.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(4)]
LEFT JOIN dl_dynamo_db_raw.tdbk_industry_list_mtb industry
ON cast(industry.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(6)]
left join educationtype on educationtype.digitalLoanAccountId=digitalloan.digitalLoanAccountId
left join ref_type on ref_type.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join `risk_credit_mis.loan_level_master_table` loan_level on loan_level.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join (SELECT * FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_vas_holiday_htb`  where  req_type=1
qualify row_number() over (partition by digitalLoanAccountId order by repayment_number desc)=1) vasflag on loanmaster.digitalLoanAccountId=vasflag.digitalLoanAccountId
LEFT JOIN (SELECT customer_id,    latitude,    longitude,  FROM    `risk_mart.customer_gps_location`
  WHERE event_description = 'Apigee Logs' AND latitude <> 'undefined' AND longitude <> 'undefined' 
  QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id order by gps_collection_date desc ) =1 ) loc on cast(loc.customer_id as int64)=loanmaster.customerId
  LEFT JOIN (select digitalLoanAccountId,max(installmentAmount) monthlytermamount from `risk_credit_mis.loan_installments_table` instal
--join core_raw_staging.loan_accounts loanacct on instal.loanAccountNumber=loanacct.ACCOUNTNUMBER  where LOANSTATUS in ('Normal','In Arrears')  
group by 1 ) loanaccts on loanaccts.digitalLoanAccountId = loanmaster.digitalLoanAccountId 
left join user_type_min on cast(user_type_min.customerId as string)=cust_mtb.cust_id 
left join user_type_loan_level on user_type_loan_level.digitalLoanAccountId=loanmaster.digitalLoanAccountId 
left join user_type_loan_level_v1 on user_type_loan_level_v1.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join worktable_data_analysis.lov_sf_industry_now_emp_status_apl15 empstatus on empstatus.digitalLoanAccountId=loanmaster.digitalLoanAccountId
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  LEFT JOIN dl_customers_db_raw.tdbk_cust_profile_mtb cust_profile on cust_profile.user_id = cust_mtb.user_id
  left join dl_customers_db_raw.tdbk_customer_add_mtb  per_addr on per_addr.user_id = cust_mtb.user_id
   left join deliquency on loanmaster.loanAccountNumber=deliquency.loanAccountNumber
   left join default_outstanding_principal on default_outstanding_principal.digitalLoanAccountId=loanmaster.digitalLoanAccountId
   left join `dl_loans_db_raw.tdbk_loan_poi3_response` poi3 on loanmaster.crifApplicationId=poi3.crifApplicationId
left join (SELECT digitalLoanAccountId,score FROM `dl_loans_db_derived.tdbk_loan_poi3_response_trustingsocial_data` 
qualify row_number() over (partition by digitalLoanAccountId order by timestamp_received desc)=1) trustingsocial on 
loanmaster.digitalLoanAccountId=trustingsocial.digitalLoanAccountId
 left join (select digitalLoanAccountId, MAX(coalesce(cast(FlagMatched as int64),0)) as matched_flag from dl_loans_db_derived.tdbk_loan_poi3_response_bureau_string_xml_parsed_matchedsubject
group by 1) flagmatched
on loanmaster.digitalLoanAccountId = flagmatched.digitalLoanAccountId
left join 
(SELECT digitalLoanAccountId,if(substr(ScoreRange,1,3)='NH_',0,1) summary_matchedflag,ScoreRange FROM `dl_loans_db_derived.tdbk_loan_poi3_response_bureau_string_xml_parsed_summary` 
qualify row_number() over (partition by digitalLoanAccountId order by run_date desc)=1) summary 
on summary.digitalLoanAccountId = loanmaster.digitalLoanAccountId

--only applied loans 2 condition's has to check 
left join credo_score_static_insght credo_score_dima on credo_score_dima.digitalLoanAccountId=loanmaster.digitalLoanAccountId
--worktable_data_analysis.dema_score_final_may12 credo_score_dima  on 
--credo_score_dima.reference_number=loanmaster.credolabRefNumber 
--and loanmaster.credolabRefNumber is not null
left join (select userId,refno,createdOn from  `dl_loans_db_raw.tdbk_credolab_track`  qualify  row_number() over (partition by userId order by createdOn)=1) onb_osversion 
on onb_osversion.userId=digitalloan.userid
left join osversion_credolab on onb_osversion.refno=osversion_credolab.deviceid
left join ln_model_credolab on loanmaster.credolabRefNumber=ln_model_credolab.deviceid
left join onb_self_dec_income on onb_self_dec_income.cust_id=cast(loanmaster.customerId as string)
left join cust_closed_date on cust_closed_date.OFCUSTOMERID=cust_mtb.cust_id 
left join credolab_creation on credolab_creation.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join datalake_worktables.customer_mobile_mail_dtls ln_mail on ln_mail.cust_id=cast(loanmaster.customerId as string)																																																														   
and (coalesce(loanmaster.termsAndConditionsSubmitDateTime,if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)) between DATETIME(ln_mail.valid_from_dt)   and  DATETIME(ln_mail.valid_to_dt))																							
left join (select * from `prj-prod-dataplatform.dl_loans_db_raw.tdbk_customer_loan_address` 
where status=1 qualify row_number() over (partition by digital_loan_account_id order by created_dt desc)=1) loanaddress on loanaddress.digital_loan_account_id=loanmaster.digitalLoanAccountId
															   
where coalesce(loanmaster.termsAndConditionsSubmitDateTime,if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)) 
between '{SDATE}' 
-- and date_sub(date(current_date()), interval 1 day) 
and '{EDATE}'
 --and loanmaster.credolabRefNumber is not null
 ) main
qualify row_number() over(partition by main.digitalLoanAccountId)=1
;
"""

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema1}.{altrans} created successfully')

Table risk_mart.applied_loans_20210701_20250818_trans created successfully


# gamma Model Trench2 Ever applied snapshot customer

# Gamma Model Trench2 Ever Applied Snapshot Table

In [12]:
schema2 = 'worktable_data_analysis'

OBDATE = '2025-08-18'
# change it to obdate from current date if the requirement is for a specific date
SDATE = '2023-01-01'
EDATE = '2025-08-18'
# change it to obdate from current date if the requirement is for a specific date

formatted_date = datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')

print(formatted_date)  # Output: 20230101

gammatrench2everappliedcustomersnapshot = f'gamma_model_trench2_ever_applied_snapshot_{datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')}_{datetime.strptime(EDATE, '%Y-%m-%d').strftime('%Y%m%d')}'

print(gammatrench2everappliedcustomersnapshot)


20230101
gamma_model_trench2_ever_applied_snapshot_20230101_20250818


In [13]:
sq = f""" 
CREATE OR REPLACE TABLE {schema2}.{gammatrench2everappliedcustomersnapshot} as
with eligible_customers as (
SELECT 
cast(customerId as string) customerid,
credolabRefNumber,
credo_inquiry_date,
ln_loan_type,
onb_age,
ln_os_type,
ln_prod_type,
ln_chosen_tenor,
ln_chosen_principal,
onb_tsa_onboarding_datetime,
digitalloanaccountid,
ln_age,
ln_city,
ln_province,
onb_doc_type,
ln_self_dec_income,
onb_email,
ln_marital_status,
ln_industry_new,
ln_employment_type_new,
ln_education_level,
DATE('{OBDATE}') as snapshot_date,
DATE_DIFF(DATE('{OBDATE}'),DATE(onb_tsa_onboarding_datetime),DAY) dob_snapshot_date,
DATE_DIFF(DATE(onb_tsa_onboarding_datetime),credo_inquiry_date,DAY) days_since_credo_call_onb,
DATE_DIFF(DATE('{OBDATE}'),credo_inquiry_date,DAY) AS days_since_snapshot_date,
onb_self_dec_income,
ln_nature_of_work_new,
ln_source_funds_new,
onb_first_name,
onb_middle_name,
onb_last_name,
ln_brand,
ln_cnt_dependents,
ln_doc_type,
ln_osversion,
onb_osversion,
onb_kyc_status ln_kyc_status,
ln_loan_applied_flag,
ln_appln_submit_datetime,
ln_disb_flag,
c_credo_score,
s_credo_score,
FROM
{schema1}.{altrans}
where ln_appln_submit_datetime >= '{SDATE}'
),
cust_min_loan_disb_time as
(
  SELECT customerId,
  MIN(ln_appln_submit_datetime) AS min_loan_appln_time_disbursed_loan
  from eligible_customers
  where ln_disb_flag = 1 
  group by 1
),
customers_with_out_disb_loan as (
select  eligible_customers.customerid,onb_tsa_onboarding_datetime,onb_email,onb_doc_type,snapshot_date,
ln_nature_of_work_new,onb_osversion,c_credo_score,s_credo_score,min_loan_appln_time_disbursed_loan 
from eligible_customers
LEFT JOIN cust_min_loan_disb_time ON eligible_customers.customerid = cust_min_loan_disb_time.customerId
where ((eligible_customers.snapshot_date < min_loan_appln_time_disbursed_loan) OR (min_loan_appln_time_disbursed_loan IS NULL))

),
loans_within_90d as (
select eligible_customers.* from eligible_customers
LEFT JOIN cust_min_loan_disb_time ON eligible_customers.customerid = cust_min_loan_disb_time.customerId 
where date(ln_appln_submit_datetime) between date(onb_tsa_onboarding_datetime) and date(snapshot_date)
and ln_disb_flag = 0 and (eligible_customers.ln_appln_submit_datetime <= min_loan_appln_time_disbursed_loan OR min_loan_appln_time_disbursed_loan IS NULL)
QUALIFY ROW_NUMBER() OVER(PARTITION BY eligible_customers.customerId order by ln_appln_submit_datetime desc)= 1
),
input_customers as (
select  
eligible_customers.customerid,
eligible_customers.credolabRefNumber,
eligible_customers.credo_inquiry_date,
loans_within_90d.ln_loan_type ln_loan_type,
loans_within_90d.ln_loan_type ln_os_type,
loans_within_90d.ln_prod_type ln_prod_type,
eligible_customers.onb_tsa_onboarding_datetime,
loans_within_90d.digitalloanaccountid,
eligible_customers.dob_snapshot_date,
COALESCE(loans_within_90d.ln_age,eligible_customers.onb_age) ln_age,
loans_within_90d.ln_chosen_tenor,
loans_within_90d.ln_chosen_principal,
eligible_customers.days_since_snapshot_date days_since_credo_call_snapshot_date,
eligible_customers.days_since_credo_call_onb,
COALESCE(loans_within_90d.ln_self_dec_income,eligible_customers.onb_self_dec_income) ln_self_dec_income,
COALESCE(loans_within_90d.onb_email,eligible_customers.onb_email) ln_email,
COALESCE(loans_within_90d.ln_marital_status,eligible_customers.ln_marital_status) ln_marital_status,
COALESCE(loans_within_90d.ln_industry_new,eligible_customers.ln_industry_new) ln_industry_new,
COALESCE(loans_within_90d.ln_employment_type_new,eligible_customers.ln_employment_type_new) ln_employment_type_new,
COALESCE(loans_within_90d.ln_education_level,eligible_customers.ln_education_level) ln_education_level,
eligible_customers.snapshot_date,
COALESCE(loans_within_90d.ln_nature_of_work_new,eligible_customers.ln_nature_of_work_new) ln_nature_of_work_new,
COALESCE(loans_within_90d.ln_source_funds_new,eligible_customers.ln_source_funds_new) ln_source_funds_new,
COALESCE(loans_within_90d.onb_first_name,eligible_customers.onb_first_name) onb_first_name,
COALESCE(loans_within_90d.onb_middle_name,eligible_customers.onb_middle_name) onb_middle_name,
COALESCE(loans_within_90d.onb_last_name,eligible_customers.onb_last_name) onb_last_name,
COALESCE(loans_within_90d.ln_brand,eligible_customers.ln_brand) ln_brand,
COALESCE(loans_within_90d.ln_cnt_dependents,eligible_customers.ln_cnt_dependents) ln_cnt_dependents,
COALESCE(loans_within_90d.ln_doc_type,eligible_customers.onb_doc_type) ln_doc_type,
COALESCE(loans_within_90d.ln_osversion,eligible_customers.onb_osversion) ln_osversion,
COALESCE(loans_within_90d.ln_city,eligible_customers.ln_city) ln_city,
COALESCE(loans_within_90d.ln_province,eligible_customers.ln_province) ln_province,
loans_within_90d.ln_loan_applied_flag ln_loan_applied_flag,
loans_within_90d.ln_appln_submit_datetime,
loans_within_90d.ln_disb_flag,
COALESCE(loans_within_90d.c_credo_score,eligible_customers.c_credo_score) c_credo_score,
COALESCE(loans_within_90d.s_credo_score,eligible_customers.s_credo_score) s_credo_score
from (select  eligible_customers.customerid,onb_tsa_onboarding_datetime,onb_email,onb_doc_type,snapshot_date,ln_nature_of_work_new,onb_osversion,c_credo_score,s_credo_score,ln_self_dec_income,ln_marital_status,ln_industry_new,ln_employment_type_new,ln_education_level,days_since_credo_call_onb,days_since_snapshot_date,onb_self_dec_income,dob_snapshot_date,credolabRefNumber,credo_inquiry_date,onb_age,ln_city,ln_province,
ln_source_funds_new,
onb_first_name,
onb_middle_name,
onb_last_name,
ln_brand,
ln_cnt_dependents,
 from eligible_customers

qualify row_number() over (partition by customerid order by ln_appln_submit_datetime asc) = 1) eligible_customers
LEFT JOIN loans_within_90d ON loans_within_90d.customerid = eligible_customers.customerid
JOIN (SELECT DISTINCT customerid FROM customers_with_out_disb_loan) customers_with_out_disb_loan on eligible_customers.customerid = customers_with_out_disb_loan.customerid
),
first_reject_data as (
  SELECT eligible_customers.customerId,
  coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime)) AS first_reject_loan_appln_time,
  new_loan_type AS first_reject_loan_type,
  loanRequestTenure as first_reject_loan_tenor,
  loanRequestAmount as first_reject_loan_amount,
  applied_loans.digitalLoanAccountId as first_rejected_digitalloanaccountId,
  from `risk_credit_mis.loan_master_table`  applied_loans
  JOIN input_customers eligible_customers ON CAST(applied_loans.customerid AS STRING) = eligible_customers.customerid and coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime)) < snapshot_date
--LEFT JOIN cust_min_loan_disb_time ON eligible_customers.customerid = cust_min_loan_disb_time.customerId
  where  disbursementDateTime is null --and   ln_appln_submit_datetime <= snapshot_date and (eligible_customers.ln_appln_submit_datetime <= min_loan_appln_time_disbursed_loan OR min_loan_appln_time_disbursed_loan IS NULL)
  QUALIFY ROW_NUMBER() OVER(PARTITION BY customerId order by ln_appln_submit_datetime ASC) = 1
),
last_reject_data as (
  SELECT eligible_customers.customerId,
  coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime)) AS last_reject_loan_appln_time,
  new_loan_type AS last_reject_loan_type,
  loanRequestTenure as last_reject_loan_tenor,
  loanRequestAmount as last_reject_loan_amount,
  applied_loans.digitalLoanAccountId as last_rejected_digitalloanaccountId,
  from `risk_credit_mis.loan_master_table`  applied_loans
  JOIN input_customers eligible_customers ON CAST(applied_loans.customerid AS STRING) = eligible_customers.customerid and coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime)) < snapshot_date
--LEFT JOIN cust_min_loan_disb_time ON eligible_customers.customerid = cust_min_loan_disb_time.customerId
  where  disbursementDateTime is null --and   ln_appln_submit_datetime <= snapshot_date and (eligible_customers.ln_appln_submit_datetime <= min_loan_appln_time_disbursed_loan OR min_loan_appln_time_disbursed_loan IS NULL)
  QUALIFY ROW_NUMBER() OVER(PARTITION BY customerId order by ln_appln_submit_datetime DESC) = 1
),

Reject_flag_data_new as (SELECT digitalLoanAccountId,
case when  applicationStatus IN ('EXPIRED', 'EXEMPT') and prev_applicationStatus='REJECT' THEN 1 when applicationStatus ='REJECT' THEN 1 else 0 end reject_flag,
case when  applicationStatus IN ('ACCEPT', 'CANCELLED','EXEMPT','EXPIRED','REJECT') and prev_applicationStatus='APPROVED' THEN 1 when applicationStatus ='APPROVED' THEN 1 else 0 end approved_flag

 FROM (SELECT digitalLoanAccountId, applicationStatus, created_dt, LEAD(applicationStatus) OVER(PARTITION BY digitalLoanAccountId order by created_dt desc, statusTraceId desc) AS  prev_applicationStatus,LEAD(created_dt) OVER(PARTITION BY digitalLoanAccountId order by created_dt desc, statusTraceId desc) AS prev_created_dt,  row_number() over (partition by digitalLoanAccountId order by created_dt desc, statusTraceId desc ) rn  from dl_loans_db_raw.tdbk_status_trace ) where rn =1),
first_applied_loan_data as (
  SELECT 
  applied_loans.customerId,
  COALESCE(applied_loans.termsAndConditionsSubmitDateTime,startApplyDateTime) AS first_applied_loan_appln_time,
  new_loan_type AS first_applied_loan_type,
  loanRequestTenure as first_applied_loan_tenor,
  loanRequestAmount as first_applied_loan_amount,
  --applied_loans.applicationStatus,
  --reject_flag, approved_flag,
  CASE WHEN reject_flag = 1 or applied_loans.applicationStatus in ('REJECT','EXPIRED','RESET','EXEMPT','CANCELLED')  THEN 'REJECT'
  WHEN reject_flag =0 and (approved_flag = 1 or applied_loans.applicationStatus in ('APPROVED','COMPLETED','ACTIVATED')) THEN 'APPROVED'
  --ELSE lmt.applicationStatus
  END AS first_applied_loan_decision,
  case when applied_loans.loantype='BNPL' and store_type =1 then 'Appliance'
    when applied_loans.loantype='BNPL' and store_type =2 then 'Mobile' 
    when applied_loans.loantype='BNPL' and store_type =3 then 'Mall' 
    when applied_loans.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as first_applied_product_type,
  from `risk_credit_mis.loan_master_table`  applied_loans
JOIN input_customers eligible_customers
ON cast(applied_loans.customerid as string) = eligible_customers.customerid and COALESCE(applied_loans.termsAndConditionsSubmitDateTime) < snapshot_date
JOIN Reject_flag_data_new on Reject_flag_data_new.digitalLoanAccountId = applied_loans.digitalLoanAccountId
left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on applied_loans.purpleKey=sil_category.mer_refferal_code
--LEFT JOIN cust_min_loan_disb_time ON eligible_customers.customerid = cust_min_loan_disb_time.customerId
--where (eligible_customers.snapshot_date <= min_loan_appln_time_disbursed_loan OR min_loan_appln_time_disbursed_loan IS NULL)
  QUALIFY ROW_NUMBER() OVER(PARTITION BY customerId order by COALESCE(applied_loans.termsAndConditionsSubmitDateTime) ASC) = 1
),
last_applied_loan_data as (
  SELECT eligible_customers.customerId,
  COALESCE(termsAndConditionsSubmitDateTime,startApplyDateTime) AS last_applied_loan_appln_time,
  eligible_customers.ln_os_type as last_applied_os_type,
  new_loan_type AS last_applied_loan_type,
  loanRequestTenure as last_applied_loan_tenor,
  loanRequestAmount as last_applied_loan_amount,
  lmt.digitalLoanAccountId as last_applied_crif_id,
  case when lmt.loantype='BNPL' and store_type =1 then 'Appliance'
    when lmt.loantype='BNPL' and store_type =2 then 'Mobile' 
    when lmt.loantype='BNPL' and store_type =3 then 'Mall' 
    when lmt.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as last_applied_product_type,
  CASE WHEN reject_flag = 1 or lmt.applicationStatus in ('REJECT','EXPIRED','RESET','EXEMPT','CANCELLED')  THEN 'REJECT'
  WHEN approved_flag = 1 or lmt.applicationStatus in ('APPROVED','COMPLETED','ACTIVATED') THEN 'APPROVED'
  END AS last_applied_loan_decision,
  CASE WHEN  lmt.new_loan_type LIKE 'SIL%' THEN s_credo_score
  WHEN lmt.new_loan_type in ('Quick','Flex','Flex-up','Big Loan','ACL TSA') THEN c_credo_score
  END AS last_applied_credo_score,
  lmt.credolabRefNumber as last_applied_credo_ref_no
  from `risk_credit_mis.loan_master_table` lmt
JOIN input_customers eligible_customers ON cast(lmt.customerid as string) = eligible_customers.customerid 
JOIN Reject_flag_data_new on Reject_flag_data_new.digitalLoanAccountId = lmt.digitalLoanAccountId
left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on lmt.purpleKey=sil_category.mer_refferal_code
where  COALESCE(termsAndConditionsSubmitDateTime) < snapshot_date
  QUALIFY ROW_NUMBER() OVER(PARTITION BY customerId order by COALESCE(termsAndConditionsSubmitDateTime) DESC) = 1
)

SELECT 
acc.*,
    first_reject_data.* except(customerid),
    last_reject_data.* except(customerid),
    first_applied_loan_data.* EXCEPT(customerid),
    last_applied_loan_data.* except(customerid),

FROM input_customers acc
LEFT JOIN first_applied_loan_data on acc.customerId = cast(first_applied_loan_data.customerid as string)
LEFT JOIN last_applied_loan_data on acc.customerId = cast(last_applied_loan_data.customerid as string)
LEFT JOIN first_reject_data on acc.customerId = cast(first_reject_data.customerid as string)
LEFT JOIN last_reject_data on acc.customerId = cast(last_reject_data.customerid as string)

"""

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema2}.{gammatrench2everappliedcustomersnapshot} created successfully')

Table worktable_data_analysis.gamma_model_trench2_ever_applied_snapshot_20230101_20250818 created successfully


# Gamma Model trench2_ever_applied_snapshot_transactions

# trench2_ever_applied_snapshot_transactions Table

In [14]:

trench2everappliedsnapshottransactions = f'trench2_ever_applied_snapshot_transactions{datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')}_{datetime.strptime(EDATE, '%Y-%m-%d').strftime('%Y%m%d')}'

print(trench2everappliedsnapshottransactions)


trench2_ever_applied_snapshot_transactions20230101_20250818


# trench2_ever_applied_snapshot_transactions Query

In [15]:
sq = f""" 
create or replace table {schema2}.{trench2everappliedsnapshottransactions} as 
WITH input_customers as (
  select * from  {schema2}.{gammatrench2everappliedcustomersnapshot}
),
Reject_flag_data_new as (SELECT digitalLoanAccountId,applicationStatus,
case when  applicationStatus IN ('EXPIRED', 'EXEMPT') and prev_applicationStatus='REJECT' THEN 1 when applicationStatus ='REJECT' THEN 1 else 0 end reject_flag,
case when  applicationStatus IN ('ACCEPT', 'CANCELLED','EXEMPT','EXPIRED','REJECT') and prev_applicationStatus='APPROVED' THEN 1 when applicationStatus ='APPROVED' THEN 1 else 0 end approved_flag

 FROM (SELECT digitalLoanAccountId, applicationStatus, created_dt, LEAD(applicationStatus) OVER(PARTITION BY digitalLoanAccountId order by created_dt desc, statusTraceId desc) AS  prev_applicationStatus,LEAD(created_dt) OVER(PARTITION BY digitalLoanAccountId order by created_dt desc, statusTraceId desc) AS prev_created_dt,  row_number() over (partition by digitalLoanAccountId order by created_dt desc, statusTraceId desc ) rn  from dl_loans_db_raw.tdbk_status_trace ) where rn =1),
 delinquency_data as (
select loanAccountNumber,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30  in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from risk_credit_mis.loan_deliquency_data
),
loan_metrics as (
SELECT 
input_customers.customerid,


CASE WHEN COUNT(CASE WHEN ln_disb_flag = 1 AND ln_loan_type LIKE 'SIL%' THEN 1 ELSE NULL END) > 1 THEN 1 ELSE 0 END AS ln_any_prev_disb_loan_sil_mobile_flag,
count(DISTINCT(IF(coalesce(lmt.termsAndConditionsSubmitDateTime,if (lmt.new_loan_type ='Flex-up',lmt.startApplyDateTime,lmt.termsAndConditionsSubmitDateTime)) is not null,lmt.digitalLoanAccountId,NULL))) tx_cnt_applied_loan_apps,
count(DISTINCT(IF(Reject_flag_data_new.reject_flag = 1,lmt.digitalLoanAccountId,NULL))) tx_cnt_rejected_loan_apps,
count(DISTINCT(IF( lmt.applicationStatus in ('COMPLETED','ACTIVATED','APPROVED'),lmt.digitalLoanAccountId,NULL))) tx_cnt_approved_loan_apps,
count(DISTINCT(IF(lmt.applicationStatus = 'COMPLETED',lmt.digitalLoanAccountId,NULL))) tx_cnt_completed_loan_apps,
count(DISTINCT(IF(disbursementDateTime is not null ,lmt.digitalLoanAccountId,NULL))) tx_cnt_disbursed_loan_apps,
count(DISTINCT(IF(disbursementDateTime is not null and lmt.applicationStatus <> 'COMPLETED' ,lmt.digitalLoanAccountId,NULL))) tx_cnt_active_loan_apps,
count(DISTINCT(IF (lmt.digitalLoanAccountId IS NOT NULL AND coalesce(lmt.termsAndConditionsSubmitDateTime,if (lmt.new_loan_type ='Flex-up',lmt.startApplyDateTime,lmt.termsAndConditionsSubmitDateTime)) is null,lmt.digitalLoanAccountId,NULL))) tx_incomplete_loan_apps,

count(CASE WHEN defFPD05 = 1 THEN 1 ELSE NULL END) as cnt_fpd5,
count(CASE WHEN delinquency_data.defFPD10 = 1 THEN 1 ELSE NULL END) as cnt_fpd10,
count(CASE WHEN delinquency_data.defFPD30 = 1 THEN 1 ELSE NULL END) as cnt_fpd30,
count(CASE WHEN delinquency_data.deffspd30 = 1 THEN 1 ELSE NULL END) as cnt_fspd30,
COUNT(case when DPDwoToleranceCustom_DPD > 5 THEN 1 ELSE NULL END) AS cnt_dpd_gt_5,
MAX(DPDwoToleranceCustom_DPD) AS max_ever_dpd,
MAX(case when lastPaymentDate < DATE_SUB(DATE(input_customers.snapshot_date),INTERVAL 30 DAY) THEN DPDwoToleranceCustom_DPD ELSE NULL END) AS max_ever_dpd_30d,
MAX(case when lastPaymentDate < DATE_SUB(DATE(input_customers.snapshot_date),INTERVAL 60 DAY) THEN DPDwoToleranceCustom_DPD ELSE NULL END) AS max_ever_dpd_60d,
MAX(case when lastPaymentDate < DATE_SUB(DATE(input_customers.snapshot_date),INTERVAL 90 DAY) THEN DPDwoToleranceCustom_DPD ELSE NULL END) AS max_ever_dpd_90d,
MAX(case when lastPaymentDate < DATE_SUB(DATE(input_customers.snapshot_date),INTERVAL 120 DAY) THEN DPDwoToleranceCustom_DPD ELSE NULL END) AS max_ever_dpd_120d,
MAX(case when lastPaymentDate < DATE_SUB(DATE(input_customers.snapshot_date),INTERVAL 150 DAY) THEN DPDwoToleranceCustom_DPD ELSE NULL END) AS max_ever_dpd_150d,
MAX(case when lastPaymentDate < DATE_SUB(DATE(input_customers.snapshot_date),INTERVAL 180 DAY) THEN DPDwoToleranceCustom_DPD ELSE NULL END) AS max_ever_dpd_180d,
from input_customers
JOIN `risk_credit_mis.loan_master_table` lmt on cast(lmt.customerId as string) = input_customers.customerid and COALESCE(DATE(termsAndConditionsSubmitDateTime),DATE(startApplyDateTime))  < DATE(input_customers.snapshot_date)
LEFT JOIN `risk_credit_mis.loan_installments_table` a on a.loanAccountNumber = lmt.loanAccountNumber and lastPaymentDate < DATE(input_customers.snapshot_date)
LEFT JOIN Reject_flag_data_new ON Reject_flag_data_new.digitalLoanAccountId = lmt.digitalLoanAccountId
LEFT JOIN delinquency_data on delinquency_data.loanAccountNumber = lmt.loanAccountNumber
group by 1
),
cust_onboarding_acc_data as (
    SELECT
    DATE(opendate,'Asia/Manila') OFDATEOPENED,
    DATE(closuredate,'Asia/Manila') ofdateclosed,
    closed OFISCLOSED,
    DATE(c.created_dt) registration_date,
    c.created_dt as onboarding_date,
    DATETIME(reccreatedon,'Asia/Manila') reccreatedon,
    cust_id,
    accountid,
    productid,
    accountdescription as account_type,
    clearedbalance,
    snapshot_date
    FROM `dl_customers_db_raw.tdbk_customer_mtb` c    
    JOIN `finastra_raw.account` b ON c.cust_id = b.ubcustomercode
    JOIN input_customers on c.cust_id = CAST(input_customers.customerid AS STRING)
),
main_transaction_data AS 
(
     SELECT 
    transaction_date,
    OFDATEOPENED,
    OFISCLOSED,
    registration_date,
    transaction_id,
    b.cust_id customer_id,
    a.accountid,
    productid,
    b.account_type,
    transaction_code,
    a.status,
    channel,
    credit_debit_indicator,
    inter_exter_flag,
    trx_amount,
    core_narration,
    input_customers.snapshot_date
    -- customer_transactions to get the transactions
    FROM cust_onboarding_acc_data b
    JOIN input_customers on b.cust_id = CAST(input_customers.customerid AS STRING)
    LEFT JOIN `risk_mart.customer_transactions` a ON a.accountid = b.accountid
    and a.transaction_date <= input_customers.snapshot_date and a.status = 'Success'
    --and a.transaction_datetime < input_customers.snapshot_date


    WHERE 1=1
   
),


#### Net Cash In ####
  -- 1. Outside Tonik to TSA
  -- 2. Other Tonik user to Own Tonik Account


net_cash_in AS 
(
  ## 1. Outside Tonik to TSA
  SELECT
    transaction_date,
    OFDATEOPENED,
    OFISCLOSED,
    registration_date,
    transaction_id,
    customer_id,
    accountid,
    account_type,
    status,
    channel,
    credit_debit_indicator,
    inter_exter_flag,
    trx_amount,
    core_narration,
    'Net Cash In' main_transaction_type,
    'Outside Tonik to TSA' sub_transaction_type,
    snapshot_date
  FROM main_transaction_data
  WHERE 1=1
  -- main conditions: should be a successful transaciton and credit and all coming from Tonik Account
  AND credit_debit_indicator = 'CREDIT'
  AND account_type = 'Tonik Account' and LOWER(core_narration) NOT LIKE '%blocking%' and transaction_code not like 'A0%'
  AND transaction_code IN ('N01','IP2','XE2','00T','21C','P01')
  -- 1. Outside Tonik to TSA conditions (all cash in)
  AND inter_exter_flag = 'Outside Tonik'

  UNION ALL

  ## 2. Other Tonik user to Own Tonik Account
  SELECT
    transaction_date,
    OFDATEOPENED,
    OFISCLOSED,
    registration_date,
    transaction_id,
    customer_id,
    accountid,
    account_type,
    status,
    channel,
    credit_debit_indicator,
    inter_exter_flag,
    trx_amount,
    core_narration,
    'Net Cash In' main_transaction_type,
    'Other Tonik Users to Town Tonik Account' sub_transaction_type,
    snapshot_date
  FROM main_transaction_data
  WHERE 1=1
  -- main conditions: should be a successful transaciton and credit and all coming from Tonik Account
  AND credit_debit_indicator = 'CREDIT'
  AND account_type = 'Tonik Account' and LOWER(core_narration) NOT LIKE '%blocking%' and transaction_code not like 'A0%'
  AND transaction_code IN ('N01','IP2','XE2','00T','21C','P01')

  -- 2. Other Tonik user to Own Tonik Account
  AND inter_exter_flag = 'Inside Tonik'
  AND core_narration LIKE '%Receive money from other Tonik Account%'
  -- AND LEFT(core_narration,STRPOS(core_narration, ",")-1) = 'Receive money from other Tonik Account'
)

#### Net Cash Out ####
-- 1. Bills Pay
-- 2. Card Transactions
-- 3. Own TSA to other Tonik Users
-- 4. TSA to Outside Tonik

, net_cash_out AS 
(

 ## 1. Bills Pay
  SELECT 
    transaction_date,
    OFDATEOPENED,
    OFISCLOSED,
    registration_date,
    transaction_id,
    customer_id,
    accountid,
    account_type,
    status,
    channel,
    credit_debit_indicator,
    inter_exter_flag,
    trx_amount,
    core_narration,
    'Net Cash Out' main_transaction_type,
    'Bills Pay' sub_transaction_type,
    snapshot_date,
  FROM main_transaction_data
  WHERE 1=1
  -- main conditions: should be a successful transaciton and debit
  AND credit_debit_indicator = 'DEBIT'
  AND LOWER(core_narration) NOT LIKE '%blocking%'

  -- 1. Bills Pay
  AND channel = 'Billspay'


  UNION ALL

  ## 2. Card Transactions (Cash Out)
  SELECT
    a.transaction_date,
    a.OFDATEOPENED,
    a.OFISCLOSED,
    a.registration_date,
    a.transaction_id,
    a.customer_id,
    a.accountid,
    a.account_type,
    a.status,
    a.channel,
    a.credit_debit_indicator,
    a.inter_exter_flag,
    a.trx_amount,
    a.core_narration,
    'Net Cash Out' main_transaction_type,
    'Card Transactions (Cash Out)' sub_transaction_type,
    snapshot_date
  FROM main_transaction_data a
  -- 2. Card Transactions (Cash Out) -- using the table made above
  WHERE 1=1
  -- main conditions: should be a successful transaciton and debit and coming from tonik account
  AND a.credit_debit_indicator = 'DEBIT'
  AND a.account_type = 'Tonik Account'
  AND transaction_code like 'A0%' and core_narration not like '%Blocking%'

  UNION ALL

  ## 3. Own TSA to other Tonik Users
  SELECT DISTINCT
    transaction_date,
    OFDATEOPENED,
    OFISCLOSED,
    registration_date,
    transaction_id,
    customer_id,
    accountid,
    account_type,
    status,
    channel,
    credit_debit_indicator,
    inter_exter_flag,
    trx_amount,
    core_narration,
    'Net Cash Out' main_transaction_type,
    'Own TSA to Other Tonik Users' sub_transaction_type,
    snapshot_date
  FROM main_transaction_data a
  WHERE 1=1
  -- main conditions: should be a successful transaciton and debit
  AND a.credit_debit_indicator = 'DEBIT'
  AND a.account_type = 'Tonik Account'
  AND transaction_code not like 'A0%' and core_narration not like '%Blocking%'

  -- 3. Own TSA to other Tonik Users
  AND a.channel = 'Core transactions'
  AND a.inter_exter_flag = 'Inside Tonik'
  --AND LOWER(core_narration) LIKE '%send money to other tonik account%' 
  -- AND LOWER(core_narration) NOT LIKE '%scontri%'
  -- AND LOWER(core_narration) NOT LIKE '%stash%'
  -- AND LOWER(core_narration) NOT LIKE '%time deposit%'

  UNION ALL

  ## 4. TSA to Outside Tonik (Other banks)
  SELECT DISTINCT
    transaction_date,
    OFDATEOPENED,
    OFISCLOSED,
    registration_date,
    transaction_id,
    customer_id,
    accountid,
    account_type,
    status,
    channel,
    credit_debit_indicator,
    inter_exter_flag,
    trx_amount,
    core_narration,
    'Net Cash Out' main_transaction_type,
    'TSA to Outside Tonik (Other Banks)' sub_transaction_type,
    snapshot_date
  FROM main_transaction_data a
  WHERE 1=1
  -- main conditions: should be a successful transaciton and debit
  AND a.credit_debit_indicator = 'DEBIT'
  AND a.account_type = 'Tonik Account'
  AND core_narration not like '%Blocking%'

  -- channels not in core transactions and billspay with the flag as outside tonik are sending to other banks
  AND a.channel NOT IN  ('Core transactions','Billspay')
  AND a.inter_exter_flag = 'Outside Tonik'
)

, transactions_sub AS 
(
  -- merging the cash ins and cash outs
  SELECT DISTINCT *
  FROM net_cash_in 
  UNION ALL
  SELECT DISTINCT *
  FROM net_cash_out
)

, date_diff_sub AS 
(
    -- to get the date difference between 2 transactions (cash in and cash out)
    SELECT customer_id,
    'Overall' days_diff_type,
    DATE_DIFF(LEAD(transaction_date) OVER (PARTITION BY customer_id ORDER BY transaction_date,core_narration ASC),transaction_date,DAY) days_bt_trans,
    snapshot_date
    FROM 
    (
        SELECT DISTINCT
        transaction_date,
        customer_id,
        main_transaction_type,
        snapshot_date,
        core_narration
        FROM transactions_sub
        WHERE 1=1
        --   AND customer_id IN ('2077378','2081999','2475220','2485072')
    )

    UNION ALL

    -- to get the date difference between 2 cash ins
    SELECT customer_id,
    'Cash In' days_diff_type,
    DATE_DIFF(LEAD(transaction_date) OVER (PARTITION BY customer_id ORDER BY transaction_date,core_narration ASC),transaction_date,DAY) days_bt_trans,
    snapshot_date
    FROM 
    (
        SELECT DISTINCT
        transaction_date,
        customer_id,
        main_transaction_type,
        snapshot_date,
        core_narration
        FROM transactions_sub
        WHERE 1=1
        --   AND customer_id IN ('2077378','2081999','2475220','2485072')
        AND main_transaction_type = 'Net Cash In'
    )

    UNION ALL

    -- to get the date difference between 2 cash outs
    SELECT customer_id,
    'Cash Out' days_diff_type,
    DATE_DIFF(LEAD(transaction_date) OVER (PARTITION BY customer_id ORDER BY transaction_date,core_narration ASC),transaction_date,DAY) days_bt_trans,
    snapshot_date
    FROM 
    (
        SELECT DISTINCT
        transaction_date,
        customer_id,
        main_transaction_type,
        snapshot_date,
        core_narration
        FROM transactions_sub
        WHERE 1=1
        --   AND customer_id IN ('2077378','2081999','2475220','2485072')
        AND main_transaction_type = 'Net Cash Out'
    )
)

, days_bt_trans_avg AS 
(
-- get the average days in between 
SELECT DISTINCT
customer_id,
AVG(IF(days_diff_type='Overall',days_bt_trans,NULL)) overall_avg_days_bt_trans,
AVG(IF(days_diff_type='Cash In',days_bt_trans,NULL)) net_cash_in_avg_days_bt_trans,
AVG(IF(days_diff_type='Cash Out',days_bt_trans,NULL)) net_cash_out_avg_days_bt_trans
FROM date_diff_sub
GROUP BY 1
)

, days_bt_trans_med AS 
(
-- get the median days in between
SELECT DISTINCT
customer_id,
PERCENTILE_CONT(IF(days_diff_type='Overall',days_bt_trans,NULL), .50) OVER (PARTITION BY customer_id) overall_med_days_bt_trans,
PERCENTILE_CONT(IF(days_diff_type='Cash In',days_bt_trans,NULL), .50) OVER (PARTITION BY customer_id) cash_in_med_days_bt_trans,
PERCENTILE_CONT(IF(days_diff_type='Cash Out',days_bt_trans,NULL), .50) OVER (PARTITION BY customer_id) cash_out_med_days_bt_trans,
FROM date_diff_sub
)

, transactions_final AS 
(
SELECT DISTINCT
acc.customerid as customer_id,
## Number of transactions within the observation window (x days from onboarding date),
## Cash In Count Details
COUNT(DISTINCT(IF(main_transaction_type = 'Net Cash In',transaction_id,NULL))) tx_cnt_cash_in_total,
COUNT(DISTINCT(IF(sub_transaction_type='Outside Tonik to TSA' ,transaction_id,NULL))) tx_cnt_cash_in_ob2t,
COUNT(DISTINCT(IF(sub_transaction_type='Other Tonik Users to Town Tonik Account' ,transaction_id,NULL))) tx_cnt_cash_in_ot2t,

## Cash In Amount Details
SUM(IF(main_transaction_type = 'Net Cash In',trx_amount,NULL)) tx_amt_cash_in_total,
SUM((IF(sub_transaction_type='Outside Tonik to TSA' ,trx_amount,NULL))) tx_amt_cash_in_ob2t,
SUM((IF(sub_transaction_type='Other Tonik Users to Town Tonik Account',trx_amount,NULL))) tx_amt_cash_in_ot2t,

## Cash Out Count Details
COUNT(DISTINCT(IF(main_transaction_type = 'Net Cash Out',transaction_id,NULL))) tx_cnt_cash_out_total,
COUNT(DISTINCT(IF(sub_transaction_type= 'Bills Pay' ,transaction_id,NULL))) tx_cnt_cash_out_billpay,
COUNT(DISTINCT(IF(sub_transaction_type= 'Card Transactions (Cash Out)' ,transaction_id,NULL))) tx_cnt_cash_out_cards,
COUNT(DISTINCT(IF(sub_transaction_type= 'Own TSA to Other Tonik Users' ,transaction_id,NULL))) tx_cnt_cash_out_t2ot,
COUNT(DISTINCT(IF(sub_transaction_type= 'TSA to Outside Tonik (Other Banks)' ,transaction_id,NULL))) tx_cnt_cash_out_t2ob,

## Cash Out Amount Details
SUM(IF(main_transaction_type = 'Net Cash Out',trx_amount,NULL)) tx_amt_cash_out_total,
SUM(IF(sub_transaction_type= 'Bills Pay' ,trx_amount,NULL)) tx_amt_cash_out_billpay,
SUM(IF(sub_transaction_type= 'Card Transactions (Cash Out)' ,trx_amount,NULL)) tx_amt_cash_out_cards,
SUM(IF(sub_transaction_type= 'Own TSA to Other Tonik Users' ,trx_amount,NULL)) tx_amt_cash_out_t2ot,
SUM(IF(sub_transaction_type= 'TSA to Outside Tonik (Other Banks)' ,trx_amount,NULL)) tx_amt_cash_out_t2ob,
FROM input_customers acc 
LEFT JOIN transactions_sub a ON CAST(acc.customerid AS STRING) = a.customer_id and DATE(transaction_date) < DATE(acc.snapshot_date)
GROUP BY 1
ORDER BY 2 

),

utility_transaction_data AS (
        SELECT 
        customer_id,
        CASE 
            WHEN SUM(CASE WHEN transaction_code IN ('BP1', 'BP2', 'BP3', 'BP4') THEN 1 ELSE 0 END) > 0 
                THEN MIN(transaction_date) 
            ELSE NULL 
        END AS first_billpay_date,
        CASE 
            WHEN SUM(CASE WHEN transaction_code like 'A0%' AND core_narration NOT LIKE '%Blocking%' THEN 1 ELSE 0 END) > 0 
                THEN MIN(transaction_date) 
            ELSE NULL 
        END AS virtual_transaction_date,
        CASE 
            WHEN SUM(CASE WHEN transaction_code IN ('21C', 'N01', 'IP2', 'XE2','P01') THEN 1 ELSE 0 END) > 0 
                THEN MIN(transaction_date) 
            ELSE NULL 
        END AS first_tsa_topup_date
    FROM 
        main_transaction_data
    GROUP BY 1
),

combined_data AS (
    SELECT 
        COALESCE(acc.cust_id,acc_data.customer_id,utility_transaction_data.customer_id) customer_id,
        productid,
        accountdescription,
        acc_data.opendate as opendate ,
        first_billpay_date,
        virtual_transaction_date,
        first_tsa_topup_date,
        snapshot_date,
        termsAndConditionsSubmitDateTime,
        LEAST(
            IFNULL(DATE(termsAndConditionsSubmitDateTime),'9999-12-31'),
            IFNULL(DATE(acc_data.opendate),'9999-12-31'), 
            IFNULL(first_billpay_date, '9999-12-31'), 
            IFNULL(virtual_transaction_date, '9999-12-31'), 
            IFNULL(first_tsa_topup_date, '9999-12-31')
        ) AS first_opened_date
        FROM (SELECT DISTINCT cust_id,registration_date,snapshot_date from cust_onboarding_acc_data) acc 
        LEFT JOIN (SELECT cust_id as customer_id,ofdateopened opendate ,productid,account_type as accountdescription from cust_onboarding_acc_data 
        WHERE  account_type NOT IN ('Tonik Account','Tendo Individual Stash') AND ofdateopened >= '2023-01-01' AND productid in ('fixdep','savings','SaveForFuture') AND ofdateopened >= '2023-01-01' AND reccreatedon < snapshot_date
       QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY reccreatedon asc) = 1) acc_data
       ON acc.cust_id = acc_data.customer_id
      LEFT JOIN utility_transaction_data
        ON 
      acc.cust_id = utility_transaction_data.customer_id
      LEFT JOIN (SELECT customerid,termsAndConditionsSubmitDateTime FROM risk_credit_mis.loan_master_table where termsAndConditionsSubmitDateTime is not null QUALIFY ROW_NUMBER() OVER (PARTITION BY customerid order by termsAndConditionsSubmitDateTime asc ) =1) lmt on cast(lmt.customerid as string) = acc.cust_id and DATE(lmt.termsAndConditionsSubmitDateTime) < DATE(snapshot_date)
),
first_product_data as (
SELECT
    customer_id,
    --first_opened_date,
    CASE
        WHEN first_opened_date = DATE(opendate) THEN accountdescription
        WHEN first_opened_date = first_billpay_date THEN 'Bills Pay'
        WHEN first_opened_date = first_tsa_topup_date THEN 'TSA Top-Up'
        WHEN first_opened_date = virtual_transaction_date THEN 'Virtual Transaction'
        ELSE 'Unknown'
    END AS first_product,
    CASE
        WHEN first_opened_date = DATE(opendate) and productid in ('fixdep','savings','SaveForFuture') THEN 'Deposit Users'
        --WHEN first_opened_date = DATE(opendate) and productid NOT IN ('fixdep','savings','SaveForFuture') THEN 'Loan Users'
        WHEN first_opened_date = first_tsa_topup_date or first_opened_date = virtual_transaction_date or first_opened_date = first_billpay_date THEN 'Utility Users'
        WHEN first_opened_date = DATE(termsAndConditionsSubmitDateTime) THEN 'Loan Attempt'
        ELSE 'Ghost Users'
    END AS first_product_user_segment
FROM
    combined_data
),

complete_deposit_metrics as (
with  deposit_acc_main AS 
(
  SELECT
  a.OFDATEOPENED as ofdateopened,
  IF(OFISCLOSED = 'Y',DATE_DIFF(date(ofdateclosed),date(OFDATEOPENED),day),NULL) as stash_duration,
  registration_date,
  duration,
  reccreatedon,
  a.cust_id as customer_id,
  a.account_type,
  a.accountid as ofstandardaccountid,
  --balancedateasof,
  b.clearedbalance,
  a.OFISCLOSED as closed,
  ff.status as td_status,
  a.ofdateclosed,
  autorollover,
  snapshot_date,
  FROM cust_onboarding_acc_data a
  JOIN risk_mart.customer_balance b on a.accountid = b.accountid and date(balanceDateAsOf) = DATE_SUB(DATE(snapshot_date),INTERVAL 1 DAY)
  --JOIN input_customers on input_customers.customerid = a.cust_id 
  LEFT JOIN `finastra_raw.fixturefeature` ff on ff.accountid = a.accountid
  WHERE 1=1 
  --and date(balancedateasof) = DATE_SUB(p.start_date,INTERVAL 1 DAY)
  and productid in ('savings','fixdep','SaveForFuture') and  DATE(reccreatedon) < DATE(snapshot_date)
  )
  
,deposit_days_diff_sub as (
    SELECT customer_id,
    'Between All Deposits' days_diff_type,
    DATE_DIFF(LEAD(ofdateopened) OVER (PARTITION BY customer_id ORDER BY ofdateopened ASC),ofdateopened,DAY) days_bt_trans
    FROM deposit_acc_main

    UNION ALL

    SELECT customer_id,
    'Between TDs' days_diff_type,
    DATE_DIFF(LEAD(ofdateopened) OVER (PARTITION BY customer_id ORDER BY ofdateopened ASC),ofdateopened,DAY) days_bt_trans
    FROM deposit_acc_main
    WHERE account_type not like '%Stash%'

)
,dep_days_bt_trans_med AS 
(
-- get the median days in between
SELECT DISTINCT
customer_id,

PERCENTILE_CONT(IF(days_diff_type='Between All Deposits',days_bt_trans,NULL), .50) OVER (PARTITION BY customer_id) med_days_bw_new_dep_acct_open,
PERCENTILE_CONT(IF(days_diff_type='Between TDs',days_bt_trans,NULL), .50) OVER (PARTITION BY customer_id) med_days_bw_td_acct_open,
FROM deposit_days_diff_sub
)

, deposit_account_counts AS 
(
#### Number of Stash and Time Deposit accounts that are still open until the observation date with balance >= 100
SELECT DISTINCT
customer_id,
CASE WHEN SUM(autorollover) >= 1.0 THEN 1 ELSE 0 END AS tx_td_auto_roll_over_enabled,
MAX((IF(account_type LIKE '%Time Deposit%' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) < DATE(snapshot_date)),duration,0))) AS td_max_duration,
AVG((IF(account_type LIKE '%Time Deposit%'and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)) ,duration,0))) AS td_avg_duration,
MIN((IF(account_type LIKE '%Time Deposit%'and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),duration,0))) AS td_min_duration,
MAX((IF(account_type LIKE '%Stash%' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),stash_duration,0))) AS stash_max_duration,
AVG((IF(account_type LIKE '%Stash%' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),stash_duration,0))) AS stash_avg_duration,
MIN((IF(account_type LIKE '%Stash%' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),stash_duration,0))) AS stash_min_duration,
SUM(DISTINCT(IF(account_type LIKE '%Time Deposit%' AND td_status = '1' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),clearedbalance,NULL))) td_balance,
SUM(DISTINCT(IF(account_type LIKE '%Stash%' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),clearedbalance,NULL))) stash_balance,
COUNT(DISTINCT(IF(account_type LIKE '%Time Deposit%' AND td_status = '4' and closed = 'Y' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),ofstandardaccountid,NULL))) td_accounts_completed_cnt,
COUNT(DISTINCT(IF(account_type LIKE '%Time Deposit%' AND td_status = '9' and closed = 'Y' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),ofstandardaccountid,NULL))) td_accounts_broken_cnt,
COUNT(DISTINCT(IF(account_type <> 'Tonik Account', ofstandardaccountid,NULL))) deposit_accs_cnt,
COUNT(DISTINCT(IF(account_type LIKE '%Stash%',ofstandardaccountid,NULL))) stash_accounts_opened_cnt,
COUNT(DISTINCT(IF(account_type LIKE '%Time Deposit%',ofstandardaccountid,NULL))) td_accounts_opened_cnt,
COUNT(DISTINCT(IF(account_type LIKE '%Stash%' and closed = 'Y' and (DATE(ofdateclosed) = '1970-01-01' OR  DATE(ofdateclosed) > DATE(snapshot_date)) ,ofstandardaccountid,NULL))) stash_accounts_closed_cnt,
FROM 
deposit_acc_main
where
DATE(reccreatedon) < DATE(snapshot_date) and account_type <> 'Tonik Account'
GROUP BY 1
)

SELECT DISTINCT 
a.customer_id,
td_balance,
stash_balance,
tx_td_auto_roll_over_enabled,
deposit_accs_cnt,
stash_accounts_opened_cnt,
stash_accounts_closed_cnt,
td_accounts_opened_cnt,
td_accounts_completed_cnt,
td_accounts_broken_cnt,
med_days_bw_new_dep_acct_open,
med_days_bw_td_acct_open,
td_max_duration,
td_min_duration,
td_avg_duration,
stash_max_duration,
stash_avg_duration,
stash_min_duration,
FROM deposit_account_counts a
--LEFT JOIN deposit_account_90th_day_counts d ON a.customer_id = d.customer_id
LEFT JOIN dep_days_bt_trans_med b ON b.customer_id = a.customer_id
ORDER BY 2 DESC
)
SELECT
COALESCE(first_product_data.first_product,'Unknown') as tx_first_product,
COALESCE(first_product_data.first_product_user_segment,'Ghost Users') tx_first_product_user_segment,
acc.customerid as customer_id,
onb_tsa_onboarding_datetime,
digitalloanaccountid,
ln_disb_flag,
ln_loan_type,
ln_prod_type,
ln_appln_submit_datetime as ln_loan_appln_time,
null as first_disb_loan_disbursed_time,
first_applied_loan_appln_time,
first_applied_loan_type,
first_applied_loan_tenor,
first_applied_loan_amount,
first_applied_loan_decision,
first_applied_product_type,
last_applied_loan_appln_time last_applied_loan_appln_time,
"REJECT" AS last_applied_loan_decision,
last_applied_os_type last_applied_os_type,
last_applied_loan_type last_applied_loan_type,
last_applied_loan_tenor last_applied_loan_tenor,
last_applied_loan_amount last_applied_loan_amount,
ln_prod_type last_applied_product_type,
last_applied_crif_id,
last_applied_credo_ref_no,
last_applied_credo_score,
null as ln_fpd30_flag,
null as ln_mature_fpd30_flag,
null as ln_mature_fspd30_flag,
null as ln_fspd30_flag,
first_reject_loan_appln_time,
first_rejected_digitalloanaccountId,
first_reject_loan_type,
first_reject_loan_amount,
first_reject_loan_tenor,
last_reject_loan_appln_time,
last_rejected_digitalloanaccountId,
last_reject_loan_type,
last_reject_loan_amount,
last_reject_loan_tenor,
credo_inquiry_date,
credolabRefNumber,
ln_appln_submit_datetime,
ln_age,
ln_email,
ln_industry_new,
ln_employment_type_new,
snapshot_date,
ln_nature_of_work_new,
ln_source_funds_new,
onb_first_name,
onb_middle_name,
onb_last_name,
ln_brand,
ln_cnt_dependents,
ln_doc_type,
ln_osversion,
ln_self_dec_income,
ln_marital_status,
ln_education_level,
c_credo_score,
1 as ln_ever_applied_flag,
dob_snapshot_date,
days_since_credo_call_onb,
days_since_credo_call_snapshot_date,
EXTRACT(YEAR FROM onb_tsa_onboarding_datetime) onb_year,
EXTRACT(MONTH FROM onb_tsa_onboarding_datetime) onb_month_of_year,
EXTRACT(WEEK FROM onb_tsa_onboarding_datetime) - EXTRACT(WEEK FROM DATE_TRUNC(onb_tsa_onboarding_datetime, MONTH)) + 1 onb_week_of_month,
EXTRACT(DAY FROM onb_tsa_onboarding_datetime) onb_day_of_month,
EXTRACT(TIME FROM onb_tsa_onboarding_datetime) onb_time_of_day,
a.tx_cnt_cash_in_total,
a.tx_cnt_cash_in_ob2t,
a.tx_cnt_cash_in_ot2t,
a.tx_amt_cash_in_total,
a.tx_amt_cash_in_ob2t,
a.tx_amt_cash_in_ot2t,
a.tx_cnt_cash_out_total,
a.tx_cnt_cash_out_billpay,
a.tx_cnt_cash_out_cards,
a.tx_cnt_cash_out_t2ot,
a.tx_cnt_cash_out_t2ob,
a.tx_amt_cash_out_total,
a.tx_amt_cash_out_billpay,
a.tx_amt_cash_out_cards,
a.tx_amt_cash_out_t2ot,
a.tx_amt_cash_out_t2ob,
overall_avg_days_bt_trans tx_avg_days_bt_trans,
net_cash_in_avg_days_bt_trans tx_avg_days_bt_cash_in_trans,
net_cash_out_avg_days_bt_trans tx_avg_days_bt_cash_out_trans,
overall_med_days_bt_trans tx_med_days_bt_trans,
cash_in_med_days_bt_trans tx_med_days_bt_cash_in_trans,
cash_out_med_days_bt_trans tx_med_days_bt_cash_out_trans,
deposit_accs_cnt tx_deposit_accnt_cnt,
stash_accounts_opened_cnt tx_stash_accnt_opened_cnt,
stash_accounts_closed_cnt tx_stash_accnt_closed_cnt,
stash_balance tx_stash_balance,
td_accounts_opened_cnt tx_td_accnt_opened_cnt,
td_accounts_completed_cnt tx_td_accnt_completed_cnt,
td_accounts_broken_cnt tx_td_accnt_broken_cnt,
tx_td_auto_roll_over_enabled,
td_balance tx_td_balance,
td_max_duration,
td_min_duration,
td_avg_duration,
stash_max_duration,
stash_avg_duration,
stash_min_duration,
med_days_bw_new_dep_acct_open tx_med_days_bw_new_dep_acct_open,
med_days_bw_td_acct_open tx_med_days_bw_td_acct_open,
tx_cnt_completed_loan_apps ln_cnt_completed_loans,
tx_cnt_rejected_loan_apps ln_cnt_rejected_loans,
tx_cnt_active_loan_apps ln_cnt_active_loans,
tx_cnt_applied_loan_apps ln_cnt_applied_loan,
tx_cnt_approved_loan_apps ln_cnt_approved_loans,
tx_cnt_disbursed_loan_apps ln_cnt_disbursed_loans,
tx_incomplete_loan_apps ln_cnt_incomplete_loan_apps,
null as tx_cnt_installments_paid_tot,
null as tx_amt_installments_paid_tot,
null as tx_cnt_installments_paid_last_disb,
null as tx_amt_installments_paid_last_disb,
null as ln_cnt_fpd10_ever,
null as ln_cnt_fpd30_ever,
null as ln_cnt_fspd30_ever,
null as ln_cnt_dpd_gt_5_ever,
null as ln_max_ever_dpd,
null as ln_max_dpd_30d,
null as ln_max_dpd_60d,
null as ln_max_dpd_90d,
null as ln_max_dpd_120d,
null as ln_max_dpd_150d,
null as ln_max_dpd_180d,
null as ln_any_prev_disb_loan_sil_mobile_flag,
CASE WHEN tdbk_referral_code_mtb.cust_id is not null THEN 1
ELSE 0 END AS referral_flag
FROM input_customers acc
LEFT JOIN first_product_data ON first_product_data.customer_id = acc.customerid 
LEFT JOIN transactions_final a ON acc.customerid = a.customer_id
LEFT JOIN days_bt_trans_avg b ON acc.customerid = b.customer_id 
LEFT JOIN days_bt_trans_med c ON acc.customerid = c.customer_id
LEFT JOIN complete_deposit_metrics d on d.customer_id = acc.customerid
--LEFT JOIN first_applied_loan_data on acc.customerId = cast(first_applied_loan_data.customerid as string)
--LEFT JOIN last_applied_loan_data on acc.customerId = cast(last_applied_loan_data.customerid as string)
--LEFT JOIN first_reject_data on acc.customerId = cast(first_reject_data.customerid as string)
--LEFT JOIN last_reject_data on acc.customerId = cast(last_reject_data.customerid as string)
LEFT JOIN loan_metrics ON cast(loan_metrics.customerid as string) = acc.customerid
LEFT JOIN (SELECT cust_id,member_type,referral_type FROM dl_customers_db_raw.tdbk_referral_code_mtb
WHERE tdbk_referral_code_mtb.member_type='REFEREE') tdbk_referral_code_mtb on  tdbk_referral_code_mtb.cust_id = acc.customerid
;
"""

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema2}.{trench2everappliedsnapshottransactions} created successfully')

Table worktable_data_analysis.trench2_ever_applied_snapshot_transactions20230101_20250818 created successfully


# trench2_ever_applied_snapshot_events

# trench2_ever_applied_snapshot_events Table

In [16]:

trench2everappliedsnapshotevents = f'trench2_ever_applied_snapshot_events{datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')}_{datetime.strptime(EDATE, '%Y-%m-%d').strftime('%Y%m%d')}'

print(trench2everappliedsnapshotevents)


trench2_ever_applied_snapshot_events20230101_20250818


# # trench2_ever_applied_snapshot_events query

In [17]:
sq = f""" 
CREATE OR REPLACE TABLE {schema2}.{trench2everappliedsnapshotevents} as
WITH input_customers as (
  select * from  {schema2}.{gammatrench2everappliedcustomersnapshot}
),
af_link AS
(
  ## To get the AF ID and Customer ID Link (using the first install of a customer)
  SELECT DISTINCT appsflyer_id, customer_user_id, install_time,snapshot_date
  FROM `appsflyer_raw.in_app_events_report` in_apps_events
  JOIN `dl_customers_db_raw.tdbk_customer_mtb` c ON c.cust_id = in_apps_events.customer_user_id
  JOIN input_customers ON In_apps_events.customer_user_id = input_customers.customerid and DATE(in_apps_events._partitiondate) < DATE(input_customers.snapshot_date)
  WHERE 1=1
  AND customer_user_id IS NOT NULL
  QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_user_id ORDER BY install_time ASC) = 1
 
  UNION ALL
 
  SELECT DISTINCT appsflyer_id, customer_user_id, install_time,snapshot_date
  FROM `appsflyer_raw.organic_in_app_events_report` organic_in_apps_events
  JOIN `dl_customers_db_raw.tdbk_customer_mtb` c ON c.cust_id = organic_in_apps_events.customer_user_id
 JOIN input_customers ON organic_in_apps_events.customer_user_id = input_customers.customerid and DATE(organic_in_apps_events._partitiondate) < DATE(input_customers.snapshot_date)
  WHERE 1=1
  AND customer_user_id IS NOT NULL
  QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_user_id ORDER BY install_time ASC) = 1
)
, events AS
(
  SELECT DISTINCT
  cust_id as customer_id,
  --snapshot_date,
COUNT(DISTINCT IF(event_name = 'App_Launch',event_uuid,NULL)) meng_no_of_logins,
COUNT(DISTINCT IF(
    event_name IN ('Loans_QL_Calculator'),moengagerefid,NULL)) meng_ql_calculator_count,
COUNT(DISTINCT IF(
    event_name IN ('Loans_QL_Calculator'),event_uuid,NULL)) meng_ql_calculator_tot_visit_cnt
  FROM `dl_customers_db_raw.tdbk_customer_mtb` c
  LEFT JOIN `moengage_raw.events_hourly` a ON c.cust_id = cast(a.customer_id as string)
 JOIN input_customers ON cast(a.customer_id as string) = input_customers.customerid 
  WHERE 1=1 and DATE(event_time) < DATE(input_customers.snapshot_date)
  --AND event_name IN ('App_Launch','Loans_QL_Calculator','Loans_QL_Launch','Loans_SIL_Launch')
  group by 1
  ),

campaign_data as
(  SELECT DISTINCT a.customer_user_id, a.appsflyer_id, a.media_source, a.partner, a.campaign, a.Retargeting_Conversion_Type, 
  CASE
  WHEN a.media_source = 'website_channel=website_ss_ui=true_ss_gtm_ui=true_ss_qr=c' THEN 'Website'
  WHEN a.media_source IS NULL AND a.partner IS NULL AND a.campaign IS NULL THEN 'Organic'
  ELSE b.source END source,
CASE
  WHEN a.media_source = 'website_channel=website_ss_ui=true_ss_gtm_ui=true_ss_qr=c' THEN 'Website'
  WHEN a.media_source IS NULL AND a.partner IS NULL AND a.campaign IS NULL THEN 'Organic'
  ELSE b.source_group END source_group,
  FROM 
  (
    SELECT DISTINCT
    install_time,
    customer_user_id,
    AppsFlyer_ID,
    media_source,
    partner,
    campaign,
    'Install' Retargeting_Conversion_Type,
    FROM `appsflyer_raw.in_app_events_report`
    WHERE 1=1

    UNION ALL

    SELECT DISTINCT
    install_time,
    customer_user_id,
    AppsFlyer_ID,
    media_source,
    partner,
    campaign,
    'Install' Retargeting_Conversion_Type
    FROM `appsflyer_raw.organic_in_app_events_report`
    WHERE 1=1

    UNION ALL

    SELECT DISTINCT
    install_time,
    customer_user_id,
    AppsFlyer_ID,
    media_source,
    partner,
    campaign,
    Retargeting_Conversion_Type
    FROM `appsflyer_raw.in_app_events_retarget`
    WHERE 1=1
  ) a
  LEFT JOIN `prj-prod-dataplatform.worktable_datachampions.installs_attribution_mapping` b
  ON 
    COALESCE(a.media_source,a.partner,a.campaign) = COALESCE(b.media_source,b.partner,b.campaign)

  WHERE 1=1
 
  QUALIFY ROW_NUMBER() OVER (PARTITION BY a.customer_user_id ORDER BY install_time DESC) = 1
  ORDER BY customer_user_id
 ),
final_output as (
SELECT DISTINCT
input_customers.customerId,
TIMESTAMP_DIFF(MIN(cust_mtb.created_dt),MIN(install_time),MINUTE) appsflyer_install_to_registration_minutes,
FROM input_customers
JOIN `dl_customers_db_raw.tdbk_customer_mtb` cust_mtb ON cust_mtb.cust_id = input_customers.customerId
LEFT JOIN af_link b ON b.customer_user_id = input_customers.customerId and input_customers.snapshot_date = b.snapshot_date
group by 1
)
SELECT 
final_output.*,
CASE WHEN campaign_data.source_group = 'Organic' THEN 'Organic'
ELSE 'InOrganic' END AS channel_source_group,
source as marketing_source_name,
meng_no_of_logins,
meng_ql_calculator_count,
coalesce(meng_ql_calculator_tot_visit_cnt,0)meng_ql_calculator_tot_visit_cnt,
from final_output
LEFT JOIN campaign_data ON campaign_data.customer_user_id = final_output.customerId
LEFT JOIN events c ON c.customer_id = final_output.customerId
;
"""

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema2}.{trench2everappliedsnapshotevents} created successfully')

Table worktable_data_analysis.trench2_ever_applied_snapshot_events20230101_20250818 created successfully


# trench2_ever_applied_snapshot_callcount

# trench2_ever_applied_snapshot_callcount Table

In [18]:
trench2everappliedsnapshotcallcount = f'trench2_ever_applied_snapshot_callcount{datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')}_{datetime.strptime(EDATE, '%Y-%m-%d').strftime('%Y%m%d')}'

print(trench2everappliedsnapshotcallcount)


trench2_ever_applied_snapshot_callcount20230101_20250818


# # trench2_ever_applied_snapshot_events query

In [19]:
sq = f""" 
create or replace table {schema2}.{trench2everappliedsnapshotcallcount} as
 with input_customers as ( 
SELECT * FROM {schema2}.{gammatrench2everappliedcustomersnapshot}),
cust_emails as (
  with temp_output as (
select too.email as to_email, emailTranscript.from.email as from_email,
 
CASE WHEN SPLIT(emailTranscript.from.email,'@')[SAFE_OFFSET(1)] like '%tonik%' THEN 'outbound'
WHEN SPLIT(too.email,'@')[SAFE_OFFSET(1)] like '%tonik%' THEN 'inbound'
ELSE 'outbound'
END AS category,
creationTime
from `genesys_raw.emails`,
unnest(emailTranscript) emailTranscript,unnest(emailTranscript.to) too
)
SELECT
*,
CASE WHEN category = 'outbound' THEN to_email
WHEN category = 'inbound'THEN from_email
WHEN from_email not like '%tonik%' THEN from_email
END AS customer_email_address,
FROM temp_output
),
 
cust_mobile_genesys_data as (
SELECT
input_customers.customerid,
MAX(
    CASE
      WHEN DATE(call_history.callDatetime) between DATE_SUB(input_customers.snapshot_date, INTERVAL 90 DAY) and DATE(input_customers.snapshot_date) THEN 1
      ELSE 0
  END
    ) AS flag_contactable_last90D,
  count(
    CASE
      WHEN DATE(call_history.callDatetime) between DATE_SUB(input_customers.snapshot_date, INTERVAL 90 DAY) and DATE(input_customers.snapshot_date) THEN 1
      ELSE null
  END
    ) AS count_contactable_last90D,
COUNT(CASE WHEN calldirection = 'outbound' THEN 1 ELSE null END) AS outbound_call_count,
COUNT(CASE WHEN calldirection = 'inbound' THEN 1 ELSE null END) AS inbound_call_count,

FROM input_customers
LEFT JOIN (SELECT DISTINCT cust_id,mobile_no, valid_from_dt FROM `datalake_worktables.customer_mobile_mail_dtls`
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_id,mobile_no order by valid_from_dt asc) = 1
) mobile_details on input_customers.customerid = mobile_details.cust_id and DATE(mobile_details.valid_from_dt) < DATE(snapshot_date)
LEFT JOIN `risk_credit_mis.call_attempt_history_gensys` call_history ON RIGHT(mobile_details.mobile_no,10) = right(call_history.mobileNumber,10) and DATE(call_history.callDatetime) < DATE(input_customers.snapshot_date) and connected = 1 and mediaType = 'voice'
GROUP BY 1
),
cust_email_genesys_data as (
SELECT
input_customers.customerid,
COUNT(CASE WHEN category = 'outbound' THEN 1 ELSE null END) AS outbound_email_count,
COUNT(CASE WHEN category = 'inbound'  THEN 1 ELSE null END) AS inbound_email_count
FROM input_customers
LEFT JOIN (SELECT DISTINCT cust_id,email, valid_from_dt FROM `datalake_worktables.customer_mobile_mail_dtls`
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_id,email order by valid_from_dt asc) = 1
) email_details on input_customers.customerid = email_details.cust_id and DATE(email_details.valid_from_dt) < DATE(snapshot_date)
left JOIN cust_emails ON email_details.email = cust_emails.customer_email_address and DATE(cust_emails.creationTime) < DATE(input_customers.snapshot_date)
GROUP BY 1
)
SELECT
  input_customers.customerid,
  input_customers.snapshot_date,
 flag_contactable_last90D,
 count_contactable_last90D,
inbound_call_count,
outbound_call_count,
outbound_email_count,
inbound_email_count,
FROM input_customers
LEFT JOIN cust_mobile_genesys_data on cust_mobile_genesys_data.customerid = CAST(input_customers.customerid AS STRING) --and cust_mobile_genesys_data.snapshot_date = input_customers.snapshot_date
LEFT JOIN cust_email_genesys_data ON  cust_email_genesys_data.customerid = CAST(input_customers.customerid AS STRING) --and cust_email_genesys_data.snapshot_date = input_customers.snapshot_date
; 
"""

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema2}.{trench2everappliedsnapshotcallcount} created successfully')

Table worktable_data_analysis.trench2_ever_applied_snapshot_callcount20230101_20250818 created successfully


# gamma_model_ever_applied_snapshot

# trench2_ever_applied_snapshot_callcount Table

In [20]:

gammamodeleverappliedsnapshot = f'gamma_model_ever_applied_snapshot_{datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')}_{datetime.strptime(EDATE, '%Y-%m-%d').strftime('%Y%m%d')}'

print(gammamodeleverappliedsnapshot)


gamma_model_ever_applied_snapshot_20230101_20250818


# gamma_model_ever_applied_snapshot_ query

In [21]:
sq = f""" 
create or replace table {schema2}.{gammamodeleverappliedsnapshot} as
SELECT 
a.customer_id,
digitalloanaccountid,
dob_snapshot_date,
days_since_credo_call_onb,
days_since_credo_call_snapshot_date,
credolabRefNumber,
credo_inquiry_date,
ln_age,
ln_email,
ln_city,
ln_province,
ln_industry_new,
ln_employment_type_new,
a.snapshot_date,
ln_nature_of_work_new,
ln_source_funds_new,
onb_first_name,
onb_middle_name,
onb_last_name,
ln_brand,
ln_cnt_dependents,
ln_doc_type,
ln_osversion,
ln_self_dec_income,
ln_marital_status,
ln_education_level,
ln_ever_applied_flag,
c_credo_score,
tx_first_product,
tx_first_product_user_segment,
null as ln_user_type,
ln_loan_type,
ln_prod_type ln_product_type,
CASE WHEN digitalloanaccountid IS NULL THEN 'Never_Applied_Before'
ELSE 'Rejected_Before' END as existing_user_type,
onb_tsa_onboarding_datetime onboarding_date,
a.ln_loan_appln_time,
first_disb_loan_disbursed_time ln_loan_disb_time,
ln_mature_fspd30_flag,
--CASE WHEN LOANMATURITYDATE < secondDueDate THEN 0 ELSE ln_mature_fspd30_flag
--END AS  ln_mature_fspd30_flag,
ln_fspd30_flag,
ln_mature_fpd30_flag,
--CASE WHEN LOANMATURITYDATE < firstDueDate THEN 0 ELSE ln_mature_fpd30_flag
--END AS ln_mature_fpd30_flag,
ln_fpd30_flag,
first_applied_loan_appln_time,
first_applied_loan_decision,
first_applied_loan_type,
first_applied_product_type,
first_applied_loan_amount,
first_applied_loan_tenor,
last_applied_loan_appln_time,
last_applied_loan_decision,
last_applied_loan_type,
last_applied_product_type,
last_applied_loan_amount,
last_applied_loan_tenor,
last_applied_os_type,
last_applied_crif_id last_applied_digitalloanaccountId,
last_applied_crif_id,
last_applied_credo_ref_no,
last_applied_credo_score,
onb_year,
onb_month_of_year,
onb_week_of_month,
onb_day_of_month,
onb_time_of_day,
tx_cnt_cash_in_total,
tx_cnt_cash_in_ob2t,
tx_cnt_cash_in_ot2t,
tx_amt_cash_in_total,
tx_amt_cash_in_ob2t,
tx_amt_cash_in_ot2t,
tx_cnt_cash_out_total,
tx_cnt_cash_out_billpay,
tx_cnt_cash_out_cards,
tx_cnt_cash_out_t2ot,
tx_cnt_cash_out_t2ob,
tx_amt_cash_out_total,
tx_amt_cash_out_billpay,
tx_amt_cash_out_cards,
tx_amt_cash_out_t2ot,
tx_amt_cash_out_t2ob,
tx_deposit_accnt_cnt,
tx_stash_accnt_opened_cnt,
tx_stash_accnt_closed_cnt,
tx_stash_balance,
tx_td_accnt_opened_cnt,
tx_td_accnt_completed_cnt,
tx_td_accnt_broken_cnt,
tx_td_auto_roll_over_enabled,
tx_td_balance,
td_max_duration tx_td_max_duration,
td_min_duration tx_td_min_duration,
td_avg_duration tx_td_avg_duration,
stash_max_duration tx_stash_max_duration,
stash_avg_duration tx_stash_avg_duration,
stash_min_duration tx_stash_min_duration,
tx_med_days_bw_new_dep_acct_open,
tx_med_days_bw_td_acct_open,
tx_avg_days_bt_trans,
tx_avg_days_bt_cash_in_trans,
tx_avg_days_bt_cash_out_trans,
tx_med_days_bt_trans,
tx_med_days_bt_cash_in_trans,
tx_med_days_bt_cash_out_trans,
ln_cnt_applied_loan tx_cnt_applied_loan,
ln_cnt_rejected_loans tx_cnt_rejected_loans,
ln_cnt_approved_loans tx_cnt_approved_loans,
ln_cnt_disbursed_loans tx_cnt_disbursed_loans,
ln_cnt_completed_loans tx_cnt_completed_loans,
ln_cnt_active_loans tx_cnt_active_loans,
ln_cnt_incomplete_loan_apps tx_cnt_incomplete_loan_apps,
referral_flag onb_referral_flag,
COALESCE(flag_contactable_last90D,0) cs_contactable_last_90d_flag,
ln_any_prev_disb_loan_sil_mobile_flag,
CASE WHEN appsflyer_install_to_registration_minutes < 0 THEN NULL
ELSE appsflyer_install_to_registration_minutes END appsflyer_install_to_registration_minutes,
meng_no_of_logins,
meng_ql_calculator_count,
meng_ql_calculator_tot_visit_cnt,
channel_source_group,
marketing_source_name,
outbound_call_count cs_cnt_outbound_calls,
inbound_call_count cs_cnt_inbound_calls,
outbound_email_count cs_cnt_outbound_emails,
inbound_email_count cs_cnt_inbound_emails,
FROM 
 {schema2}.{trench2everappliedsnapshottransactions} a
 JOIN (SELECT customerid,ln_city,ln_province FROM {schema2}.{gammatrench2everappliedcustomersnapshot}) customers on customers.customerid = a.customer_id
LEFT JOIN {schema2}.{trench2everappliedsnapshotevents} b on a.customer_id = b.customerid 
LEFT JOIN {schema2}.{trench2everappliedsnapshotcallcount} cc on cc.customerid = a.customer_id

"""

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema2}.{gammamodeleverappliedsnapshot} created successfully')

Table worktable_data_analysis.gamma_model_ever_applied_snapshot_20230101_20250818 created successfully


# Never Applied

# Never Applied Table

In [22]:
schema1 = 'risk_mart'

OBDATE = '2025-08-18'
# change it to obdate from current date if the requirement is for a specific date
SDATE = '2021-07-01'
EDATE = '2025-08-18'
# change it to obdate from current date if the requirement is for a specific date

formatted_date = datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')

print(formatted_date)  # Output: 20230101

nal = f'tsa_onboarded_but_never_applied_loan_{datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')}_{datetime.strptime(EDATE, '%Y-%m-%d').strftime('%Y%m%d')}'

print(nal)


20210701
tsa_onboarded_but_never_applied_loan_20210701_20250818


# Never Applied Querie

In [23]:
sq = f"""
create or replace table {schema1}.{nal} as
select * from( 
with 
osversion_credolab as (select * from (
(select distinct deviceId,'android'||generalInfo.release as credolab_osversion ,GeneralInfo.model onb_model from credolab_raw.android_credolab_datasets_struct_columns
union all 
select distinct deviceId,'ios'||generalInfo.release as credolab_osversion ,GeneralInfo.model onb_model from  credolab_raw.ios_credolab_datasets_struct_columns))),

onb_doc_type_cust as (
SELECT user_id,max(doc_type) onb_type_cust_doc FROM `prj-prod-dataplatform.dl_customers_db_raw.tdbk_cust_doc_mtb` 
group by 1)

select distinct customerId,
main.digitalLoanAccountId,
loanAccountNumber,
onboarding_datetime onb_tsa_onboarding_datetime,
first_name onb_first_name,
middle_name onb_middle_name,
last_name onb_last_name,
onb_age,
gender onb_gender,
onb_mobile_no,
email onb_email,
email_verified_flag onb_email_verified_flag,
place_of_birth onb_place_of_birth,
coalesce(onb_type_cust_doc,onb_doc_type) onb_doc_type,
onb_country,
onb_province,
onb_city,
onb_barangay,
onb_postalcode,
onb_latitude,
onb_longitude,
coalesce(osversion_credolab.credolab_osversion,osversion) onb_os_version,
osversion_credolab.onb_model ln_model,		
kyc_status onb_kyc_status,
																						 
--kyc_status_change_dt onb_kyc_status_upgrade_datetime,
datetime(timestamp(kyc_status_change_dt), 'Asia/Manila') onb_kyc_status_upgrade_datetime,
--user_type ln_user_type,
'4_Never_start_loan_appln' as ln_user_type,
loan_type ln_loan_type,
loan_product_type ln_prod_type,
flg_applied_loan ln_loan_applied_flag,
fatca ln_facta_flag,
flg_reject_rule_check ln_dl_rule_reject_flag,
ln_taran_rule_reject_flag,
flg_taran_reject_flag ln_taran_scorecard_reject_flag,
flg_cdd_reject_flag ln_cdd_reject_flag,
ln_marked_underwriter_check_flag,
flg_hard_reject_flag ln_underwriting_reject_flag, 
if(flg_disbursed_loan=1,1,flg_loan_approved_not_disbursed) ln_final_approved_flag,
flg_disbursed_loan ln_disb_flag,
flg_loan_approved_not_disbursed ln_approved_not_disb_flag,
loan_submission_date ln_appln_submit_datetime,
disbursementDateTime ln_disb_dtime,
loan_chosen_principal ln_chosen_principal,
loan_chosen_tenor ln_chosen_tenor,
loan_approved_principal ln_approved_principal,
loan_approved_tenor ln_approved_tenor,
main.onb_cnt_ongoing_loans ln_cnt_ongoing_tdb_loans,
onb_tot_ongoing_loans_emi ln_tot_ongoing_tdb_loans_emi,
loan_purpose ln_purpose,
vas_opted_flag ln_vas_opted_flag,
vas_used_flag ln_vas_used_flag,
loan_age ln_age,
loan_mobile_no ln_mobile_no,
loan_alternate_mobile_no ln_alt_mobile_no,
osversion_v2 ln_osversion,
brand ln_brand,
ln_os_type,
		 
coalesce(loan_address,addressline) ln_address,
coalesce(loan_province,province) ln_province,
coalesce(loan_city,city) ln_city,
coalesce(loan_barangay,barangay) ln_barangay,
coalesce(cast(loan_postal_code as string),postalcode) ln_postal_code,
loan_latitude ln_latitude,
loan_longitude ln_longitude,
loan_docType ln_doc_type,
loan_docNumber ln_doc_number,
if(loan_marital_status='Live-in Partner','With a Live-in Partner',loan_marital_status) ln_marital_status,
--case when loan_dependents_count in ('No dependents') then null else loan_dependents_count end ln_cnt_dependents,
 loan_dependents_count ln_cnt_dependents,
loan_education_level ln_education_level,
coalesce(main.source_funds) ln_source_funds,
--coalesce(source_funds_new.source_funds_new,main.source_funds) ln_source_funds_new,
case when coalesce(purpose_source_of_fund_name,main.source_funds) in ('Others') then null
when coalesce(purpose_source_of_fund_name,main.source_funds) in ('Commissions') then null else coalesce(purpose_source_of_fund_name,main.source_funds) end as ln_source_funds_new,
coalesce(purpose_employment_status_name,main.employment_type) ln_employment_type,
--coalesce(employment_type_new.employment_type_new,main.employment_type) ln_employment_type_new,
case when coalesce(purpose_employment_status_name,main.employment_type)='Employed' and Mod(cast(customerId as int64),10)<3 then 'Employed - Govt. Employee/Govt. Official' 
when coalesce(purpose_employment_status_name,main.employment_type)='Employed' and Mod(cast(customerId as int64),10)>2 then'Employed - Private Employee' 
else coalesce(purpose_employment_status_name,main.employment_type) end as ln_employment_type_new,
--main.employment_type_new ln_employment_type_new,
coalesce(purpose_nature_of_work_name,main.nature_of_work) ln_nature_of_work,
--coalesce(nature_of_work_new.nature_of_work_new,main.nature_of_work) ln_nature_of_work_new,
case when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Admin/Clerical Staff/Officer') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Architect/Interior Decorator') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Bank Employee') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Broker/Dealer') then  'Broker/Dealer'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Call Center Agent/Tele Marketer') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Cashier/Food Server/Waiter') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Consultant') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Doctor/Dentist/Medical Professional') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Engineer') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Entertainer/Artist') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Foreign Ambassador/Diplomat') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Gov Employee') then  'Govt Employee'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Govt Employee') then  'Govt Employee'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Insurance Agent/Financial Advisor') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('IT Professional') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Lawyer') then  'Licensed Professional - Lawyer'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Military/Armed Forces/Police/Navy/Airforce') then  'Government - Uniformed Service'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Newscaster/Media Personality') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Pilot/Airline Crew') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Seafarer') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Teacher/Instructor/Coach') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Utility Personnel/Household Help') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Accountant/Auditor') then  'Licensed Professional - Accountant'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Utilities (Electricity Gas and Water)') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Wholesale and Retail Trade') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Sales/Marketing Personnel') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Other Professional services') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Other Non Professional Services') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Owner') then  'Owner'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Religious Leader') then  'Religious Leader'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Fire Fighter') then  'Government - Uniformed Service'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Farmer/Fisherman') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Debt Collection/Recovery Agents/ Collection Agency Employee') then  'Debt Collection/Recovery Agents/ Collection Agency Employee'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Court Personnel/Bailiff') then  'Court Personnel/Bailiff'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Store/Service Manager') then  'Manager/Executive'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Sales Marketing Professional') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Barber / Beauty Salon') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Business Owner (MSME)') then  'Owner'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Business Owner') then  'Owner'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Delivery Rider/Driver') then  'Delivery Rider/Driver'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Delivery Rider / Driver') then  'Delivery Rider / Driver'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Manpower Agency') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Online Freelancing / Virtual Assistant / Online teacher') then  'Virtual/Online/Remote Employee'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Online Seller') then  'Virtual/Online/Remote Employee'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Real Estate agent') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Skilled Worker') then  'Skilled Worker'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Technician (Cellphone / Laptop)') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Vendor (Sari-Sari store Wet market etc)') then  null
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Staff/Rank and File') then  'Staff/Rank and File'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Officer/Supervisor') then  'Officer/Supervisor'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Manager/Executive') then  'Manager/Executive'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Licensed Professional - Lawyer') then  'Licensed Professional - Lawyer'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Licensed Professional - Accountant') then  'Licensed Professional - Accountant'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Licensed Professional - Others') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Religious Minister') then  'Religious Minister'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Government - Elected Official') then  'Government - Elected Official'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Government - Uniformed Service') then  'Government - Uniformed Service'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Director/Trustee/Stockholder') then  'Director/Trustee/Stockholder'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Virtual/Online/Remote Employee') then  'Virtual/Online/Remote Employee'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Government - Appointed Official') then  'Government - Appointed Official'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('AIR FORCE') then  'Military/Armed Forces/Police/Navy/Airforce'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('NAVY') then  'Military/Armed Forces/Police/Navy/Airforce'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Admin/Secretarial') then  'Admin/Clerical Staff/Officer'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Sales/Marketing Professional') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('COURT-PERSONNEL/BAILIFFS') then  'Court Personnel/Bailiff'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('DEBT COLLECTION/RECOVERY AGENTS/COLLECTION AGENCY EMPLOYEE') then  'Debt Collection/Recovery Agents/ Collection Agency Employee'
when coalesce(purpose_nature_of_work_name,main.nature_of_work) in ('Accountant') then  'Licensed Professional - Accountant' else coalesce(purpose_nature_of_work_name,main.nature_of_work) end as ln_nature_of_work_new,
if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) ln_industry,
case when main.source_funds in ('Remittance','Pension','Inheritance/Retirement Funds','Interest on Deposits and Investments','Allowance') then null 
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Wholesale and Retail Trade') then 'Wholesale and Retail Trade'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Utilities (Electricity, Gas and Water)') then 'Utilities (Electricity, Gas and Water)'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Transportation') then 'Transportation'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Tourism, Hotel and Resorts') then 'Tourism, Hotel and Resorts'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('TeleCo') then 'Telecommunication'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Telecommunication') then 'Telecommunication'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Religious Organization') then 'Religious Organization'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Entertainment and Media') then 'Recreation (Arts, Entertainment/Media, Sports)'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Real Estate') then 'Real Estate'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Pawnshop/Remittance Agent/Virtual Currency') then 'Pawnshop/Remittance Agent/Virtual Currency'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Others') then 'Others'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Non Government Organization/Foundation') then 'NGOs/NonProfit Organization/Foundation'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Military and Weaponry Business') then 'Military and Weaponry Business'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Maritime/Shipping') then 'Maritime/Shipping'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Manufacturing - Food') then 'Manufacturing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Manufacturing - Non Food') then 'Manufacturing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Manpower/Manning Agencies') then 'Manpower/Manning Agencies'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Legal Service/Law Firm') then 'Legal Service/Law Firm'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Insurance Activities') then 'Insurance Activities'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Hospital and Medical Services') then 'Hospital and Medical Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Govt. Institution') then 'Govt. Institution'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Gold, Jewelry and Precious Metals') then 'Gold/Jewelry/Precious Metals/Art/Antique Dealership'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Online Gaming/Lottery/Casino') then 'Gambling and Betting (Online Gaming, Lottery, Casino)'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Freight and Cargo') then 'Freight and Cargo'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Food Industry/Restaurants/Fastfoods') then 'Food Industry/Restaurants/Fastfoods'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Fin Tech/Financial Services') then 'Fin Tech/Financial Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('FX Dealer/Money Changer') then 'FX Dealer/Money Changer'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Education/Schools') then 'Education/Private Schools'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ("Convenience Store/Internet Cafe's") then "Convenience Store/Internet Cafe's"
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Construction') then 'Construction'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Computer/IT Related Services') then 'Computer/IT Related Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Business Process Outsourcing/Tele Marketing') then 'Business Process Outsourcing/Tele Marketing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Beauty and Health Services') then 'Beauty and Health Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Health and Beauty Services') then 'Beauty and Health Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Banking') then 'Banking'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Architecture/Engineering') then 'Architecture/Engineering'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Airline/Aviation') then 'Airline/Aviation'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Agribusiness/Animal Farming/Fishing') then 'Agribusiness/Animal Farming/Fishing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Advertising/Sales/Marketing') then 'Advertising/Sales/Marketing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Accounting/Bookkeeping') then 'Accounting/Bookkeeping/Auditing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) is null then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Leasing and Rental Services') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Repair, Sanitation and Housekeeping') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Repair, Sanitation and Housekeeping') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Embassy') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Recycling') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('Publishing and Printing') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) in ('IT services','ONLINE SELLER','business owner','Human Resources','Head of sales','Human Resource Management',
'Digital Marketing','Food services','Manufacture of Leather and Related Products','Manufacture of Other Non-Metallic Mineral Products',
'Businesswoman','Business Owner','Agency Manpower','Advertising and marketing','Manufacture of Coke and Refined Petroleum Products',
'Sales Coffee Trading','Pharmaceutical','Manufacture of Furniture','Software as a Service','Nurse','Manufacture of Rubber Products','Shared Services',
'Waste and recycling solutions','Event Supplier','Fishing and Aquaculture','Manufacture of Motor Vehicles, Trailers and Semi-Trailers',
'Crop and Animal Production, Hunting and Related Service Activities; Forestry and Logging','Publishing Activities',
'Manufacture of Basic Metals','Manufacture of Textiles','Manufacture of Electrical Equipment',
'Water Collection, Treatment and Supply','Water Transport','Manufacture of Basic Pharmaceutical Products and Pharmaceutical Preparation',
'Manufacture of Chemicals and Chemical Products','Insurance, Reinsurance and Pension Funding except Compulsory Social Security',
'Electricity, Gas, Steam and Air Conditioning Supply','Manufacture of Machinery and Equipment, n.e.c.',
'Travel Agency, Tour Operator, Reservation Service and Related Activities','Medical Transcription  Activities',
'Manufacture of Fabricated Metal Products except Machinery and Equipment','Manufacture of Plastic Products',
'Postal and Courier Activities','Real Estate Activities','Printing and Reproduction of Recorded Media',
'Manufacture of Computer, Electronic and Optical Products','Transport via Buses',
'Religious Organization ','Designated NonFinancial Businesses ','Manufacture of Wearing Apparel',
'Warehousing and Support Activities for Transportation','Architectural and Engineering Activities and Related Technical Consultancy',
'Wholesale  and Retail Trade and Repair of Motor Vehicles and Motorcycles','Designated NonFinancial Businesses',
'Human Health Activities except Public Health Activities','Manufacture of Food Products',
'Military and Weaponry Business','Retail Trade except of Motor Vehicles and Motorcycles',
'Land Transport and Transport via Pipelines except  Transport via Buses','Education except Public Education',
'Financial Service Activities except Insurance, Pension Funding and Central Banking','Wholesale Trade except of Motor Vehicles and Motorcycles',
'Accounting, Bookkeeping and Auditing Activities; Tax Consultancy','Accommodation and Food Service Activities',
'NGOs/NonProfit Organization/Foundation ','Mining and Quarrying ','Computer Programming, Consultancy and Related Activities; Information Service Activities',
'Call Center Activities (Voice)','Back-Office Operations Activities (Non-Voice) Except Medical Transcription Activities',
'Nightclubs/Discos/Karaoke Center','Import Export Companies ','Mining and Quarrying','Import Export Companies',
'Gambling and Betting (Online Gaming, Lottery, Casino) ','Other Personal Services Activities','FX Dealer/Money Changer',
'Religious Organization','Securities/Holding Companies ','Car/Boat/Plane/Used Truck/AutoParts Manufacturing/Dealership or Auction Houses') then null																																											 
else if(purpose_created_dt>='2025-05-20',purpose_industryName,main.industry_description) end as ln_industry_new, 
																						 
--coalesce(industry_description_new.industry_description_new,main.industry_description) ln_industry_new,
loan_companyName ln_company_name,
loan_self_dec_income ln_self_dec_income,
loan_salary_scaled_income ln_salary_scaled_income,
loan_ref_type1 ln_ref1_type,
loan_ref_type2 ln_ref2_type,
flg_cic_called cic_called_flag,
flg_cic_hit cic_hit_flag,
ScoreRange cic_score_range,						   
demo_score old_demo_score,
cic_score old_cic_score,
credo_score old_gen_credo_score,
trusting_social_score ts_trusting_social_score,
bureau_score bu_bureau_score,
c_credo_score,s_credo_score,fu_credo_score,r_credo_score,gen_credo_score,
flg_mature_fpd10 ln_mature_fpd10_flag,
flg_mature_fpd30 ln_mature_fpd30_flag,
flg_mature_fspd_30 ln_mature_fspd30_flag,
flg_mature_fstpd_30 ln_mature_fstpd30_flag,
deffpd10 ln_fpd10_flag,
deffpd30 ln_fpd30_flag,
deffspd30 ln_fspd30_flag,
deffstpd30 ln_fstpd30_flag,
--loan_defFPD30 ln_defFPD30,
loan_defSPD30 ln_spd30_flag,loan_defTPD30 ln_tpd30_flag,
defpd10_outstanding_principal ln_fpd10_os_principal,defpd30_outstanding_principal ln_fpd30_os_principal,
defspd30_outstanding_principal ln_fspd30_os_principal,deftpd30_outstanding_principal ln_fstpd30_os_principal,
refno credolabRefNumber,
datetime(credo_creation_date,'Asia/Manila') credo_inquiry_date,
status cust_status_flag,
					
onb_company_name,
cust_status_close_date

from 
(with educationtype AS (
        SELECT
        loan.digitalLoanAccountId,loanAccountNumber, loan.loanPurposeId,   education_id,
        description AS education_level
        FROM  dl_loans_db_raw.tdbk_digital_loan_application loan
        LEFT JOIN  dl_loans_db_raw.tdbk_loan_purpose purpose
        ON   loan.loanPurposeId = purpose.loanPurposeId
        LEFT JOIN dl_loans_db_raw.tdbk_loan_lov_mtb
        ON education_id = id AND module = 'Education'
         ),

ref_type as (with Ref_type1 as (select  digitalLoanAccountId,relationship_id , description AS loan_ref_type1 from dl_loans_db_raw.tdbk_loan_refernce_details A
        LEFT JOIN dl_loans_db_raw.tdbk_loan_lov_mtb B  ON A.relationship_id = B.id
 where refPrefrenceOrder='1' qualify row_number() over (partition by digitalLoanAccountId order by refCreatedDateAndTime desc) =1),
    Ref_type2 as (select  digitalLoanAccountId,relationship_id , description AS loan_ref_type2 from dl_loans_db_raw.tdbk_loan_refernce_details A
        LEFT JOIN dl_loans_db_raw.tdbk_loan_lov_mtb B  ON A.relationship_id = B.id
 where refPrefrenceOrder='2' qualify row_number() over (partition by digitalLoanAccountId order by refCreatedDateAndTime desc) =1)
select Ref_type1.digitalLoanAccountId,loan_ref_type1,loan_ref_type2 from Ref_type1 join Ref_type2 on Ref_type1.digitalLoanAccountId=Ref_type2.digitalLoanAccountId) ,

user_type_min as (select customerId,min(case 
when digitalLoanAccountId is null then '4_Never_start_loan_appln'
when application_submit_date is null and rn1_application_status <=1 then '4_Never_start_loan_appln'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status is null then '3_Applied_Not_Disbursed'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status =1 then '2_New Applicant'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status >1 then '1_Repeat Applicant'

else '5_others' end ) user_type from
(select A.*,B.rn_disburement_status from
(select digitalLoanAccountId,customerId,applicationStatus, coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime)) application_submit_date,disbursementDateTime,
row_number() over (partition by customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))) rn1_application_status,
from risk_credit_mis.loan_master_table) A

left join 

(select digitalLoanAccountId,customerId,applicationStatus, disbursementDateTime,
case when disbursementDateTime is not null then 
row_number() over (partition by customerId order by disbursementDateTime) else 0 end rn_disburement_status,
from risk_credit_mis.loan_master_table where  disbursementDateTime is not null) B on A.digitalLoanAccountId=B.digitalLoanAccountId) 
--order by A.customerId 
group by 1 ),	
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30  in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),

cust_info as (
with cust_base as(
select * from `dl_customers_db_raw.tdbk_customer_mtb`
where cust_id is not null
--and status in ('C','A')
--and date(created_dt,'Asia/Manila') >= '{SDATE}'
),
first_loan_application as (
select custId,digitalLoanAccountId,loanaccountnumber,isTermsAndConditionstimestamp,poi1RejectType,loantype from
--(select * from 
`dl_loans_db_raw.tdbk_digital_loan_application`
--where custId in (select cast(cust_id as integer) from cust_base) qualify row_number() over (partition by custId order by created_dt) = 1)
--where --loanType in ('TUL') and coalesce(flowType,'1') <> '2'
),
loan_calculator as (
select custid,digitalLoanAccountId from
(SELECT fla.custid,fla.digitalLoanAccountId,
sum(case when a.applicationStatus='INITIATE' then 1 else 0 end) loan_calculator
 FROM `dl_loans_db_raw.tdbk_status_trace`  a
 join first_loan_application fla
 on fla.digitalLoanAccountId=a.digitalLoanAccountId
 group by 1,2)
where loan_calculator>0
),
disbursed_undisbursed_loan as (
SELECT fla.custid,fla.digitalLoanAccountId,
 FROM `dl_loans_db_raw.tdbk_status_trace`  a
 join first_loan_application fla
 on fla.digitalLoanAccountId=a.digitalLoanAccountId
 where a.applicationStatus ='APPROVED'
 group by 1,2
)
select 
 cb.*, fla.digitalLoanAccountId,--onboarding_datetime, 
 fla.loantype,
case when lc.custid is not null then 1 else 0 end as flg_loan_calculator,
case when fla.poi1RejectType is not null or fla.loantype is null then 1 else 0 end as flg_reject_rule_check,
case when (fla.isTermsAndConditionstimestamp is not null or loan.new_loan_type ='Flex-up')  then 1 else 0 end as flg_applied_loan,
case when ul.custid is not null then 1 else 0 end as flg_approved_scorecard_loan,
case when fla.loanaccountnumber is not null then 1 else 0 end as flg_disbursed_loan,
fla.loanaccountnumber as loan_id
from cust_base cb
left join first_loan_application fla
on cast(cb.cust_id as integer)=fla.custId
left join loan_calculator lc
--on cast(cb.cust_id as integer)=lc.custid and 
on lc.digitalLoanAccountId=fla.digitalLoanAccountId
left join disbursed_undisbursed_loan ul
--on cast(cb.cust_id as integer)=ul.custid)
on ul.digitalLoanAccountId=fla.digitalLoanAccountId
left join 
risk_credit_mis.loan_master_table loan on loan.digitalLoanAccountId=fla.digitalLoanAccountId),


default_outstanding_principal as (with all_default_account as (select digitalLoanAccountId,loanAccountNumber,firstDueDate,secondDueDate,thirdDueDate,defFPD30,defSPD30,defTPD30 from 
`risk_credit_mis.loan_master_table`
where defFPD10=1 or defFPD30=1 or defSPD30=1 or defTPD30=1),

defpd10_pricipal_outstanding as (
select loanAccountNumber,firstDueDate,PRINCIPALARREARS defpd10_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(firstDueDate ,INTERVAL 10 DAY)=core._Partitiondate or Date_add(firstDueDate ,INTERVAL 11 DAY)=core._Partitiondate)
where defFPD10=1 qualify row_number() over (partition by loanAccountNumber order by defpd10_outstanding_principal desc )=1),

defpd30_pricipal_outstanding as (
select loanAccountNumber,firstDueDate,PRINCIPALARREARS defpd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(firstDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(firstDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defFPD30=1 qualify row_number() over (partition by loanAccountNumber order by defpd30_outstanding_principal desc )=1),

defspd30_pricipal_outstanding as (
select loanAccountNumber,secondDueDate,PRINCIPALARREARS defspd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(secondDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(secondDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defSPD30=1 qualify row_number() over (partition by loanAccountNumber order by defspd30_outstanding_principal desc )=1),

deftpd30_pricipal_outstanding as (
select loanAccountNumber,thirdDueDate,PRINCIPALARREARS deftpd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(thirdDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(thirdDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defTPD30=1 qualify row_number() over (partition by loanAccountNumber order by deftpd30_outstanding_principal desc )=1)

select digitalLoanAccountId,allaccount.loanAccountNumber,defpd10_outstanding_principal,defpd30_outstanding_principal,defspd30_outstanding_principal,deftpd30_outstanding_principal,
allaccount.firstDueDate,allaccount.secondDueDate,allaccount.thirdDueDate,defFPD30 loan_defFPD30,defSPD30 loan_defSPD30,defTPD30 loan_defTPD30  from all_default_account allaccount
left join defpd10_pricipal_outstanding day10dpd on allaccount.loanAccountNumber=day10dpd.loanAccountNumber
left join defpd30_pricipal_outstanding firstdpd on allaccount.loanAccountNumber=firstdpd.loanAccountNumber
left join defspd30_pricipal_outstanding seconddpd on allaccount.loanAccountNumber=seconddpd.loanAccountNumber
left join deftpd30_pricipal_outstanding thirddpd on allaccount.loanAccountNumber=thirddpd.loanAccountNumber),



ln_model_credolab as (select * from (
(select distinct deviceId, GeneralInfo.model ln_model  from credolab_raw.android_credolab_datasets_struct_columns
union all 
select distinct deviceId,GeneralInfo.model ln_model from  credolab_raw.ios_credolab_datasets_struct_columns))),

onb_self_dec_income as 
(select custmtb.cust_id,monthly_income,company_name from 
`prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_mtb`  custmtb
join `prj-prod-dataplatform.dl_customers_db_raw.tdbk_cust_profile_mtb` profile on custmtb.user_id=profile.user_id
where monthly_income is not null),

cust_closed_date as (select OFCUSTOMERID,OFDATEOPENED, IF(OFDATECLOSED='1970-01-01' , NULL, OFDATECLOSED) OFDATECLOSED from `prj-prod-dataplatform.core_raw.customer_accounts`
join `dl_customers_db_raw.tdbk_customer_mtb` mtb on mtb.cust_id=OFCUSTOMERID
 where  CRINTERCODE = 'POC20000'
qualify row_number() over (partition by OFCUSTOMERID order by OFDATECLOSED desc) =1 and mtb.status='C')

select cust_mtb.cust_id customerId,
cust_mtb.email as email,
cust_mtb.email_status as email_verified_flag,
cust_mtb.birthplace as place_of_birth,
cust_profile.first_nm first_name,
cust_profile.middle_nm middle_name,
cust_profile.last_nm last_name,
hm_city as onb_city,
hm_province as onb_province,
hm_postalcode as onb_postalcode,
hm_barangay as onb_barangay,
hm_country as onb_country,
SPLIT(device.geolocation, ' | ')[SAFE_OFFSET(0)] AS onb_latitude,
SPLIT(device.geolocation, ' | ')[SAFE_OFFSET(1)] AS onb_longitude,
device.osversion,
cust_profile.doc_type onb_doc_type,
onb_type_cust_doc,

-- latitude onb_latitude,
-- longitude onb_longitude,
DATE_DIFF(DATE(datetime(cust_mtb.created_dt,'Asia/Manila')), parse_date('%d %b %Y', cust_mtb.dob), YEAR) - IF(EXTRACT(MONTH FROM parse_date('%d %b %Y', cust_mtb.dob))*100 + EXTRACT(DAY FROM parse_date('%d %b %Y', cust_mtb.dob)) > EXTRACT(MONTH FROM DATE(datetime(cust_mtb.created_dt,'Asia/Manila')))*100 + EXTRACT(DAY FROM DATE(datetime(cust_mtb.created_dt,'Asia/Manila'))),1,0) as onb_age,
DATE_DIFF(DATE(if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)), parse_date('%d %b %Y', cust_mtb.dob), YEAR) - IF(EXTRACT(MONTH FROM parse_date('%d %b %Y', cust_mtb.dob))*100 + EXTRACT(DAY FROM parse_date('%d %b %Y', cust_mtb.dob)) > EXTRACT(MONTH FROM DATE(if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)))*100 + EXTRACT(DAY FROM DATE(if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime))),1,0) as loan_age,
cust_mtb.gender,
if (user_type_min.user_type='1_Repeat Applicant' or (new_loan_type in ('Flex-up') and decision_date is not null),1,0) onb_cnt_ongoing_loans,
loanmaster.digitalLoanAccountId,loanmaster.loanAccountNumber,datetime(cust_mtb.created_dt,'Asia/Manila') onboarding_datetime,
cust_mtb.mobile_no onb_mobile_no, mobileNo loan_mobile_no,case when length(digitalloan.alternateNumber)=10 then concat('63',digitalloan.alternateNumber) else digitalloan.alternateNumber end loan_alternate_mobile_no,loanmaster.purposeDescription loan_purpose,
source_fund.description AS source_funds,
emp_status.description AS employment_type,
nat.description AS nature_of_work,
industry.value industry_description,
cust_profile.company_name loan_companyName,
loanmaster.maritalStatus loan_marital_status,
loanmaster.dependentsCount loan_dependents_count,
educationtype.education_level loan_education_level,
if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime) loan_submission_date,
cust_mtb.kyc_status,
cust_mtb.kyc_status_change_dt,
loan_ref_type1,loan_ref_type2,
loanmaster.disbursementDateTime,
loan_level.addressline,
loan_level.province,
loan_level.city,
loan_level.barangay,
loan_level.postalcode,
loanaddress.full_address loan_address,
loanaddress.province loan_province,
loanaddress.city loan_city,
loanaddress.barangay loan_barangay,
loanaddress.postal_code loan_postal_code,			   
		 
SPLIT(loan_level.geolocation, ' | ')[SAFE_OFFSET(0)] AS loan_latitude,
SPLIT(loan_level.geolocation, ' | ')[SAFE_OFFSET(1)] AS loan_longitude,
loan_level.docType loan_docType,
loan_level.docNumber loan_docNumber,
if(user_type_min.user_type is null and loanmaster.digitalLoanAccountId is null,'4_Never_start_loan_appln',user_type_min.user_type) user_type,
													   
case when loanmaster.new_loan_type ='Flex-up' and reloan_flag=1 then 'Reloan' else new_loan_type end loan_type,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
loanmaster.osversion_v2 osversion_v2,
loan_level.brand,
if(loan_level.brand in ('iPadOS','iOS','Apple') ,'iOS',if(loan_level.brand is not null,'Android','')) ln_os_type,
cust_profile.monthly_income loan_self_dec_income,
incomeperSalaryScale loan_salary_scaled_income,
case when vasflag.digitalLoanAccountId is not null then 1 else 0 end vas_used_flag,
if(poi3.crifApplicationId is not null,1,0) flg_cic_called,
flagmatched.matched_flag flg_cic_hit,
summary.ScoreRange,
--summary_matchedflag flg_cic_hit,			   
loanmaster.demographicScore demo_score,poi3.cicScorecard cic_score,loanmaster.credolabScore credo_score,trustingsocial.score trusting_social_score,poi3.bureauScore bureau_score,
--quick_score c_credo_score,sil_score s_credo_score,felxup_score fu_credo_score,reloan_score r_credo_score,generic_score gen_credo_score,
--quick_score c_credo_score,sil_score s_credo_score,flexup_score fu_credo_score,reloan_score r_credo_score,generic_score gen_credo_score,
score_quick_probability c_credo_score,score_sil_probability s_credo_score,score_flex_probability fu_credo_score,score_reloan_probability r_credo_score,score_all_probability gen_credo_score,
if (user_type_min.user_type='1_Repeat Applicant' or (new_loan_type in ('Flex-up') and decision_date is not null),loanaccts.monthlytermamount,0) as onb_tot_ongoing_loans_emi,
--empstatus.employment_type_new,
deliquency.flg_mature_fpd10,
deliquency.flg_mature_fpd30,
deliquency.flg_mature_fspd_30,
deliquency.flg_mature_fstpd_30,
deliquency.deffpd10,
deliquency.deffpd30,
deliquency.deffspd30,
deliquency.deffstpd30,
if(digitalloan.vas_flag='true',1,0) vas_opted_flag,
cust_profile.fatca,
flg_reject_rule_check,
flg_applied_loan,
--flg_approved_scorecard_loan,
flg_disbursed_loan,
CASE WHEN cddRejectReason IS NOT NULL and cddStatus <> 'ACTIVATED' THEN 1 ELSE 0 END flg_cdd_reject_flag,
CASE WHEN rejectedStage = 'Stage0' THEN 1 ELSE 0 END flg_hard_reject_flag, 
CASE WHEN rejectedStage = 'Stage1' THEN 1 ELSE 0 END flg_taran_reject_flag, 
case when digitalloan.poi3RejectType in ('SOFT','HARD') and digitalloan.poi3StatusCode like 'RJ%' then 1 else 0 end ln_taran_rule_reject_flag,
case when poi3.uw_flag in ('Streamline UW','Full UW') then 1 else 0 end ln_marked_underwriter_check_flag,

-- CASE WHEN cddRejectReason IS NOT NULL and cddStatus = 'ACTIVATED' THEN 1 ELSE 0 END flg_cdd_approved_flag,
-- CASE WHEN rejectedStage <> 'Stage0' THEN 1 ELSE 0 END flg_not_hard_uw_approved, 
-- CASE WHEN rejectedStage <> 'Stage1' THEN 1 ELSE 0 END flg_taran_approved_flag, 

CASE WHEN loanmaster.applicationstatus='APPROVED' THEN 1 ELSE 0 END flg_loan_approved_not_disbursed,
digitalloan.loanRequestAmount loan_chosen_principal,digitalloan.loanRequestTenure loan_chosen_tenor,
coalesce(disbursedLoanAmount,if(loanmaster.applicationstatus='APPROVED',loanmaster.approvedLoanAmount,NULL)) loan_approved_principal,
coalesce(disbursedLoanTenur,if(loanmaster.applicationstatus='APPROVED',loanmaster.approvedLoanTenure,NULL)) loan_approved_tenor,
loan_defFPD30,loan_defSPD30,loan_defTPD30,defpd10_outstanding_principal,defpd30_outstanding_principal,defspd30_outstanding_principal,deftpd30_outstanding_principal,loanmaster.credolabRefNumber,
credo_ref_score.refno,
credo_ref_score.createdOn credo_creation_date,
																			 
cust_mtb.status,
													   
onb_self_dec_income.company_name onb_company_name,
cust_closed_date.OFDATECLOSED cust_status_close_date,
purpose.source_of_fund_name purpose_source_of_fund_name,purpose.nature_of_work_name purpose_nature_of_work_name,purpose.employment_status_name purpose_employment_status_name,purpose.industryName purpose_industryName,purpose.industryid pupose_industryid,purpose.created_dt purpose_created_dt,
		  
--CASE WHEN loanPaidStatus in ('Normal','In Arrears') THEN 1 ELSE 0 END count_ongoing_loans 

from `dl_customers_db_raw.tdbk_customer_mtb` cust_mtb
left join (select userId,refno,deviceId,createdOn from  `dl_loans_db_raw.tdbk_credolab_track` where userId in (select user_id from (
select distinct user_id, cust_id,created_dt from `dl_customers_db_raw.tdbk_customer_mtb` 
))  qualify  row_number() over (partition by userId order by createdOn desc)=1) credo_ref_score
on credo_ref_score.userid=cust_mtb.user_id
--left join osversion_credolab on device_id=credo_ref_score.refno
--left join credo_score_dima  on credo_score_dima.refno=credo_ref_score.refno

left join  

(select distinct * from (select 'static' as source, loan.digitalLoanAccountId,reference_number referenceNumber,datetime(credo_inquiry_date) requestedDate,null score_all_score,generic_score score_all_probability,
null score_sil_score,sil_score score_sil_probability, null score_quick_score,quick_score score_quick_probability,null score_flex_score,flexup_score score_flex_probability,null score_reloan_score, reloan_score score_reloan_probability from worktable_data_analysis.dema_score_final_may12 dima
left join risk_credit_mis.loan_master_table loan on dima.reference_number=loan.credolabRefNumber

union all

select 'credotrace' as source, loan.digitalLoanAccountId,referenceNumber referenceNumber,datetime(requestedDate) requestedDate,score_all_score,cast (score_all_probability as float64) score_all_probability,
score_sil_score,cast (score_sil_probability as float64) score_sil_probability,score_quick_score,cast(score_quick_probability as float64) score_quick_probability,score_flex_score,cast(score_flex_probability as float64) score_flex_probability,score_reloan_score,cast(score_reloan_probability as float64) score_reloan_probability from risk_credit_mis.credolab_trace_insight_all_loans  dima
left join risk_credit_mis.loan_master_table loan on dima.referenceNumber=loan.credolabRefNumber)
qualify row_number() over(partition by referenceNumber order by requestedDate)  =1) credo_score_dima  on 
credo_score_dima.referenceNumber=credo_ref_score.refno

left join risk_credit_mis.loan_master_table loanmaster on cast(loanmaster.customerId as string)=cust_mtb.cust_id
left join `dl_loans_db_raw.tdbk_digital_loan_application` digitalloan on loanmaster.digitalLoanAccountId=digitalloan.digitalLoanAccountId
left join dl_loans_db_raw.tdbk_digital_loan_application digital on digital.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join dl_loans_db_raw.tdbk_loan_purpose purpose on purpose.loanPurposeId=digital.loanPurposeId
left join cust_info on cust_info.digitalLoanAccountId=loanmaster.digitalLoanAccountId
LEFT JOIN dl_dynamo_db_raw.tdbk_nature_of_work_mtb nat
ON cast(nat.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(3)]
LEFT JOIN dl_dynamo_db_raw.tdbk_employment_status_mtb emp_status
ON cast(emp_status.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(0)]
LEFT JOIN dl_dynamo_db_raw.tdbk_source_of_funds_mtb source_fund
ON cast(source_fund.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(4)]
LEFT JOIN dl_dynamo_db_raw.tdbk_industry_list_mtb industry
ON cast(industry.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(6)]
left join educationtype on educationtype.digitalLoanAccountId=digitalloan.digitalLoanAccountId
left join ref_type on ref_type.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join `risk_credit_mis.loan_level_master_table` loan_level on loan_level.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join (SELECT * FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_vas_holiday_htb`  where  req_type=1
qualify row_number() over (partition by digitalLoanAccountId order by repayment_number desc)=1) vasflag on loanmaster.digitalLoanAccountId=vasflag.digitalLoanAccountId
left join `dl_dynamo_db_raw.tdbk_device_regdtl_mtb` device on device.id=cust_mtb.device_id
-- LEFT JOIN (SELECT customer_id,    latitude,    longitude,  FROM    `risk_mart.customer_gps_location`
--   WHERE event_description = 'Onboarding' AND latitude <> 'undefined' AND longitude <> 'undefined' 
--   QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id order by gps_collection_date desc ) =1 ) loc on loc.customer_id=cust_mtb.cust_id
  LEFT JOIN (select digitalLoanAccountId,max(installmentAmount) monthlytermamount from `risk_credit_mis.loan_installments_table` instal
--join core_raw_staging.loan_accounts loanacct on instal.loanAccountNumber=loanacct.ACCOUNTNUMBER  where LOANSTATUS in ('Normal','In Arrears')  
group by 1 ) loanaccts on loanaccts.digitalLoanAccountId = loanmaster.digitalLoanAccountId 
left join user_type_min on cast(user_type_min.customerId as string)=cust_mtb.cust_id 
																											
--left join worktable_data_analysis.lov_sf_industry_now_emp_status_Feb23 empstatus on empstatus.digitalLoanAccountId=loanmaster.digitalLoanAccountId
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  LEFT JOIN dl_customers_db_raw.tdbk_cust_profile_mtb cust_profile on cust_profile.user_id = cust_mtb.user_id
  left join dl_customers_db_raw.tdbk_customer_add_mtb  per_addr on per_addr.user_id = cust_mtb.user_id
   left join deliquency on loanmaster.loanAccountNumber=deliquency.loanAccountNumber
   left join default_outstanding_principal on default_outstanding_principal.digitalLoanAccountId=loanmaster.digitalLoanAccountId
   left join `dl_loans_db_raw.tdbk_loan_poi3_response` poi3 on loanmaster.crifApplicationId=poi3.crifApplicationId
left join (SELECT digitalLoanAccountId,score FROM `dl_loans_db_derived.tdbk_loan_poi3_response_trustingsocial_data` 
qualify row_number() over (partition by digitalLoanAccountId order by timestamp_received desc)=1) trustingsocial on 
loanmaster.digitalLoanAccountId=trustingsocial.digitalLoanAccountId
 left join (select digitalLoanAccountId, MAX(coalesce(cast(FlagMatched as int64),0)) as matched_flag from dl_loans_db_derived.tdbk_loan_poi3_response_bureau_string_xml_parsed_matchedsubject
group by 1) flagmatched
on loanmaster.digitalLoanAccountId = flagmatched.digitalLoanAccountId
left join 
(SELECT digitalLoanAccountId,if(substr(ScoreRange,1,3)='NH_',0,1) summary_matchedflag,ScoreRange FROM `dl_loans_db_derived.tdbk_loan_poi3_response_bureau_string_xml_parsed_summary` 
qualify row_number() over (partition by digitalLoanAccountId order by run_date desc)=1) summary 
on summary.digitalLoanAccountId = loanmaster.digitalLoanAccountId
left join onb_doc_type_cust on onb_doc_type_cust.user_id=cust_mtb.user_id

																					  
left join onb_self_dec_income on onb_self_dec_income.cust_id=cast(loanmaster.customerId as string)
left join cust_closed_date on cust_closed_date.OFCUSTOMERID=cust_mtb.cust_id 
left join (select * from `prj-prod-dataplatform.dl_loans_db_raw.tdbk_customer_loan_address` 
where status=1 qualify row_number() over (partition by digital_loan_account_id order by created_dt desc)=1) loanaddress on loanaddress.digital_loan_account_id=loanmaster.digitalLoanAccountId
where cast(cust_mtb.cust_id as int64) not in (select coalesce(customerId,1) from {schema1}.{altrans}) and cust_mtb.cust_id is not null and 
cust_mtb.created_dt between '{SDATE}' and '{EDATE}' and loanmaster.loanAccountNumber is null
--where coalesce(loanmaster.termsAndConditionsSubmitDateTime,if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)) between '{SDATE}' and '2025-04-15'

) main
left join osversion_credolab on osversion_credolab.deviceid=refno
where onboarding_datetime between '{SDATE}' 
-- and date_sub(date(current_date()), interval 1 day) 
and '{EDATE}'
and loan_submission_date is null
qualify row_number() over (partition by customerId)=1)
--where c_credo_score is not null and s_credo_score	is not null and fu_credo_score is not null and r_credo_score is not null and  gen_credo_score is not null
;
"""

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema1}.{nal} created successfully')

Table risk_mart.tsa_onboarded_but_never_applied_loan_20210701_20250818 created successfully


# trench2_never_applied_snapshot_transactions

# trench2_never_applied_snapshot_transactions Table

In [24]:
schema2 = 'worktable_data_analysis'

trench2neverappliedsnapshottransactions = f'trench2_never_applied_snapshot_transactions_{datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')}_{datetime.strptime(EDATE, '%Y-%m-%d').strftime('%Y%m%d')}'

print(trench2neverappliedsnapshottransactions)


trench2_never_applied_snapshot_transactions_20210701_20250818


# trench2_never_applied_snapshot_transactions query

In [25]:
sq = f""" 
create or replace table {schema2}.{trench2neverappliedsnapshottransactions} as 
with input_customers as (
SELECT 
customerId,
credolabRefNumber onb_credolabRefNumber,
credo_inquiry_date as onb_credo_inquiry_date,
onb_tsa_onboarding_datetime,
onb_age,
onb_doc_type,
onb_os_version as onb_osversion,
onb_email,
ln_industry_new,
ln_employment_type_new,
DATE('{OBDATE}') as snapshot_date,
DATE_DIFF(DATE('{OBDATE}'),DATE(onb_tsa_onboarding_datetime),DAY) dob_snapshot_date,
DATE_DIFF(DATE(onb_tsa_onboarding_datetime),credo_inquiry_date,DAY) days_since_credo_call_onb,
DATE_DIFF(DATE('{OBDATE}'),credo_inquiry_date,DAY) AS days_since_credo_call_snapshot_date,
ln_self_dec_income,
ln_marital_status,
ln_education_level,
ln_nature_of_work_new,
ln_source_funds_new,
onb_first_name,
onb_middle_name,
onb_city,
onb_province,
onb_last_name,
ln_brand,
ln_cnt_dependents,
ln_doc_type,
ln_osversion,
onb_kyc_status,
onb_latitude,
onb_longitude,
ln_user_type,
ln_loan_type,
ln_prod_type,
ln_loan_applied_flag,
c_credo_score,
cust_status_flag,
cust_status_close_date,

FROM
{schema1}.{nal}
--where DATE_ADD(onb_tsa_onboarding_datetime,INTERVAL 180 DAY) <= '2025-03-09'
),
cust_onboarding_acc_data as (
    SELECT
    DATE(opendate,'Asia/Manila') OFDATEOPENED,
    DATE(closuredate,'Asia/Manila') ofdateclosed,
    closed OFISCLOSED,
    DATE(c.created_dt) registration_date,
    c.created_dt as onboarding_date,
    cust_id,
    DATETIME(reccreatedon,'Asia/Manila') reccreatedon,
    accountid,
    productid,
    accountdescription as account_type,
    clearedbalance,
    snapshot_date
    FROM `dl_customers_db_raw.tdbk_customer_mtb` c    
    JOIN `finastra_raw.account` b ON c.cust_id = b.ubcustomercode
    JOIN input_customers on c.cust_id = CAST(input_customers.customerid AS STRING)
),
main_transaction_data AS 
(
     SELECT 
    transaction_date,
    OFDATEOPENED,
    OFISCLOSED,
    registration_date,
    transaction_id,
    b.cust_id customer_id,
    a.accountid,
    productid,
    b.account_type,
    transaction_code,
    a.status,
    channel,
    credit_debit_indicator,
    inter_exter_flag,
    trx_amount,
    core_narration,
    input_customers.snapshot_date
    -- customer_transactions to get the transactions
    FROM cust_onboarding_acc_data b
    JOIN input_customers on b.cust_id = CAST(input_customers.customerid AS STRING)
    LEFT JOIN `risk_mart.customer_transactions` a ON a.accountid = b.accountid
    and a.transaction_date < input_customers.snapshot_date and a.status = 'Success'
    --and a.transaction_datetime < input_customers.snapshot_date
    WHERE 1=1
   
),


#### Net Cash In ####
  -- 1. Outside Tonik to TSA
  -- 2. Other Tonik user to Own Tonik Account


net_cash_in AS 
(
  ## 1. Outside Tonik to TSA
  SELECT
    transaction_date,
    OFDATEOPENED,
    OFISCLOSED,
    registration_date,
    transaction_id,
    customer_id,
    accountid,
    account_type,
    status,
    channel,
    credit_debit_indicator,
    inter_exter_flag,
    trx_amount,
    core_narration,
    'Net Cash In' main_transaction_type,
    'Outside Tonik to TSA' sub_transaction_type,
    snapshot_date
  FROM main_transaction_data
  WHERE 1=1
  -- main conditions: should be a successful transaciton and credit and all coming from Tonik Account
  AND credit_debit_indicator = 'CREDIT'
  AND account_type = 'Tonik Account' and LOWER(core_narration) NOT LIKE '%blocking%' and transaction_code not like 'A0%'
  AND transaction_code IN ('N01','IP2','XE2','00T','21C','P01')
  -- 1. Outside Tonik to TSA conditions (all cash in)
  AND inter_exter_flag = 'Outside Tonik'

  UNION ALL

  ## 2. Other Tonik user to Own Tonik Account
  SELECT
    transaction_date,
    OFDATEOPENED,
    OFISCLOSED,
    registration_date,
    transaction_id,
    customer_id,
    accountid,
    account_type,
    status,
    channel,
    credit_debit_indicator,
    inter_exter_flag,
    trx_amount,
    core_narration,
    'Net Cash In' main_transaction_type,
    'Other Tonik Users to Town Tonik Account' sub_transaction_type,
    snapshot_date
  FROM main_transaction_data
  WHERE 1=1
  -- main conditions: should be a successful transaciton and credit and all coming from Tonik Account
  AND credit_debit_indicator = 'CREDIT'
  AND account_type = 'Tonik Account' and LOWER(core_narration) NOT LIKE '%blocking%' and transaction_code not like 'A0%'
  AND transaction_code IN ('N01','IP2','XE2','00T','21C','P01')

  -- 2. Other Tonik user to Own Tonik Account
  AND inter_exter_flag = 'Inside Tonik'
  AND core_narration LIKE '%Receive money from other Tonik Account%'
  -- AND LEFT(core_narration,STRPOS(core_narration, ",")-1) = 'Receive money from other Tonik Account'
)

#### Net Cash Out ####
-- 1. Bills Pay
-- 2. Card Transactions
-- 3. Own TSA to other Tonik Users
-- 4. TSA to Outside Tonik

, net_cash_out AS 
(

 ## 1. Bills Pay
  SELECT 
    transaction_date,
    OFDATEOPENED,
    OFISCLOSED,
    registration_date,
    transaction_id,
    customer_id,
    accountid,
    account_type,
    status,
    channel,
    credit_debit_indicator,
    inter_exter_flag,
    trx_amount,
    core_narration,
    'Net Cash Out' main_transaction_type,
    'Bills Pay' sub_transaction_type,
    snapshot_date,
  FROM main_transaction_data
  WHERE 1=1
  -- main conditions: should be a successful transaciton and debit
  AND credit_debit_indicator = 'DEBIT'
  AND LOWER(core_narration) NOT LIKE '%blocking%'

  -- 1. Bills Pay
  AND channel = 'Billspay'


  UNION ALL

  ## 2. Card Transactions (Cash Out)
  SELECT
    a.transaction_date,
    a.OFDATEOPENED,
    a.OFISCLOSED,
    a.registration_date,
    a.transaction_id,
    a.customer_id,
    a.accountid,
    a.account_type,
    a.status,
    a.channel,
    a.credit_debit_indicator,
    a.inter_exter_flag,
    a.trx_amount,
    a.core_narration,
    'Net Cash Out' main_transaction_type,
    'Card Transactions (Cash Out)' sub_transaction_type,
    snapshot_date
  FROM main_transaction_data a
  -- 2. Card Transactions (Cash Out) -- using the table made above
  WHERE 1=1
  -- main conditions: should be a successful transaciton and debit and coming from tonik account
  AND a.credit_debit_indicator = 'DEBIT'
  AND a.account_type = 'Tonik Account'
  AND transaction_code like 'A0%' and core_narration not like '%Blocking%'

  UNION ALL

  ## 3. Own TSA to other Tonik Users
  SELECT DISTINCT
    transaction_date,
    OFDATEOPENED,
    OFISCLOSED,
    registration_date,
    transaction_id,
    customer_id,
    accountid,
    account_type,
    status,
    channel,
    credit_debit_indicator,
    inter_exter_flag,
    trx_amount,
    core_narration,
    'Net Cash Out' main_transaction_type,
    'Own TSA to Other Tonik Users' sub_transaction_type,
    snapshot_date
  FROM main_transaction_data a
  WHERE 1=1
  -- main conditions: should be a successful transaciton and debit
  AND a.credit_debit_indicator = 'DEBIT'
  AND a.account_type = 'Tonik Account'
  AND transaction_code not like 'A0%' and core_narration not like '%Blocking%'

  -- 3. Own TSA to other Tonik Users
  AND a.channel = 'Core transactions'
  AND a.inter_exter_flag = 'Inside Tonik'
  --AND LOWER(core_narration) LIKE '%send money to other tonik account%' 
  -- AND LOWER(core_narration) NOT LIKE '%scontri%'
  -- AND LOWER(core_narration) NOT LIKE '%stash%'
  -- AND LOWER(core_narration) NOT LIKE '%time deposit%'

  UNION ALL

  ## 4. TSA to Outside Tonik (Other banks)
  SELECT DISTINCT
    transaction_date,
    OFDATEOPENED,
    OFISCLOSED,
    registration_date,
    transaction_id,
    customer_id,
    accountid,
    account_type,
    status,
    channel,
    credit_debit_indicator,
    inter_exter_flag,
    trx_amount,
    core_narration,
    'Net Cash Out' main_transaction_type,
    'TSA to Outside Tonik (Other Banks)' sub_transaction_type,
    snapshot_date
  FROM main_transaction_data a
  WHERE 1=1
  -- main conditions: should be a successful transaciton and debit
  AND a.credit_debit_indicator = 'DEBIT'
  AND a.account_type = 'Tonik Account'
  AND core_narration not like '%Blocking%'

  -- channels not in core transactions and billspay with the flag as outside tonik are sending to other banks
  AND a.channel NOT IN  ('Core transactions','Billspay')
  AND a.inter_exter_flag = 'Outside Tonik'
)

, transactions_sub AS 
(
  -- merging the cash ins and cash outs
  SELECT DISTINCT *
  FROM net_cash_in 
  UNION ALL
  SELECT DISTINCT *
  FROM net_cash_out
)

, date_diff_sub AS 
(
    -- to get the date difference between 2 transactions (cash in and cash out)
    SELECT customer_id,
    'Overall' days_diff_type,
    DATE_DIFF(LEAD(transaction_date) OVER (PARTITION BY customer_id ORDER BY transaction_date,core_narration ASC),transaction_date,DAY) days_bt_trans,
    snapshot_date
    FROM 
    (
        SELECT DISTINCT
        transaction_date,
        customer_id,
        main_transaction_type,
        snapshot_date,
        core_narration
        FROM transactions_sub
        WHERE 1=1
        --   AND customer_id IN ('2077378','2081999','2475220','2485072')
    )

    UNION ALL

    -- to get the date difference between 2 cash ins
    SELECT customer_id,
    'Cash In' days_diff_type,
    DATE_DIFF(LEAD(transaction_date) OVER (PARTITION BY customer_id ORDER BY transaction_date ASC),transaction_date,DAY) days_bt_trans,
    snapshot_date
    FROM 
    (
        SELECT DISTINCT
        transaction_date,
        customer_id,
        main_transaction_type,
        snapshot_date,
        core_narration
        FROM transactions_sub
        WHERE 1=1
        --   AND customer_id IN ('2077378','2081999','2475220','2485072')
        AND main_transaction_type = 'Net Cash In'
    )

    UNION ALL

    -- to get the date difference between 2 cash outs
    SELECT customer_id,
    'Cash Out' days_diff_type,
    DATE_DIFF(LEAD(transaction_date) OVER (PARTITION BY customer_id ORDER BY transaction_date,core_narration ASC),transaction_date,DAY) days_bt_trans,
    snapshot_date
    FROM 
    (
        SELECT DISTINCT
        transaction_date,
        customer_id,
        main_transaction_type,
        snapshot_date,
        core_narration
        FROM transactions_sub
        WHERE 1=1
        --   AND customer_id IN ('2077378','2081999','2475220','2485072')
        AND main_transaction_type = 'Net Cash Out'
    )
)

, days_bt_trans_avg AS 
(
-- get the average days in between 
SELECT DISTINCT
customer_id,
AVG(IF(days_diff_type='Overall',days_bt_trans,NULL)) overall_avg_days_bt_trans,
AVG(IF(days_diff_type='Cash In',days_bt_trans,NULL)) net_cash_in_avg_days_bt_trans,
AVG(IF(days_diff_type='Cash Out',days_bt_trans,NULL)) net_cash_out_avg_days_bt_trans
FROM date_diff_sub
GROUP BY 1
)

, days_bt_trans_med AS 
(
-- get the median days in between
SELECT DISTINCT
customer_id,
PERCENTILE_CONT(IF(days_diff_type='Overall',days_bt_trans,NULL), .50) OVER (PARTITION BY customer_id) overall_med_days_bt_trans,
PERCENTILE_CONT(IF(days_diff_type='Cash In',days_bt_trans,NULL), .50) OVER (PARTITION BY customer_id) cash_in_med_days_bt_trans,
PERCENTILE_CONT(IF(days_diff_type='Cash Out',days_bt_trans,NULL), .50) OVER (PARTITION BY customer_id) cash_out_med_days_bt_trans,
FROM date_diff_sub
)

, transactions_final AS 
(
SELECT DISTINCT
acc.customerid as customer_id,
## Number of transactions within the observation window (x days from onboarding date),
## Cash In Count Details
COUNT(DISTINCT(IF(main_transaction_type = 'Net Cash In',transaction_id,NULL))) tx_cnt_cash_in_total,
COUNT(DISTINCT(IF(sub_transaction_type='Outside Tonik to TSA' ,transaction_id,NULL))) tx_cnt_cash_in_ob2t,
COUNT(DISTINCT(IF(sub_transaction_type='Other Tonik Users to Town Tonik Account' ,transaction_id,NULL))) tx_cnt_cash_in_ot2t,

## Cash In Amount Details
SUM(IF(main_transaction_type = 'Net Cash In',trx_amount,NULL)) tx_amt_cash_in_total,
SUM((IF(sub_transaction_type='Outside Tonik to TSA' ,trx_amount,NULL))) tx_amt_cash_in_ob2t,
SUM((IF(sub_transaction_type='Other Tonik Users to Town Tonik Account',trx_amount,NULL))) tx_amt_cash_in_ot2t,

## Cash Out Count Details
COUNT(DISTINCT(IF(main_transaction_type = 'Net Cash Out',transaction_id,NULL))) tx_cnt_cash_out_total,
COUNT(DISTINCT(IF(sub_transaction_type= 'Bills Pay' ,transaction_id,NULL))) tx_cnt_cash_out_billpay,
COUNT(DISTINCT(IF(sub_transaction_type= 'Card Transactions (Cash Out)' ,transaction_id,NULL))) tx_cnt_cash_out_cards,
COUNT(DISTINCT(IF(sub_transaction_type= 'Own TSA to Other Tonik Users' ,transaction_id,NULL))) tx_cnt_cash_out_t2ot,
COUNT(DISTINCT(IF(sub_transaction_type= 'TSA to Outside Tonik (Other Banks)' ,transaction_id,NULL))) tx_cnt_cash_out_t2ob,

## Cash Out Amount Details
SUM(IF(main_transaction_type = 'Net Cash Out',trx_amount,NULL)) tx_amt_cash_out_total,
SUM(IF(sub_transaction_type= 'Bills Pay' ,trx_amount,NULL)) tx_amt_cash_out_billpay,
SUM(IF(sub_transaction_type= 'Card Transactions (Cash Out)' ,trx_amount,NULL)) tx_amt_cash_out_cards,
SUM(IF(sub_transaction_type= 'Own TSA to Other Tonik Users' ,trx_amount,NULL)) tx_amt_cash_out_t2ot,
SUM(IF(sub_transaction_type= 'TSA to Outside Tonik (Other Banks)' ,trx_amount,NULL)) tx_amt_cash_out_t2ob,
FROM input_customers acc 
LEFT JOIN transactions_sub a ON CAST(acc.customerid AS STRING) = a.customer_id and date(transaction_date) < date(acc.snapshot_date)
GROUP BY 1
ORDER BY 2 

),

utility_transaction_data AS (
        SELECT 
        customer_id,
        CASE 
            WHEN SUM(CASE WHEN transaction_code IN ('BP1', 'BP2', 'BP3', 'BP4') THEN 1 ELSE 0 END) > 0 
                THEN MIN(transaction_date) 
            ELSE NULL 
        END AS first_billpay_date,
        CASE 
            WHEN SUM(CASE WHEN transaction_code like 'A0%' AND core_narration NOT LIKE '%Blocking%' THEN 1 ELSE 0 END) > 0 
                THEN MIN(transaction_date) 
            ELSE NULL 
        END AS virtual_transaction_date,
        CASE 
            WHEN SUM(CASE WHEN transaction_code IN ('21C', 'N01', 'IP2', 'XE2','P01') THEN 1 ELSE 0 END) > 0 
                THEN MIN(transaction_date) 
            ELSE NULL 
        END AS first_tsa_topup_date
    FROM 
        main_transaction_data
    GROUP BY 1
),

combined_data AS (
    SELECT 
        COALESCE(acc.cust_id,acc_data.customer_id,utility_transaction_data.customer_id) customer_id,
        productid,
        accountdescription,
        acc_data.opendate as opendate ,
        first_billpay_date,
        virtual_transaction_date,
        first_tsa_topup_date,
        snapshot_date,
        LEAST(
            DATE(snapshot_date),
            IFNULL(DATE(acc_data.opendate),'9999-12-31'), 
            IFNULL(first_billpay_date, '9999-12-31'), 
            IFNULL(virtual_transaction_date, '9999-12-31'), 
            IFNULL(first_tsa_topup_date, '9999-12-31')
        ) AS first_opened_date
        FROM (SELECT DISTINCT cust_id,registration_date,snapshot_date from cust_onboarding_acc_data) acc 
        LEFT JOIN (SELECT cust_id as customer_id,ofdateopened opendate ,productid,account_type as accountdescription from cust_onboarding_acc_data 
        WHERE  account_type NOT IN ('Tonik Account','Tendo Individual Stash') AND productid in ('fixdep','savings','SaveForFuture') AND ofdateopened >= '2023-01-01' AND DATE(reccreatedon) < DATE(snapshot_date)
       QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY reccreatedon asc) = 1) acc_data
       ON acc.cust_id = acc_data.customer_id
      LEFT JOIN utility_transaction_data
        ON 
      acc.cust_id = utility_transaction_data.customer_id
),
first_product_data as (
SELECT
    customer_id,
    --first_opened_date,
    CASE
        WHEN first_opened_date = DATE(opendate) THEN accountdescription
        WHEN first_opened_date = first_billpay_date THEN 'Bills Pay'
        WHEN first_opened_date = first_tsa_topup_date THEN 'TSA Top-Up'
        WHEN first_opened_date = virtual_transaction_date THEN 'Virtual Transaction'
        ELSE 'Unknown'
    END AS first_product,
    CASE
        WHEN first_opened_date = DATE(opendate) and productid in ('fixdep','savings','SaveForFuture') THEN 'Deposit Users'
        --WHEN first_opened_date = DATE(opendate) and productid NOT IN ('fixdep','savings','SaveForFuture') THEN 'Loan Users'
        WHEN first_opened_date = first_tsa_topup_date or first_opened_date = virtual_transaction_date or first_opened_date = first_billpay_date THEN 'Utility Users'
        ELSE 'Ghost Users'
    END AS first_product_user_segment
FROM
    combined_data
),
loan_metrics as (
SELECT 
input_customers.customerid,

count(DISTINCT(IF (lmt.digitalLoanAccountId IS NOT NULL AND coalesce(lmt.termsAndConditionsSubmitDateTime,if (lmt.new_loan_type ='Flex-up',lmt.startApplyDateTime,lmt.termsAndConditionsSubmitDateTime)) is null,lmt.digitalLoanAccountId,NULL))) tx_incomplete_loan_apps,
from input_customers
JOIN `risk_credit_mis.loan_master_table` lmt on cast(lmt.customerId as string) = input_customers.customerid and COALESCE(DATE(termsAndConditionsSubmitDateTime),DATE(startApplyDateTime))  < DATE(input_customers.snapshot_date)
group by 1
),
complete_deposit_metrics as (
with  deposit_acc_main AS 
(
   SELECT
  a.OFDATEOPENED as ofdateopened,
  IF(OFISCLOSED = 'Y',DATE_DIFF(date(ofdateclosed),date(OFDATEOPENED),day),NULL) as stash_duration,
  registration_date,
  duration,
  reccreatedon,
  a.cust_id as customer_id,
  a.account_type,
  a.accountid as ofstandardaccountid,
  --balancedateasof,
  b.clearedbalance,
  a.OFISCLOSED as closed,
  ff.status as td_status,
  a.ofdateclosed,
  autorollover,
  snapshot_date,
  FROM cust_onboarding_acc_data a
  JOIN risk_mart.customer_balance b on a.accountid = b.accountid and date(balanceDateAsOf) = DATE_SUB(DATE(snapshot_date),INTERVAL 1 DAY)
  --JOIN input_customers on input_customers.customerid = a.cust_id 
  LEFT JOIN `finastra_raw.fixturefeature` ff on ff.accountid = a.accountid
  WHERE 1=1 
  --and date(balancedateasof) = DATE_SUB(p.start_date,INTERVAL 1 DAY)
  and productid in ('savings','fixdep','SaveForFuture') and  DATE(reccreatedon) < DATE(snapshot_date)
  )
  
,deposit_days_diff_sub as (
    SELECT customer_id,
    'Between All Deposits' days_diff_type,
    DATE_DIFF(LEAD(ofdateopened) OVER (PARTITION BY customer_id ORDER BY ofdateopened ASC),ofdateopened,DAY) days_bt_trans
    FROM deposit_acc_main

    UNION ALL

    SELECT customer_id,
    'Between TDs' days_diff_type,
    DATE_DIFF(LEAD(ofdateopened) OVER (PARTITION BY customer_id ORDER BY ofdateopened ASC),ofdateopened,DAY) days_bt_trans
    FROM deposit_acc_main
    WHERE account_type not like '%Stash%'

)
,dep_days_bt_trans_med AS 
(
-- get the median days in between
SELECT DISTINCT
customer_id,

PERCENTILE_CONT(IF(days_diff_type='Between All Deposits',days_bt_trans,NULL), .50) OVER (PARTITION BY customer_id) med_days_bw_new_dep_acct_open,
PERCENTILE_CONT(IF(days_diff_type='Between TDs',days_bt_trans,NULL), .50) OVER (PARTITION BY customer_id) med_days_bw_td_acct_open,
FROM deposit_days_diff_sub
)

, deposit_account_counts AS 
(
#### Number of Stash and Time Deposit accounts that are still open until the observation date with balance >= 100
SELECT
customer_id,
CASE WHEN SUM(autorollover) >= 1.0 THEN 1 ELSE 0 END AS tx_td_auto_roll_over_enabled,
MAX((IF(account_type LIKE '%Time Deposit%' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) < DATE(snapshot_date)),duration,0))) AS td_max_duration,
AVG((IF(account_type LIKE '%Time Deposit%'and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)) ,duration,0))) AS td_avg_duration,
MIN((IF(account_type LIKE '%Time Deposit%'and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),duration,0))) AS td_min_duration,
MAX((IF(account_type LIKE '%Stash%' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),stash_duration,0))) AS stash_max_duration,
AVG((IF(account_type LIKE '%Stash%' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),stash_duration,0))) AS stash_avg_duration,
MIN((IF(account_type LIKE '%Stash%' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),stash_duration,0))) AS stash_min_duration,
SUM(DISTINCT(IF(account_type LIKE '%Time Deposit%' AND td_status = '1' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),clearedbalance,NULL))) td_balance,
SUM(DISTINCT(IF(account_type LIKE '%Stash%' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),clearedbalance,NULL))) stash_balance,
COUNT(DISTINCT(IF(account_type LIKE '%Time Deposit%' AND td_status = '4' and closed = 'Y' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),ofstandardaccountid,NULL))) td_accounts_completed_cnt,
COUNT(DISTINCT(IF(account_type LIKE '%Time Deposit%' AND td_status = '9' and closed = 'Y' and (DATE(ofdateclosed) = '1970-01-01' OR DATE(ofdateclosed) > DATE(snapshot_date)),ofstandardaccountid,NULL))) td_accounts_broken_cnt,
COUNT(DISTINCT(IF(account_type <> 'Tonik Account', ofstandardaccountid,NULL))) deposit_accs_cnt,
COUNT(DISTINCT(IF(account_type LIKE '%Stash%',ofstandardaccountid,NULL))) stash_accounts_opened_cnt,
COUNT(DISTINCT(IF(account_type LIKE '%Time Deposit%',ofstandardaccountid,NULL))) td_accounts_opened_cnt,
COUNT(DISTINCT(IF(account_type LIKE '%Stash%' and closed = 'Y' and (DATE(ofdateclosed) = '1970-01-01' OR  DATE(ofdateclosed) > DATE(snapshot_date)) ,ofstandardaccountid,NULL))) stash_accounts_closed_cnt,
FROM 
deposit_acc_main
where
DATE(reccreatedon) < DATE(snapshot_date) and account_type <> 'Tonik Account'
GROUP BY 1
)

SELECT DISTINCT 
a.customer_id,
td_balance,
stash_balance,
tx_td_auto_roll_over_enabled,
deposit_accs_cnt,
stash_accounts_opened_cnt,
stash_accounts_closed_cnt,
td_accounts_opened_cnt,
td_accounts_completed_cnt,
td_accounts_broken_cnt,
med_days_bw_new_dep_acct_open,
med_days_bw_td_acct_open,
td_max_duration,
td_min_duration,
td_avg_duration,
stash_max_duration,
stash_avg_duration,
stash_min_duration,
FROM deposit_account_counts a
--LEFT JOIN deposit_account_90th_day_counts d ON a.customer_id = d.customer_id
LEFT JOIN dep_days_bt_trans_med b ON b.customer_id = a.customer_id
ORDER BY 2 DESC
)
SELECT
COALESCE(first_product_data.first_product,'Unknown') as tx_first_product,
COALESCE(first_product_data.first_product_user_segment,'Ghost Users') tx_first_product_user_segment,
acc.customerid as customer_id,
onb_tsa_onboarding_datetime,
onb_credo_inquiry_date,
onb_credolabRefNumber,
null as ln_disb_flag,
null as ln_loan_type,
null as ln_prod_type,
null as ln_loan_appln_time,
null as first_disb_loan_disbursed_time,
null as first_applied_loan_appln_time,
null as first_applied_loan_type,
null as first_applied_loan_tenor,
null as first_applied_loan_amount,
null as first_applied_loan_decision,
null as first_applied_product_type,
null as last_applied_loan_appln_time,
null as last_applied_loan_decision,
null as last_applied_os_type,
null as last_applied_loan_type,
null as last_applied_loan_tenor,
null as last_applied_loan_amount,
null as last_applied_product_type,
null as last_applied_crif_id,
null as last_applied_cic_score,
null as last_applied_credo_ref_no,
null as last_applied_credo_score,
null as last_applied_demo_score,
null as last_applied_apps_score,
null as ln_fpd30_flag,
null as ln_mature_fpd30_flag,
null as ln_mature_fspd30_flag,
null as ln_fspd30_flag,
null as first_reject_loan_appln_time,
null as first_rejected_digitalloanaccountId,
null as first_reject_loan_type,
null as first_reject_loan_amount,
null as first_reject_loan_tenor,
null as last_reject_loan_appln_time,
null as last_rejected_digitalloanaccountId,
null as last_reject_loan_type,
null as last_reject_loan_amount,
null as last_reject_loan_tenor,
dob_snapshot_date,
days_since_credo_call_onb,
days_since_credo_call_snapshot_date,
onb_age,
onb_doc_type,
onb_osversion,
c_credo_score,
ln_self_dec_income,
ln_marital_status,
ln_education_level,
onb_email,
ln_industry_new,
ln_employment_type_new,
snapshot_date,
ln_nature_of_work_new,
ln_source_funds_new,
onb_first_name,
onb_middle_name,
onb_last_name,
ln_brand,
ln_cnt_dependents,
ln_doc_type,
ln_osversion,
ln_user_type,
cust_status_flag,
cust_status_close_date,
0 as ln_ever_applied_flag,
onb_city,
onb_province,
EXTRACT(YEAR FROM onb_tsa_onboarding_datetime) onb_year,
EXTRACT(MONTH FROM onb_tsa_onboarding_datetime) onb_month_of_year,
EXTRACT(WEEK FROM onb_tsa_onboarding_datetime) - EXTRACT(WEEK FROM DATE_TRUNC(onb_tsa_onboarding_datetime, MONTH)) + 1 onb_week_of_month,
EXTRACT(DAY FROM onb_tsa_onboarding_datetime) onb_day_of_month,
EXTRACT(TIME FROM onb_tsa_onboarding_datetime) onb_time_of_day,
a.tx_cnt_cash_in_total,
a.tx_cnt_cash_in_ob2t,
a.tx_cnt_cash_in_ot2t,
a.tx_amt_cash_in_total,
a.tx_amt_cash_in_ob2t,
a.tx_amt_cash_in_ot2t,
a.tx_cnt_cash_out_total,
a.tx_cnt_cash_out_billpay,
a.tx_cnt_cash_out_cards,
a.tx_cnt_cash_out_t2ot,
a.tx_cnt_cash_out_t2ob,
a.tx_amt_cash_out_total,
a.tx_amt_cash_out_billpay,
a.tx_amt_cash_out_cards,
a.tx_amt_cash_out_t2ot,
a.tx_amt_cash_out_t2ob,
overall_avg_days_bt_trans tx_avg_days_bt_trans,
net_cash_in_avg_days_bt_trans tx_avg_days_bt_cash_in_trans,
net_cash_out_avg_days_bt_trans tx_avg_days_bt_cash_out_trans,
overall_med_days_bt_trans tx_med_days_bt_trans,
cash_in_med_days_bt_trans tx_med_days_bt_cash_in_trans,
cash_out_med_days_bt_trans tx_med_days_bt_cash_out_trans,
deposit_accs_cnt tx_deposit_accnt_cnt,
stash_accounts_opened_cnt tx_stash_accnt_opened_cnt,
stash_accounts_closed_cnt tx_stash_accnt_closed_cnt,
stash_balance tx_stash_balance,
td_accounts_opened_cnt tx_td_accnt_opened_cnt,
td_accounts_completed_cnt tx_td_accnt_completed_cnt,
td_accounts_broken_cnt tx_td_accnt_broken_cnt,
tx_td_auto_roll_over_enabled,
td_balance tx_td_balance,
td_max_duration,
td_min_duration,
td_avg_duration,
stash_max_duration,
stash_avg_duration,
stash_min_duration,
med_days_bw_new_dep_acct_open tx_med_days_bw_new_dep_acct_open,
med_days_bw_td_acct_open tx_med_days_bw_td_acct_open,
null as tx_cnt_completed_loans,
null as tx_cnt_rejected_loans,
null as tx_cnt_active_loans,
null as tx_cnt_applied_loan,
null as tx_cnt_approved_loans,
null as tx_cnt_disbursed_loans,
tx_incomplete_loan_apps as tx_cnt_incomplete_loan_apps,
null as tx_cnt_installments_paid_tot,
null as tx_amt_installments_paid_tot,
null as tx_cnt_installments_paid_last_disb,
null as tx_amt_installments_paid_last_disb,
null as tx_cnt_fpd10_ever,
null as tx_cnt_fpd30_ever,
null as tx_cnt_fspd30_ever,
null as tx_cnt_dpd_gt_5_ever,
null as tx_max_ever_dpd,
null as tx_max_dpd_30d,
null as tx_max_dpd_60d,
null as tx_max_dpd_90d,
null as tx_max_dpd_120d,
null as tx_max_dpd_150d,
null as tx_max_dpd_180d,
null as tx_max_current_dpd,
null as ln_any_prev_disb_loan_sil_mobile_flag,
CASE WHEN tdbk_referral_code_mtb.cust_id is not null THEN 1
ELSE 0 END AS referral_flag

FROM input_customers acc
LEFT JOIN first_product_data ON first_product_data.customer_id = acc.customerid 
LEFT JOIN transactions_final a ON acc.customerid = a.customer_id
LEFT JOIN days_bt_trans_avg b ON acc.customerid = b.customer_id 
LEFT JOIN days_bt_trans_med c ON acc.customerid = c.customer_id
LEFT JOIN complete_deposit_metrics d on d.customer_id = acc.customerid
LEFT JOIN loan_metrics ON cast(loan_metrics.customerid as string) = acc.customerid
LEFT JOIN (SELECT cust_id,member_type,referral_type FROM dl_customers_db_raw.tdbk_referral_code_mtb
WHERE tdbk_referral_code_mtb.member_type='REFEREE') tdbk_referral_code_mtb on  tdbk_referral_code_mtb.cust_id = acc.customerid
;
"""

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema2}.{trench2neverappliedsnapshottransactions} created successfully')

Table worktable_data_analysis.trench2_never_applied_snapshot_transactions_20210701_20250818 created successfully


# trench2_never_applied_snapshot_events

# trench2_never_applied_snapshot_events Table

In [26]:
schema2 = 'worktable_data_analysis'

trench2neverappliedsnapshotevents = f'trench2_never_applied_snapshot_events_{datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')}_{datetime.strptime(EDATE, '%Y-%m-%d').strftime('%Y%m%d')}'

print(trench2neverappliedsnapshotevents)


trench2_never_applied_snapshot_events_20210701_20250818


# trench2_never_applied_snapshot_events query

In [27]:
sq = f"""
CREATE OR REPLACE TABLE {schema2}.{trench2neverappliedsnapshotevents} as
with input_customers as ( 
SELECT 
customerId,
onb_tsa_onboarding_datetime,
onb_age,
onb_email,
ln_industry_new,
ln_employment_type_new,
DATE('{OBDATE}') as snapshot_date,
ln_nature_of_work_new,
ln_doc_type,
ln_osversion,
onb_kyc_status,
onb_latitude,
onb_longitude,
ln_user_type,
ln_loan_type,
ln_prod_type,
ln_loan_applied_flag,
FROM
{schema1}.{nal}
),
af_link AS
(
  ## To get the AF ID and Customer ID Link (using the first install of a customer)
  SELECT DISTINCT appsflyer_id, customer_user_id, install_time,snapshot_date
  FROM `appsflyer_raw.in_app_events_report` in_apps_events
  JOIN `dl_customers_db_raw.tdbk_customer_mtb` c ON c.cust_id = in_apps_events.customer_user_id
  JOIN input_customers ON In_apps_events.customer_user_id = input_customers.customerid and DATE(in_apps_events._partitiondate) < DATE(input_customers.snapshot_date)
  WHERE 1=1
  AND customer_user_id IS NOT NULL
  QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_user_id ORDER BY install_time ASC) = 1
 
  UNION ALL
 
  SELECT DISTINCT appsflyer_id, customer_user_id, install_time,snapshot_date
  FROM `appsflyer_raw.organic_in_app_events_report` organic_in_apps_events
  JOIN `dl_customers_db_raw.tdbk_customer_mtb` c ON c.cust_id = organic_in_apps_events.customer_user_id
 JOIN input_customers ON organic_in_apps_events.customer_user_id = input_customers.customerid and DATE(organic_in_apps_events._partitiondate) < DATE(input_customers.snapshot_date)
  WHERE 1=1
  AND customer_user_id IS NOT NULL
  QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_user_id ORDER BY install_time ASC) = 1
)
, events AS
(
  SELECT DISTINCT
  cust_id as customer_id,
  --snapshot_date,
COUNT(DISTINCT IF(event_name = 'App_Launch',event_uuid,NULL)) meng_no_of_logins,
COUNT(DISTINCT IF(
    event_name IN ('Loans_QL_Calculator'),moengagerefid,NULL)) meng_ql_calculator_count,
COUNT(DISTINCT IF(
    event_name IN ('Loans_QL_Calculator'),event_uuid,NULL)) meng_ql_calculator_tot_visit_cnt
  FROM `dl_customers_db_raw.tdbk_customer_mtb` c
  LEFT JOIN `moengage_raw.events_hourly` a ON c.cust_id = cast(a.customer_id as string)
 JOIN input_customers ON cast(a.customer_id as string) = input_customers.customerid 
  WHERE 1=1 and DATE(event_time) < DATE(input_customers.snapshot_date)
  --AND event_name IN ('App_Launch','Loans_QL_Calculator','Loans_QL_Launch','Loans_SIL_Launch')
  group by 1
  ),

campaign_data as
(  SELECT DISTINCT a.customer_user_id, a.appsflyer_id, a.media_source, a.partner, a.campaign, a.Retargeting_Conversion_Type, 
  CASE
  WHEN a.media_source = 'website_channel=website_ss_ui=true_ss_gtm_ui=true_ss_qr=c' THEN 'Website'
  WHEN a.media_source IS NULL AND a.partner IS NULL AND a.campaign IS NULL THEN 'Organic'
  ELSE b.source END source,
CASE
  WHEN a.media_source = 'website_channel=website_ss_ui=true_ss_gtm_ui=true_ss_qr=c' THEN 'Website'
  WHEN a.media_source IS NULL AND a.partner IS NULL AND a.campaign IS NULL THEN 'Organic'
  ELSE b.source_group END source_group,
  FROM 
  (
    SELECT DISTINCT
    install_time,
    customer_user_id,
    AppsFlyer_ID,
    media_source,
    partner,
    campaign,
    'Install' Retargeting_Conversion_Type,
    FROM `appsflyer_raw.in_app_events_report`
    WHERE 1=1

    UNION ALL

    SELECT DISTINCT
    install_time,
    customer_user_id,
    AppsFlyer_ID,
    media_source,
    partner,
    campaign,
    'Install' Retargeting_Conversion_Type
    FROM `appsflyer_raw.organic_in_app_events_report`
    WHERE 1=1

    UNION ALL

    SELECT DISTINCT
    install_time,
    customer_user_id,
    AppsFlyer_ID,
    media_source,
    partner,
    campaign,
    Retargeting_Conversion_Type
    FROM `appsflyer_raw.in_app_events_retarget`
    WHERE 1=1
  ) a
  LEFT JOIN `prj-prod-dataplatform.worktable_datachampions.installs_attribution_mapping` b
  ON 
    COALESCE(a.media_source,a.partner,a.campaign) = COALESCE(b.media_source,b.partner,b.campaign)

  WHERE 1=1
 
  QUALIFY ROW_NUMBER() OVER (PARTITION BY a.customer_user_id ORDER BY install_time DESC) = 1
  ORDER BY customer_user_id
 ),
final_output as (
SELECT DISTINCT
input_customers.customerId,
--input_customers.snapshot_date,
TIMESTAMP_DIFF(MIN(cust_mtb.created_dt),MIN(install_time),MINUTE) appsflyer_install_to_registration_minutes,
FROM input_customers
JOIN `dl_customers_db_raw.tdbk_customer_mtb` cust_mtb ON cust_mtb.cust_id = input_customers.customerId
--LEFT JOIN events c ON c.customer_id = input_customers.customerId and input_customers.snapshot_date = c.snapshot_date
LEFT JOIN af_link b ON b.customer_user_id = input_customers.customerId and input_customers.snapshot_date = b.snapshot_date
--LEFT JOIN campaign_data ON campaign_data.customer_user_id = input_customers.customerId
group by 1
)
SELECT 
final_output.*,
meng_no_of_logins,
meng_ql_calculator_count,
COALESCE(meng_ql_calculator_tot_visit_cnt,0) meng_ql_calculator_tot_visit_cnt,
CASE WHEN campaign_data.source_group = 'Organic' THEN 'Organic'
ELSE 'InOrganic' END AS channel_source_group,
source as marketing_source_name,
from final_output
LEFT JOIN campaign_data ON campaign_data.customer_user_id = final_output.customerId
LEFT JOIN events c ON c.customer_id = final_output.customerId-- and final_output.ln_loan_appln_time = c.ln_loan_appln_time
"""

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema2}.{trench2neverappliedsnapshotevents} created successfully')

Table worktable_data_analysis.trench2_never_applied_snapshot_events_20210701_20250818 created successfully


# trench2_never_applied_snapshot_call_count

# trench2_never_applied_snapshot_call_count Table

In [28]:
schema2 = 'worktable_data_analysis'

trench2neverappliedsnapshotcallcount = f'trench2_never_applied_snapshot_call_count_{datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')}_{datetime.strptime(EDATE, '%Y-%m-%d').strftime('%Y%m%d')}'

print(trench2neverappliedsnapshotcallcount)


trench2_never_applied_snapshot_call_count_20210701_20250818


# trench2_never_applied_snapshot_call_count query

In [29]:
sq = f"""
create or replace table {schema2}.{trench2neverappliedsnapshotcallcount} as
with input_customers as (
SELECT 
customerId,
onb_tsa_onboarding_datetime,
onb_age,
onb_email,
ln_industry_new,
ln_employment_type_new,
DATE('{OBDATE}') as snapshot_date,
ln_nature_of_work_new,
ln_doc_type,
ln_osversion,
onb_kyc_status,
onb_latitude,
onb_longitude,
ln_user_type,
ln_loan_type,
ln_prod_type,
ln_loan_applied_flag,
FROM
{schema1}.{nal}
--where DATE_ADD(onb_tsa_onboarding_datetime,INTERVAL 90 DAY) <= '2025-04-15'
),
cust_emails as (
  with temp_output as (
select too.email as to_email, emailTranscript.from.email as from_email,
 
CASE WHEN SPLIT(emailTranscript.from.email,'@')[SAFE_OFFSET(1)] like '%tonik%' THEN 'outbound'
WHEN SPLIT(too.email,'@')[SAFE_OFFSET(1)] like '%tonik%' THEN 'inbound'
ELSE 'outbound'
END AS category,
creationTime
from `genesys_raw.emails`,
unnest(emailTranscript) emailTranscript,unnest(emailTranscript.to) too
)
SELECT
*,
CASE WHEN category = 'outbound' THEN to_email
WHEN category = 'inbound'THEN from_email
WHEN from_email not like '%tonik%' THEN from_email
END AS customer_email_address,
FROM temp_output
),
 
cust_mobile_genesys_data as (
SELECT
input_customers.customerid,
MAX(
    CASE
      WHEN DATE(call_history.callDatetime) between DATE_SUB(input_customers.snapshot_date, INTERVAL 90 DAY) and DATE(input_customers.snapshot_date) THEN 1
      ELSE 0
  END
    ) AS flag_contactable_last90D,
  count(
    CASE
      WHEN DATE(call_history.callDatetime) between DATE_SUB(input_customers.snapshot_date, INTERVAL 90 DAY) and DATE(input_customers.snapshot_date) THEN 1
      ELSE null
  END
    ) AS count_contactable_last90D,
COUNT(CASE WHEN calldirection = 'outbound' THEN 1 ELSE null END) AS outbound_call_count,
COUNT(CASE WHEN calldirection = 'inbound' THEN 1 ELSE null END) AS inbound_call_count,

FROM input_customers
LEFT JOIN (SELECT DISTINCT cust_id,mobile_no, valid_from_dt FROM `datalake_worktables.customer_mobile_mail_dtls`
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_id,mobile_no order by valid_from_dt asc) = 1
) mobile_details on input_customers.customerid = mobile_details.cust_id and DATE(mobile_details.valid_from_dt) < DATE(snapshot_date)
LEFT JOIN `risk_credit_mis.call_attempt_history_gensys` call_history ON RIGHT(mobile_details.mobile_no,10) = right(call_history.mobileNumber,10) and DATE(call_history.callDatetime) < DATE(input_customers.snapshot_date) and connected = 1 and mediaType = 'voice'
GROUP BY 1
),
cust_email_genesys_data as (
SELECT
input_customers.customerid,
COUNT(CASE WHEN category = 'outbound' THEN 1 ELSE null END) AS outbound_email_count,
COUNT(CASE WHEN category = 'inbound'  THEN 1 ELSE null END) AS inbound_email_count
FROM input_customers
LEFT JOIN (SELECT DISTINCT cust_id,email, valid_from_dt FROM `datalake_worktables.customer_mobile_mail_dtls`
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_id,email order by valid_from_dt asc) = 1
) email_details on input_customers.customerid = email_details.cust_id and DATE(email_details.valid_from_dt) < DATE(snapshot_date)
left JOIN cust_emails ON email_details.email = cust_emails.customer_email_address and DATE(cust_emails.creationTime) < DATE(input_customers.snapshot_date)
GROUP BY 1
)
SELECT
  input_customers.customerid,
  input_customers.snapshot_date,
 flag_contactable_last90D,
 count_contactable_last90D,
inbound_call_count,
outbound_call_count,
outbound_email_count,
inbound_email_count,
FROM input_customers
LEFT JOIN cust_mobile_genesys_data on cust_mobile_genesys_data.customerid = CAST(input_customers.customerid AS STRING) --and cust_mobile_genesys_data.snapshot_date = input_customers.snapshot_date
LEFT JOIN cust_email_genesys_data ON  cust_email_genesys_data.customerid = CAST(input_customers.customerid AS STRING) --and cust_email_genesys_data.snapshot_date = input_customers.snapshot_date
 ;
 """
 
job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema2}.{trench2neverappliedsnapshotcallcount} created successfully')

Table worktable_data_analysis.trench2_never_applied_snapshot_call_count_20210701_20250818 created successfully


# gamma_model_never_applied_snapshot

# gamma_model_never_applied_snapshot Table

In [30]:
schema2 = 'worktable_data_analysis'

gammamodelneverappliedsnapshot = f'gamma_model_never_applied_snapshot_{datetime.strptime(SDATE, '%Y-%m-%d').strftime('%Y%m%d')}_{datetime.strptime(EDATE, '%Y-%m-%d').strftime('%Y%m%d')}'

print(gammamodelneverappliedsnapshot)


gamma_model_never_applied_snapshot_20210701_20250818


# gamma_model_never_applied_snapshot Query

In [31]:
sq = f"""
create or replace table {schema2}.{gammamodelneverappliedsnapshot} as
SELECT 
a.customer_id,
null as digitalloanaccountid,
dob_snapshot_date,
days_since_credo_call_onb,
days_since_credo_call_snapshot_date,
onb_credolabRefNumber,
onb_credo_inquiry_date,
onb_email,
onb_age,
onb_city,
onb_province,
ln_industry_new,
ln_employment_type_new,
a.snapshot_date,
ln_nature_of_work_new,
ln_source_funds_new,
onb_first_name,
onb_middle_name,
onb_last_name,
ln_brand,
ln_cnt_dependents,
onb_doc_type,
onb_osversion onb_osversion,
c_credo_score,
cust_status_flag,
cust_status_close_date,
ln_self_dec_income,
ln_marital_status,
ln_education_level,
0 as ln_ever_applied_flag,
tx_first_product,
tx_first_product_user_segment,
ln_user_type,
ln_loan_type,
ln_prod_type ln_product_type,
CASE WHEN a.last_rejected_digitalloanaccountId is not null then 'Rejected_Before'
else 'Never_Applied_Before' end as existing_user_type,
onb_tsa_onboarding_datetime onboarding_date,
a.ln_loan_appln_time,
first_disb_loan_disbursed_time ln_loan_disb_time,
ln_mature_fspd30_flag,
--CASE WHEN LOANMATURITYDATE < secondDueDate THEN 0 ELSE ln_mature_fspd30_flag
--END AS  ln_mature_fspd30_flag,
ln_fspd30_flag,
ln_mature_fpd30_flag,
--CASE WHEN LOANMATURITYDATE < firstDueDate THEN 0 ELSE ln_mature_fpd30_flag
--END AS ln_mature_fpd30_flag,
ln_fpd30_flag,
first_applied_loan_appln_time,
first_applied_loan_decision,
first_applied_loan_type,
first_applied_product_type,
first_applied_loan_amount,
first_applied_loan_tenor,
last_applied_loan_appln_time,
last_applied_loan_decision,
last_applied_loan_type,
last_applied_product_type,
last_applied_loan_amount,
last_applied_loan_tenor,
last_applied_os_type,
last_applied_crif_id last_applied_digitalloanaccountId,
last_applied_crif_id,
last_applied_cic_score,
last_applied_credo_ref_no,
last_applied_credo_score,
last_applied_demo_score,
last_applied_apps_score,
onb_year,
onb_month_of_year,
onb_week_of_month,
onb_day_of_month,
onb_time_of_day,
tx_cnt_cash_in_total,
tx_cnt_cash_in_ob2t,
tx_cnt_cash_in_ot2t,
tx_amt_cash_in_total,
tx_amt_cash_in_ob2t,
tx_amt_cash_in_ot2t,
tx_cnt_cash_out_total,
tx_cnt_cash_out_billpay,
tx_cnt_cash_out_cards,
tx_cnt_cash_out_t2ot,
tx_cnt_cash_out_t2ob,
tx_amt_cash_out_total,
tx_amt_cash_out_billpay,
tx_amt_cash_out_cards,
tx_amt_cash_out_t2ot,
tx_amt_cash_out_t2ob,
tx_deposit_accnt_cnt,
tx_stash_accnt_opened_cnt,
tx_stash_accnt_closed_cnt,
tx_stash_balance,
tx_td_accnt_opened_cnt,
tx_td_accnt_completed_cnt,
tx_td_accnt_broken_cnt,
tx_td_auto_roll_over_enabled,
tx_td_balance,
td_max_duration tx_td_max_duration,
td_min_duration tx_td_min_duration,
td_avg_duration tx_td_avg_duration,
stash_max_duration tx_stash_max_duration,
stash_avg_duration tx_stash_avg_duration,
stash_min_duration tx_stash_min_duration,
tx_med_days_bw_new_dep_acct_open,
tx_med_days_bw_td_acct_open,
tx_avg_days_bt_trans,
tx_avg_days_bt_cash_in_trans,
tx_avg_days_bt_cash_out_trans,
tx_med_days_bt_trans,
tx_med_days_bt_cash_in_trans,
tx_med_days_bt_cash_out_trans,
tx_cnt_applied_loan,
COALESCE(tx_cnt_rejected_loans,0) tx_cnt_rejected_loans,
tx_cnt_approved_loans,
tx_cnt_disbursed_loans,
tx_cnt_completed_loans,
tx_cnt_active_loans,
tx_cnt_incomplete_loan_apps,
referral_flag onb_referral_flag,
flag_contactable_last90D cs_contactable_last_90d_flag,
ln_any_prev_disb_loan_sil_mobile_flag,
CASE WHEN appsflyer_install_to_registration_minutes < 0 THEN NULL
ELSE appsflyer_install_to_registration_minutes END appsflyer_install_to_registration_minutes,
meng_no_of_logins,
meng_ql_calculator_count,
meng_ql_calculator_tot_visit_cnt,
channel_source_group,
marketing_source_name,
outbound_call_count cs_cnt_outbound_calls,
inbound_call_count cs_cnt_inbound_calls,
outbound_email_count cs_cnt_outbound_emails,
inbound_email_count cs_cnt_inbound_emails,
FROM 
 {schema2}.{trench2neverappliedsnapshottransactions} a
LEFT JOIN {schema2}.{trench2neverappliedsnapshotevents} b on a.customer_id = b.customerid 
LEFT JOIN {schema2}.{trench2neverappliedsnapshotcallcount} contactability ON cast(contactability.customerid as string)= a.customer_id 
LEFT JOIN (select customer_id from {schema2}.{gammamodeleverappliedsnapshot}) ever_applied on ever_applied.customer_id = a.customer_id
where ever_applied.customer_id is null
 """
 
job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema2}.{gammamodelneverappliedsnapshot} created successfully')

Table worktable_data_analysis.gamma_model_never_applied_snapshot_20210701_20250818 created successfully


# App package Data

# App Package Tables

In [32]:
CURRENT_DATE = datetime.now().strftime("%Y%m%d")
print(f"The current date is: {CURRENT_DATE}")

SDATE = '20240101'
EDATE = '20250818'
OBDATE = '2025-08-18'


trench2everappliedsnapshotappscored = f'trench2_ever_applied_snapshot_{SDATE}_{EDATE}_app_scored'
trench2neverappliedsnapshotappscored = f'trench2_never_applied_snapshot_{SDATE}_{EDATE}_app_scored'

print(f"{trench2everappliedsnapshotappscored}, {trench2neverappliedsnapshotappscored}")


The current date is: 20250916
trench2_ever_applied_snapshot_20240101_20250818_app_scored, trench2_never_applied_snapshot_20240101_20250818_app_scored


# Gamma Trench2 Ever Applied App Package

In [ ]:
sq = f"""
create or replace table {schema1}.{trench2everappliedsnapshotappscored} as 
select distinct customerid,
digitalLoanAccountId,
A.deviceId,
credo_inquiry_date,
datetime(credo_inquiry_date,'Asia/Manila') credo_inquiry_date_manila,
ln_loan_appln_time,
app.package_name,
app.first_install_time,
app.last_update_time,
app.version_name,
app.flags application_flags 
from
(select distinct trench2.customer_id customerid, digitalLoanAccountId digitalLoanAccountId,
refno deviceId,createdOn credo_inquiry_date,ln_loan_appln_time, 
from 
(select userId,refno,deviceId,createdOn,B.cust_id from  `dl_loans_db_raw.tdbk_credolab_track` A  
join  `dl_customers_db_raw.tdbk_customer_mtb` B on A.userId=B.user_id )  tsa
join  ((select * from {schema2}.{gammamodeleverappliedsnapshot})) trench2
on tsa.cust_id=trench2.customer_id 
where  date(createdOn)<=date('{OBDATE}') 
qualify row_number()over (partition by cust_id order by date(createdOn) desc )=1)A
join (SELECT deviceId,A.package_name,A.first_install_time,A.last_update_time,A.version_name,A.flags 
FROM `prj-prod-dataplatform.credolab_raw.android_credolab_Application`
,unnest(Application) A
) app   on app.deviceId=A.deviceId;
"""

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema1}.{trench2everappliedsnapshotappscored} created successfully')

# Gamma Trench2 Never Applied App Package

In [ ]:
sq = f""" 
create or replace table {schema1}.{trench2neverappliedsnapshotappscored} as
select distinct customerid,
digitalLoanAccountId,
A.deviceId,
credo_inquiry_date
,datetime(credo_inquiry_date,'Asia/Manila') credo_inquiry_date_manila,
ln_loan_appln_time,
app.package_name,
app.first_install_time,
app.last_update_time,
app.version_name,
app.flags application_flags, 
from
(select distinct trench2.customer_id customerid,'NULL' digitalLoanAccountId,refno deviceId,createdOn credo_inquiry_date,ln_loan_appln_time, 
from 
(select userId,refno,deviceId,createdOn,B.cust_id 
    from  `dl_loans_db_raw.tdbk_credolab_track` A  
    join  `dl_customers_db_raw.tdbk_customer_mtb` B on A.userId=B.user_id )  tsa
join  ((select * from {schema2}.{gammamodelneverappliedsnapshot})) trench2
on tsa.cust_id=trench2.customer_id 
where  date(createdOn)<=date('{OBDATE}') 
qualify row_number()over (partition by cust_id order by date(createdOn) desc )=1)A
join (SELECT deviceId,A.package_name,A.first_install_time,A.last_update_time,A.version_name,A.flags 
FROM `prj-prod-dataplatform.credolab_raw.android_credolab_Application`
,unnest(Application) A
) app   on app.deviceId=A.deviceId;
"""

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema1}.{trench2neverappliedsnapshotappscored} created successfully')

# End